# 成績データファイルを作成する
## 元の成績データファイルへ新しい成績データを追加する

In [ ]:
import pandas as pd
import math
import numpy as np
from tkinter import Tk, filedialog
from datetime import datetime

# ファイル選択ダイアログを表示
root = Tk()
root.withdraw()  # メインウィンドウを非表示にする

master_file_path = r'D:\Keiba\10_ExportData\Race_Result_Master_org.csv'
merged_file_path = filedialog.askopenfilename(title="CSVファイルを選択してください", filetypes=[("CSV Files", "*.csv")])

# ファイルが選択されなかった場合は終了
if not merged_file_path:
    print("ファイルが選択されなかったため、処理を終了します。")
    exit()

#  CSVファイルを開く
master_df = pd.read_csv(master_file_path, encoding='cp932').copy()
merged_df = pd.read_csv(merged_file_path, encoding='cp932').copy()

# maseter_dfにmerged_dfを縦結合する
combined_df = pd.concat([master_df, merged_df], ignore_index=True)

# 重複行を削除する（全列が同じ行を削除）
combined_df.drop_duplicates(inplace=True)

# maseter_dfを上書き保存する
combined_df.to_csv(master_file_path, index=False, encoding='cp932')

C:\Users\rotte\AppData\Local\Temp\ipykernel_41740\163312551.py:20: DtypeWarning: Columns (21,22,90) have mixed types. Specify dtype option on import or set low_memory=False.
  master_df = pd.read_csv(master_file_path, encoding='cp932').copy()


# 基準タイムファイルを出力するコード

## ・スピード指数の算出に使うコース別の基準タイムファイルを出力する
- 1着馬および1着馬〜3着馬の基準タイム、およびその他タイム系データを格納する
- ファイルは2つ出力するが、スピード指数の算出には`00_StandardTimes2_yyyymmdd.csv`を用いる

### 処理の概要
- 成績CSVファイルを読み込み、コース・距離・競争種別コード・クラスコード・馬場状態でレースデータをグループ化する
- 各グループの中央値を基準タイムファイル1とする
- 各グループの中で1勝クラス・2勝クラスのデータの平均をとり、それを基準タイムファイル2とする

### ステップ
1. **CSVファイルの読み込み**
   - `pandas`を使って、ある期間の1着馬から3着馬の成績が記録されたCSVファイルを読み込む

2. **カテゴリの追加**
   - `競争種別コード`と`クラスコード`をもとにレースタイプやクラスカテゴリを新たに分類し、カテゴリカラムを追加
   - 馬場状態（良・稍／重・不良）に基づき馬場状態カテゴリを追加

3. **タイム系カラムの計算**
   - 1着馬から3着馬の走破タイムや3Fの平均を計算して新しいカラムとして追加

4. **グループ化して中央値を計算**
   - 「場所」、「芝・ダート」、「距離」、「馬場状態」、「競争種別カテゴリ」、「クラスカテゴリ」でグループ化
   - 各グループごとのタイム系カラムの中央値を計算し、基準タイムファイル1として保存

5. **クラス別フィルタリングと再計算**
   - 基準タイムファイル1から「良・稍」と「重・不良」でデータをフィルタリング
   - 各フィルタで1勝クラスと2勝クラスの中央値を取得し、その平均をとって基準タイムファイル2とする

6. **出力ファイルの生成**
   - `00_StandardTimes1_yyyymmdd.csv`（基準タイムファイル1）と`00_StandardTimes2_yyyymmdd.csv`（基準タイムファイル2）を出力
   - スピード指数の算出には`00_StandardTimes2_yyyymmdd.csv`を使用


In [6]:
from datetime import datetime
import pandas as pd
import numpy as np
import re
import os

#---ステップ１：成績データから基準タイムファイルを作成
# ファイルパスの指定
# 成績データファイルのパス
master_file_path = r'D:\Keiba\10_ExportData\ResultData\Original\Race_Result_Master_org.csv'

# 成績データにマージするデータファイルのパス
# 初角位置ファイルのパス
merge_filepath1 = r'D:\Keiba\00_ImportData\70_First_Corner_Position\First_Corner_Position_Master.csv'
# 2角位置ファイルのパス
merge_filepath2 = r'D:\Keiba\00_ImportData\80_Second_Corner_Position\Second_Corner_Position_Master.csv'
# 3角位置ファイルのパス
merge_filepath3 = r'D:\Keiba\00_ImportData\90_Third_Corner_Position\Third_Corner_Position_Master.csv'
# 4角位置ファイルのパス
merge_filepath4 = r'D:\Keiba\00_ImportData\100_Fourth_Corner_Position\Fourth_Corner_Position_Master.csv'
# 上り位置ファイルのパス
merge_filepath5 = r'D:\Keiba\00_ImportData\110_Spurt_Position\Spurt_Position_Master.csv'
# 馬場指数ファイルのパス
merge_filepath6 = r'D:\Keiba\00_ImportData\60_Track_Condition\Track_Condition_Master.csv'
# 前半3Fタイムファイルのパス
merge_filepath7 = r'D:\Keiba\00_ImportData\120_First3F_Lap\First3F_Lap_Master.csv'
# レイティングファイルのパス
merge_filepath8 = r'D:\Keiba\00_ImportData\200_Rating_Score\Rating_Score_Master.csv'

# 保存先のファイルパス
# 加工後の成績データファイルの保存先パス
output_filepath1 = r'D:\Keiba\10_ExportData\ResultData\Processed\Race_Result_Master.csv'
# 基準タイムファイル1の保存先パス
output_filepath2 = r'D:\Keiba\20_IndexData\Times\StdTime1.csv'
# 基準タイムファイル2の保存先パス
output_filepath3 = r'D:\Keiba\20_IndexData\Times\StdTime2.csv'
# ペース係数ファイルの保存先パス
output_filepath4 = r'D:\Keiba\20_IndexData\Times\PaceTime.csv'
# 基準33ラップファイルの保存先パス
output_filepath5 = r'D:\Keiba\20_IndexData\Times\33Lap.csv'
# レースレベル基準ファイルのパス
output_filepath6 = r'D:\Keiba\20_IndexData\Times\RaceLevel.csv'
# テン指数のファイルパス
output_filepath7 = r'D:\Keiba\00_ImportData\150_First_Score/'
# 上り指数のファイルパス
output_filepath8 = r'D:\Keiba\00_ImportData\160_Spurt_Score/'
# スピード指数のファイルパス
output_filepath9 = r'D:\Keiba\00_ImportData\170_Speed_Score/'
# 総合指数のファイルパス
output_filepath10 = r'D:\Keiba\00_ImportData\180_Total_Score/'
# 33ラップ判定のファイルパス
output_filepath11 = r'D:\Keiba\00_ImportData\140_33Lap_Category/'
# レースレベル判定のファイルパス
output_filepath12 = r'D:\Keiba\00_ImportData\190_Race_Level/'
# 馬タイプ分類ファイルのパス
output_filepath13 = r'D:\Keiba\00_ImportData\220_Horse_Type/Horse_Type_Master.csv'

# 成績データファイルとマージファイルの読み込み
df1 = pd.read_csv(master_file_path, encoding='cp932')
df2 = pd.read_csv(merge_filepath1, encoding='cp932')
df3 = pd.read_csv(merge_filepath2, encoding='cp932')
df4 = pd.read_csv(merge_filepath3, encoding='cp932')
df5 = pd.read_csv(merge_filepath4, encoding='cp932')
df6 = pd.read_csv(merge_filepath5, encoding='cp932')
df7 = pd.read_csv(merge_filepath6, encoding='cp932')
df8 = pd.read_csv(merge_filepath7, encoding='cp932')
df9 = pd.read_csv(merge_filepath8, encoding='cp932')

#　マージファイルで使用する列だけを定義
df2 = df2[['target_horseid','初角サイドポジション']]
df3 = df3[['target_horseid','2角サイドポジション']]
df4 = df4[['target_horseid','3角サイドポジション']]
df5 = df5[['target_horseid','4角サイドポジション']]
df6 = df6[['target_horseid','4角位置']]
df7 = df7[['target_raceid','馬場指数']]
df8 = df8[['target_horseid','前半3F']]
df9 = df9[['target_horseid','レイティング']]

# カラムの整理
# 成績データファイルの列名を変更
rename_map = {
    'レースID(新)': 'target_raceid',
    'レースID(新).1': 'target_horseid',
    '上り3F': 'レース上り3F',
    '上り4F': 'レース上り4F',
    '上り5F': 'レース上り5F',
    '外部指数1':'レイティング',
    '外部指数順1':'レイティング順位',
    '外部指数2':'ZI指数',
    '外部指数順2':'ZI指数順位',
    '外部指数3':'追切指数',
    '外部指数順3':'追切指数順位',
    '上り3F.1':'上り3F'
}

# df1の列名に対してrename_mapを適用
df1 = df1.rename(columns=rename_map)

# キーの型を揃える（数字/ゼロ埋めブレ防止）
for c in ['target_horseid', 'target_raceid']:
    if c in df1: df1[c] = df1[c].astype(str)
for d in [df2, df3, df4, df5, df6, df7, df8, df9]:
    for c in ['target_horseid', 'target_raceid']:
        if c in d: d[c] = d[c].astype(str)

#　種牡馬名のspace削除
df1['種牡馬'] = df1['種牡馬'].astype(str).str.replace(r'\s+', '', regex=True)

# ラベル化する列を追加
# コースラベル
insert_pos = df1.columns.get_loc('コースグループ名1') + 1
df1.insert(insert_pos, 'コースラベル', df1['場所'].astype(str) + '_' + df1['芝・ダート'].astype(str) + '_' + df1['トラックコード(JV)'].astype(str))

# 父×母の父タイプ名
insert_pos = df1.columns.get_loc('母の父タイプ名') + 1
df1.insert(insert_pos, '父×母の父タイプ名', df1['種牡馬'].astype(str) + '×' + df1['母の父タイプ名'].astype(str))

# 父タイプ名×母の父タイプ名
insert_pos = df1.columns.get_loc('父×母の父タイプ名') + 1
df1.insert(insert_pos, '父タイプ名×母の父タイプ名', df1['種牡馬タイプ名'].astype(str) + '×' + df1['母の父タイプ名'].astype(str))

# 生産者×馬主
insert_pos = df1.columns.get_loc('騎手') + 1
df1.insert(insert_pos, '生産者×馬主', df1['生産者'].astype(str) + '×' + df1['馬主'].astype(str))

# 生産者×調教師
insert_pos = df1.columns.get_loc('生産者×馬主') + 1
df1.insert(insert_pos, '生産者×調教師', df1['生産者'].astype(str) + '×' + df1['調教師'].astype(str))

# 生産者×騎手
insert_pos = df1.columns.get_loc('生産者×調教師') + 1
df1.insert(insert_pos, '生産者×騎手', df1['生産者'].astype(str) + '×' + df1['騎手'].astype(str))

# 馬主×調教師
insert_pos = df1.columns.get_loc('生産者×騎手') + 1
df1.insert(insert_pos, '馬主×調教師', df1['馬主'].astype(str) + '×' + df1['調教師'].astype(str))

# 馬主×騎手
insert_pos = df1.columns.get_loc('馬主×調教師') + 1
df1.insert(insert_pos, '馬主×騎手', df1['馬主'].astype(str) + '×' + df1['騎手'].astype(str))

# 調教師×騎手
insert_pos = df1.columns.get_loc('馬主×騎手') + 1
df1.insert(insert_pos, '調教師×騎手', df1['調教師'].astype(str) + '×' + df1['騎手'].astype(str))

# 調教師×コースラベル
insert_pos = df1.columns.get_loc('調教師×騎手') + 1
df1.insert(insert_pos, '調教師×コースラベル', df1['調教師'].astype(str) + '×' + df1['コースラベル'].astype(str))

# 騎手×コースラベル
insert_pos = df1.columns.get_loc('調教師×コースラベル') + 1
df1.insert(insert_pos, '騎手×コースラベル', df1['騎手'].astype(str) + '×' + df1['コースラベル'].astype(str))

# 成績データファイルへ対象データをマージする
# 初角位置のマージ
df1 = pd.merge(df1, df2, on='target_horseid', how='left')
# 2角位置のマージ
df1 = pd.merge(df1, df3, on='target_horseid', how='left')
# 3角位置のマージ
df1 = pd.merge(df1, df4, on='target_horseid', how='left')
# 4角位置のマージ
df1 = pd.merge(df1, df5, on='target_horseid', how='left')
# 上り位置のマージ
df1 = pd.merge(df1, df6, on='target_horseid', how='left')
# 馬場指数のマージ
df1 = pd.merge(df1, df7, on='target_raceid', how='left')
# 前半3Fのマージ
df1 = pd.merge(df1, df8, on='target_horseid', how='left')
# レイティングの代入
left  = df1.set_index('target_horseid')
right = df9[['target_horseid','レイティング']].dropna(subset=['レイティング']).set_index('target_horseid')
left.update(right, overwrite=True)
df1 = left.reset_index()

# --- 関数群 ---
def calculate_33_lap(df1):
    """距離ごとに33ラップを計算する"""
    rap_33 = []
    distance_to_lap_range = {
        1000: (0, 3),
        1200: (0, 3),
        1300: (1, 4),
        1400: (1, 4),
        1500: (2, 5),
        1600: (2, 5),
        1700: (3, 6),
        1800: (3, 6),
        1900: (4, 7),
        2000: (4, 7),
        2100: (5, 8),
        2200: (5, 8),
        2300: (6, 9),
        2400: (6, 9),
        2500: (7, 10),
        2600: (7, 10),
        2700: (8, 11),
        2800: (8, 11),
        2900: (9, 12),
        3000: (9, 12),
        3100: (10, 13),
        3200: (10, 13),
        3300: (11, 14),
        3400: (11, 14),
        3500: (12, 15),
        3600: (12, 15),
    }
    for _, row in df1.iterrows():
        distance = row['距離']
        lap_times = [row[f'Lap{str(i+1).zfill(2)}'] for i in range(25)]
        lap_times = [t for t in lap_times if not pd.isna(t)]
        if distance in distance_to_lap_range:
            start, end = distance_to_lap_range[distance]
            sum_6to4 = sum(lap_times[start:end])
        elif distance == 1150:
            sum_6to4 = round(lap_times[0] * 1.25, 1) + lap_times[1] + lap_times[2] if len(lap_times) >= 3 else None
        else:
            sum_6to4 = None
        sum_3to1 = row['レース上り3F'] if 'レース上り3F' in row and not pd.isna(row['レース上り3F']) else None
        rap_33_value = sum_6to4 - sum_3to1 if not pd.isna(sum_6to4) and not pd.isna(sum_3to1) else None
        rap_33.append(rap_33_value)
    df1['33ラップ'] = rap_33
    return df1

def calculate_middle_lap(df1):
    """距離ごとに中盤ラップ1・2を計算する"""
    middle_lap1 = []
    middle_lap2 = []
    distance_to_mid_lap = {
        1000: (2, 4, None, None),
        1150: (2, 4, None, None),
        1200: (2, 4, None, None),
        1300: (2, 4, None, None),
        1400: (2, 4, None, None),
        1500: (3, 5, None, None),
        1600: (3, 5, None, None),
        1700: (3, 6, None, None),
        1800: (3, 6, None, None),
        1900: (3, 7, None, None),
        2000: (3, 7, None, None),
        2100: (3, 5, 5, 8),
        2200: (3, 5, 5, 8),
        2300: (3, 5, 5, 8),
        2400: (3, 6, 6, 9),
        2500: (3, 6, 6, 9),
        2600: (3, 7, 7, 10),
        3000: (3, 8, 8, 12),
        3200: (3, 8, 8, 13),
        3400: (3, 9, 9, 14),
        3600: (3, 9, 9, 15),
    }
    for _, row in df1.iterrows():
        distance = row['距離']
        lap_times = [row[f'Lap{str(i+1).zfill(2)}'] for i in range(25)]
        lap_times = [t for t in lap_times if not pd.isna(t)]
        if distance in distance_to_mid_lap:
            mid1_start, mid1_end, mid2_start, mid2_end = distance_to_mid_lap[distance]
            mid1 = sum(lap_times[mid1_start:mid1_end]) if mid1_start is not None else None
            mid2 = sum(lap_times[mid2_start:mid2_end]) if mid2_start is not None else None
        else:
            mid1 = None
            mid2 = None
        middle_lap1.append(mid1)
        middle_lap2.append(mid2)
    df1['中盤ラップ1'] = middle_lap1
    df1['中盤ラップ2'] = middle_lap2
    return df1

def calculate_lap_features(df):
    """
    Lap01～Lap25 を使って
      ・最大加速（隣接ラップ差分の最小値）
      ・ゴール前ラップ差（ラスト1F - ラスト2F）
    を計算して df に列を追加する。
    """
    lap_cols = [f'Lap{str(i).zfill(2)}' for i in range(1, 26)]

    def _calc_row(row):
        # その馬のラップ一覧（NaN は除外）
        laps = []
        for c in lap_cols:
            if c in row.index and pd.notna(row[c]):
                laps.append(row[c])

        # ラップが1つ以下ならどっちも計算不能
        if len(laps) < 2:
            return pd.Series({'最大加速ラップ': np.nan, 'ゴール前ラップ差': np.nan})

        laps = np.array(laps, dtype=float)

        # 隣り合う差分（後ろ - 前）
        diffs = np.diff(laps)   # 例：Lap02-Lap01, Lap03-Lap02, ...

        # 最大加速 = 最もマイナスが大きい差分（＝最小値）
        max_accel = diffs.min() if len(diffs) > 0 else np.nan

        # 終盤ラップ差 = ラスト1F - ラスト2F
        last_diff = laps[-1] - laps[-2] if len(laps) >= 2 else np.nan

        return pd.Series({
            '最大加速ラップ': max_accel if pd.notna(max_accel) else np.nan,
            'ゴール前ラップ差': last_diff if pd.notna(last_diff) else np.nan
        })

    new_cols = df.apply(_calc_row, axis=1)
    df['最大加速ラップ'] = new_cols['最大加速ラップ'].round(1)
    df['ゴール前ラップ差'] = new_cols['ゴール前ラップ差'].round(1)

    return df

def calculate_days_since_birth(df1):
    """生後日数を計算する"""
    birth_days = []
    for _, row in df1.iterrows():
        race_date = datetime.strptime(row['日付S'], '%Y.%m.%d')
        birth_str = row['誕生日'].replace(" ", "").replace("日", "").replace("-", "")
        birth_month, birth_day = map(int, birth_str.replace("月", " ").split())
        birth_year = race_date.year - row['年齢']
        try:
            birth_date = datetime(birth_year, birth_month, birth_day)
        except ValueError:
            birth_date = datetime(birth_year, 2, 28)
        days_old = (race_date - birth_date).days
        birth_days.append(days_old)
    df1['生後日数'] = birth_days
    return df1

def calculate_distance_diff(df1):
    """前走距離との差を計算する"""
    df1['前走距離差'] = df1['距離'] - df1['前走距離']
    return df1

def calculate_firsthalf_diff(df1):
    """初角から4角位置の差分を計算"""
    df1['初角_4角差'] = df1.apply(lambda row:
        (row['通過1'] - row['通過4']) if pd.notna(row['通過1']) else \
        ((row['通過2'] - row['通過4']) if pd.notna(row['通過2']) else \
        ((row['通過3'] - row['通過4']) if pd.notna(row['通過3']) else np.nan))
    , axis=1).fillna(0)
    return df1

def calculate_goal_diff(df1):
    """4角から入線順位の差分を計算"""
    # 入線順位を一時的に数値化
    rank_num = pd.to_numeric(df1['入線順位'], errors='coerce')

    # 有効な順位（1以上）だけを判定するためのマスク
    valid_mask = rank_num >= 1

    # 出力列だけ作る
    df1['4角_入線順位差'] = np.nan

    # 有効な行だけ計算
    df1.loc[valid_mask, '4角_入線順位差'] = (
        pd.to_numeric(df1.loc[valid_mask, '通過4'], errors='coerce')
        - rank_num[valid_mask]
    )

    return df1

def calculate_sideposition(df1):
    """サイドポジションの平均を計算する"""
    cols = ['初角サイドポジション', '2角サイドポジション', '3角サイドポジション', '4角サイドポジション']
    df1['サイドポジション平均'] = df1[cols].mean(axis=1)
    return df1

def calculate_totalprize(df1):
    """獲得賞金を計算する"""
    df1['獲得賞金'] = df1['賞金'].fillna(0) + df1['付加賞金'].fillna(0)
    return df1

def convert_time_to_seconds(time_str):
    """タイム表記を秒数に変換"""
    try:
        parts = time_str.split(".")
        if len(parts) == 3:
            minutes, seconds, tenths = map(int, parts)
            total_seconds = minutes * 60 + seconds + tenths * 0.1
        else:
            return np.nan
        return total_seconds
    except:
        return np.nan

def remove_plus_sign(value):
    """数値データから `+` を削除して変換"""
    try:
        return float(str(value).replace("+", ""))
    except:
        return np.nan

def extract_weight(value):
    """斤量の数値部分だけを抽出"""
    try:
        match = re.search(r'\d+', str(value))
        return int(match.group()) if match else np.nan
    except:
        return np.nan

def calculate_corner_loss(row):
    """コーナーロスを計算"""
    corner_positions = [
        row.get('初角サイドポジション', 1) - 1,
        row.get('2角サイドポジション', 1) - 1,
        row.get('3角サイドポジション', 1) - 1,
        row.get('4角サイドポジション', 1) - 1
    ]
    total_distance_loss = sum(corner_positions) * 1.5  # m単位
    finish_time_seconds = row['タイムS']
    distance_m = row['距離']
    if pd.isna(finish_time_seconds) or pd.isna(distance_m) or finish_time_seconds == 0:
        return np.nan
    avg_speed = distance_m / finish_time_seconds if finish_time_seconds > 0 else np.nan
    corner_loss = total_distance_loss / avg_speed if avg_speed > 0 else 0
    return round(corner_loss, 2)

# 各列の整形
# 対象の列を数値変換
df1['馬場指数'] = df1['馬場指数'].astype(str).str.extract(r'(-?\d+)')[0].astype('Int64')
df1['レイティング'] = pd.to_numeric(df1['レイティング'], errors='coerce')
df1['体重'] = pd.to_numeric(df1['体重'], errors='coerce')
df1['Ave-3F'] = pd.to_numeric(df1['Ave-3F'], errors='coerce')
df1['上り3F'] = pd.to_numeric(df1['上り3F'], errors='coerce')

# 対象の列から記号を除去
for col in ['前後3F差', '前後4F差', '前後5F差', '増減']:
    df1[col] = df1[col].apply(remove_plus_sign)

df1['斤量'] = df1['斤量'].apply(extract_weight)

# タイムを秒数に変換
df1['タイムS'] = df1['タイムS'].apply(convert_time_to_seconds)
df1['-3Fタイム'] = df1['-3Fタイム'].apply(convert_time_to_seconds)

# 決め手列を変換(マップにない場合はNanにする)
lq_mapping = {
    '中団': '差し',
    '後方': '追込',
}

s = df1['決め手'].astype('string').str.strip()
df1['決め手'] = s.map(lq_mapping)

# 各種計算関数を順次実行
# 33ラップの計算
df1 = calculate_33_lap(df1)
# 中盤ラップ1・2の計算
df1 = calculate_middle_lap(df1)
# 最大加速ラップ・ゴール前ラップ差の計算
df1 = calculate_lap_features(df1)
# 生後日数の計算
df1 = calculate_days_since_birth(df1)
# 前走距離差の計算
df1 = calculate_distance_diff(df1)
# 初角から4角通過順位差の計算
df1 = calculate_firsthalf_diff(df1)
# 4角から入線順位差の計算
df1 = calculate_goal_diff(df1)
# サイドポジション平均値の計算
df1 = calculate_sideposition(df1)
# 獲得賞金の計算
df1 = calculate_totalprize(df1)
# 基準斤量の計算
df1['基準斤量'] = df1['斤量'] - df1['馬齢斤量差']
# RPCI差の計算
df1['RPCI差'] = df1['PCI'] - df1['レースPCI']
# コーナーロスの計算
df1['コーナーロス'] = df1.apply(calculate_corner_loss, axis=1)
# 補正走破タイムの計算
df1['補正走破タイム'] = df1['タイムS'] - df1['コーナーロス']
# スローorハイ関数の計算
df1['スローorハイ関数'] = df1['Ave-3F'] - df1['上り3F']

# マージ用の列作成処理
def categorize_race_type(race_type):
    if race_type == 11:
        return 'サラブレッド系2歳'
    elif race_type == 12:
        return 'サラブレッド系3歳'
    elif race_type >= 13:
        return 'サラブレッド系3歳以上'
    else:
        return np.nan

def categorize_class_code(class_code):
    if 7 <= class_code <= 15:
        return '新馬・未勝利'
    elif class_code == 23:
        return '1勝クラス'
    elif class_code == 43:
        return '2勝クラス'
    elif class_code == 67:
        return '3勝クラス'
    elif class_code >= 115:
        return 'OP・重賞'
    else:
        return np.nan

# 範囲定義
ranges = [ (-np.inf, -4.6), (-4.6, -3.6), (-3.6, -2.6), (-2.6, -1.6), (-1.6, -0.6), (-0.6, 0.6), (0.6, 1.6), (1.6, 2.6), (2.6, 3.6), (3.6, 4.6), (4.6, np.inf) ]

# 区間ラベル
def assign_range(value):
    if pd.isna(value):
        return np.nan
    for i, (lower, upper) in enumerate(ranges):
        if i < len(ranges) - 1:
            if lower <= value < upper:
                return f"{lower}～{upper}"
        else:
            # 最終区間（4.6～inf）は右も含める
            if lower <= value <= upper:
                return f"{lower}～{upper}"
    return np.nan  # 念のため

# 馬場分類・競走種別・クラス分類・スローor関数範囲の列を作成する
df1['馬場分類'] = df1['馬場状態'].replace({'良': '良・稍重','稍': '良・稍重','重': '重・不良','不': '重・不良'})
df1['競走種別'] = df1['年齢限定(競走種別コード)'].apply(categorize_race_type)
df1['クラス分類'] = df1['クラスコード'].apply(categorize_class_code)
df1['スローorハイ関数範囲'] = df1['スローorハイ関数'].apply(assign_range)

# --- 集計設定 ---
agg_cols1 = {
    'レースPCI': 'median',
    'PCI3': 'median',
    '通過3F': 'median',
    '通過4F': 'median',
    '通過5F': 'median',
    'レース上り3F': 'median',
    'レース上り4F': 'median',
    'レース上り5F': 'median',
    '中盤ラップ1': 'median',
    '中盤ラップ2': 'median',
    '33ラップ': 'median',
    '最大加速ラップ': 'median',
    'ゴール前ラップ差': 'median',
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'レイティング順位': 'median',
    'ZI指数': 'median',
    'ZI指数順位': 'median',
    'マイニング': 'median',
    'マイニング順位': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング順位': 'median',
    'PCI': 'median',
    'RPCI差': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '上り3F': 'median',
    '上り3F順位': 'median',
    'Ave-3F': 'median',
    '補正タイム': 'median',
    '補9': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '初角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}

agg_cols2 = {
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'レイティング順位': 'median',
    'ZI指数': 'median',
    'ZI指数順位': 'median',
    'マイニング': 'median',
    'マイニング順位': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング順位': 'median',
    'PCI': 'median',
    'RPCI差': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '上り3F': 'median',
    '上り3F順位': 'median',
    'Ave-3F': 'median',
    '補正タイム': 'median',
    '補9': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '初角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}

# --- 基準タイム算出処理1 ---

# 1勝クラス・2勝クラスのみ対象
base_df1 = df1[df1['クラスコード'].isin([23, 43])].copy()

# 1着～3着と1着馬のデータ抽出
df_tops1 = base_df1[base_df1['入線順位'].between(1, 3)].copy()
df_1st1 = base_df1[base_df1['入線順位'] == 1].copy()

# クラス分類の統一
df_tops1.loc[:, 'クラス分類'] = '1勝・2勝クラス'
df_1st1.loc[:, 'クラス分類'] = '1勝・2勝クラス'

# グループ化キー
group_cols1 = ['場所', '芝・ダート', '距離', 'トラックコード(JV)','馬場分類', 'クラス分類']

# --- 集計処理 ---
base_time_tops1 = df_tops1.groupby(group_cols1, dropna=False).agg(agg_cols1).reset_index()
base_time_1st1 = df_1st1.groupby(group_cols1, dropna=False).agg(agg_cols2).reset_index()
base_time_1st1 = base_time_1st1.rename(columns={col: f"{col}_1着" for col in agg_cols2.keys()})

# --- 結合 ---
base_time_df1 = pd.merge(base_time_tops1, base_time_1st1, on=group_cols1, how="left")

# --- 距離係数（速度換算） ---
base_time_df1['距離係数'] = (1 / base_time_df1['タイムS']) * 100

# --- 基準タイム算出処理2 ---

df_tops2 = df1[df1['入線順位'].between(1, 3)].copy()
df_1st2 = df1[df1['入線順位'] == 1].copy()

# グループ化キー
group_cols2 = ['場所', '芝・ダート', '距離','トラックコード(JV)', '競走種別', 'クラス分類', '馬場分類']

base_time_tops2 = df_tops2.groupby(group_cols2).agg(agg_cols1).reset_index()
base_time_1st2 = df_1st2.groupby(group_cols2).agg(agg_cols2).reset_index()
base_time_1st2 = base_time_1st2.rename(columns={col: f"{col}_1着" for col in agg_cols2.keys()})
base_time_df2 = pd.merge(base_time_tops2, base_time_1st2, on=group_cols2, how="left")

# --- 距離係数（速度換算） ---
base_time_df2['距離係数'] = (1 / base_time_df2['タイムS']) * 100

# --- ペース係数算出処理 ---

# 外れ値除外（IQR）
def remove_outliers(group):
    Q1 = group['タイムS'].quantile(0.25)
    Q3 = group['タイムS'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return group[(group['タイムS'] >= lower_bound) & (group['タイムS'] <= upper_bound)]

# グルーピング＆IQR除外
pace_df = (
    df1.groupby(['場所', '芝・ダート', '距離','トラックコード(JV)'])
       .apply(remove_outliers)
       .reset_index(drop=True)
       .copy()
)

# 区間別の中央値（タイムS & スローorハイ関数）
pace_medians_df = pace_df.groupby(
    ['場所', '芝・ダート', '距離','トラックコード(JV)', 'スローorハイ関数範囲']
).agg({
    'タイムS': 'median',
    'スローorハイ関数': 'median'
}).reset_index()

# --- ベースライン作成 ---
# 優先：-0.6～0.6（中庸帯）
# 代用：0.6～1.6 / -1.6～-0.6 の平均 → どちらか片方だけでも可
baseline_medians = {}
for (location, turf_dirt, distance,track_code), group_df in pace_medians_df.groupby(['場所', '芝・ダート', '距離','トラックコード(JV)']):
    baseline_row = group_df[group_df['スローorハイ関数範囲'] == '-0.6～0.6']
    if not baseline_row.empty:
        baseline_median = float(baseline_row['タイムS'].iloc[0])
    else:
        range_negative = group_df[group_df['スローorハイ関数範囲'] == '-1.6～-0.6']
        range_positive = group_df[group_df['スローorハイ関数範囲'] == '0.6～1.6']
        if not range_negative.empty and not range_positive.empty:
            baseline_median = (float(range_negative['タイムS'].median()) +
                               float(range_positive['タイムS'].median())) / 2.0
        elif not range_negative.empty:
            baseline_median = float(range_negative['タイムS'].median())
        elif not range_positive.empty:
            baseline_median = float(range_positive['タイムS'].median())
        else:
            baseline_median = np.nan
    baseline_medians[(location, turf_dirt, distance,track_code)] = baseline_median

# ベースラインとの差（スローorハイ関数差指数）
def calculate_difference(row):
    key = (row['場所'], row['芝・ダート'], row['距離'],row['トラックコード(JV)'])
    baseline_time = baseline_medians.get(key, np.nan)
    return row['タイムS'] - baseline_time if not pd.isna(baseline_time) else np.nan

pace_medians_df['スローorハイ関数差指数'] = pace_medians_df.apply(calculate_difference, axis=1)

# 係数計算（0除算・NaN安全＋クリップ）
def calculate_pace_adjustment_coefficient(row):
    val = row['スローorハイ関数']
    diff = row['スローorハイ関数差指数']
    if pd.notna(val) and val != 0 and pd.notna(diff):
        coefficient = round(diff / val, 2)
        return float(np.clip(coefficient, -3, 3))
    return 0.0

pace_medians_df['ペース補正係数'] = pace_medians_df.apply(calculate_pace_adjustment_coefficient, axis=1)

# --- 基準33ラップ算出処理 ---

lap33_df = (
    df1
      .dropna(subset=['33ラップ'])  # まず欠損を除外
      .groupby(['場所', '芝・ダート', '距離', 'トラックコード(JV)'], dropna=False)['33ラップ']
      .agg(['mean', 'std'])  # mean と std を同時に集計
      .reset_index()
).copy()

# 近似Zスコア用の基準値を計算
lap33_df['33ラップ±0'] = lap33_df['mean'].round(2)
lap33_df['33ラップ+1'] = (lap33_df['mean'] + lap33_df['std']).round(2)
lap33_df['33ラップ-1'] = (lap33_df['mean'] - lap33_df['std']).round(2)
lap33_df['33ラップ+2'] = (lap33_df['mean'] + 2 * lap33_df['std']).round(2)
lap33_df['33ラップ-2'] = (lap33_df['mean'] - 2 * lap33_df['std']).round(2)

# 必要なカラムだけ残す
lap33_df = lap33_df[
    ['場所', '芝・ダート', '距離', 'トラックコード(JV)',
     '33ラップ-2', '33ラップ-1', '33ラップ±0', '33ラップ+1', '33ラップ+2',
     'mean', 'std']
]

#---ステップ２：成績データへ指数の追加
# 共通関数：参照側DFのキー以外の列にサフィックスを付ける
def prepare_ref_df(df_ref, merge_keys, suffix):
    """
    マージ用の参照DFの列名に suffix を付ける（キー列以外すべて）
    """
    df_ref = df_ref.copy()
    rename_map = {
        c: f"{c}{suffix}"
        for c in df_ref.columns
        if c not in merge_keys
    }
    df_ref = df_ref.rename(columns=rename_map)
    return df_ref

# マージキー
merge_key1 = ['場所','芝・ダート','距離','トラックコード(JV)','馬場分類']
merge_key2 = ['場所','芝・ダート','距離','トラックコード(JV)','競走種別','クラス分類','馬場分類']
merge_key3 = ['場所','芝・ダート','距離','トラックコード(JV)','スローorハイ関数範囲']
merge_key4 = ['場所','芝・ダート','距離','トラックコード(JV)']

# 参照側の列名にsuffixを付与
ref_df1 = prepare_ref_df(base_time_df1, merge_key1, '_stdtime1').copy()
ref_df2 = prepare_ref_df(base_time_df2, merge_key2, '_stdtime2').copy()
ref_df3 = prepare_ref_df(pace_medians_df, merge_key3, '_paceindex').copy()
ref_df4 = prepare_ref_df(lap33_df, merge_key4, '_33lap').copy()

# df1へその他データフレームをマージ
merged_df1 = pd.merge(df1, ref_df1, on=merge_key1, how='left')
merged_df1 = pd.merge(merged_df1, ref_df2, on=merge_key2, how='left')
merged_df1 = pd.merge(merged_df1, ref_df3, on=merge_key3, how='left')
merged_df1 = pd.merge(merged_df1, ref_df4, on=merge_key4, how='left')

# 不要なデータフレームのクリア
del df1, ref_df1, ref_df2, ref_df3, ref_df4

# 各指数算出の前処理
# 基準タイム/距離係数のベース列を作る（Std1優先、なければStd2）
base_time = merged_df1['タイムS_stdtime1'].fillna(merged_df1['タイムS_stdtime2'])
dist_coef = merged_df1['距離係数_stdtime1'].fillna(merged_df1['距離係数_stdtime2'])

# 基準タイム差（秒）：基準タイム - 補正走破タイム
base_time_diff = base_time - merged_df1['補正走破タイム']

# 基準タイム差×距離係数
speed_core = base_time_diff * dist_coef

# 平均3F補正値
ave3f_base = merged_df1['Ave-3F_stdtime1'].fillna(merged_df1['Ave-3F_stdtime2'])
revi_Ave3F = ave3f_base - merged_df1['Ave-3F']

# 斤量補正値
revi_weight = merged_df1['斤量'] - merged_df1['基準斤量']

# クラス補正値（Std1が存在する行だけ有効。Std2代用行では0）
# ※ Std1が無い = その条件の1・2勝基準が作れない想定なので、クラス補正は入れない
revi_class = (merged_df1['タイムS_stdtime1'] - merged_df1['タイムS_stdtime2']).where(
    merged_df1['タイムS_stdtime1'].notna(), 0
)

# ペース補正値
revi_pace = merged_df1['スローorハイ関数'] * merged_df1['ペース補正係数_paceindex']

# 馬場指数秒数換算
merged_df1['馬場補正値'] = merged_df1['馬場指数'] / 10

# 初角位置
merged_df1['初角位置'] = (
    merged_df1['通過1']
      .fillna(merged_df1['通過2'])
      .fillna(merged_df1['通過3'])
      .fillna(merged_df1['通過4'])
)

# テン指数の計算
merged_df1['テン指数'] = round((
(merged_df1['前半3F_stdtime1'].fillna(merged_df1['前半3F_stdtime2']) - merged_df1['前半3F']) # 基準タイムとの比較評価
+ ((merged_df1['頭数'] - merged_df1['初角位置'] + 1) / merged_df1['頭数']) # 初角の位置取り評価
+ 50
), 1)

# 上り指数の計算
merged_df1['上り指数'] = round((
(merged_df1['上り3F_stdtime1'].fillna(merged_df1['上り3F_stdtime2']) - merged_df1['上り3F']) # 基準タイムとの比較評価
+ ((merged_df1['頭数']) - merged_df1['入線順位'] + 1) / merged_df1['頭数'] # 着順評価
+ (merged_df1['通過4'] - merged_df1['入線順位']) / merged_df1['頭数'] # ポジション押し上げ力評価
+ (merged_df1['頭数'] - merged_df1['通過4'] + 1) / merged_df1['頭数'] # 4角の位置取り評価
+ 50
), 1)

# スピード指数の計算 ---
merged_df1['スピード指数'] = (
    (
        speed_core
        + merged_df1['馬場補正値'].fillna(0)
        + revi_Ave3F.fillna(0)
        + revi_weight.fillna(0)
        + revi_class.fillna(0)
        + revi_pace.fillna(0)
        + 100
    )
    .round(1)
    .where(speed_core.notna() & (speed_core != 0))
)

# 総合指数の計算
merged_df1['総合指数'] = merged_df1[['スピード指数', '補正タイム', '補9']].mean(axis=1).round(1)

# 総合指数の代表値(1着～3着)をdf1へ追加
central_score_df1 = merged_df1[merged_df1['入線順位'].between(1, 3)].copy()
central_score_df1 = central_score_df1.groupby('target_raceid' ,as_index = False)['総合指数'].mean()
central_score_df1.rename(columns={'総合指数': 'Top3総合指数'}, inplace=True)
central_score_df1['Top3総合指数'] = round(central_score_df1['Top3総合指数'], 1)
merged_df1 = pd.merge(merged_df1, central_score_df1, on='target_raceid', how='left',suffixes=('', '_centralscore1')).copy()

# レイティングの平均値を成績データへ追加
merged_df1['レイティング'] = pd.to_numeric(merged_df1['レイティング'], errors='coerce')
central_score_df2 = merged_df1.groupby('target_raceid' ,as_index = False)['レイティング'].mean().copy()
central_score_df2.rename(columns={'レイティング': 'レイティング平均値'}, inplace=True)
central_score_df2['レイティング平均値'] = round(central_score_df2['レイティング平均値'], 1)
merged_df1 = pd.merge(merged_df1, central_score_df2, on='target_raceid', how='left' ,suffixes=('', '_centralscore2')).copy()

# 不要なデータフレームのクリア
del central_score_df1, central_score_df2

# 各指数の偏差値を計算
def deviation_in_race(series):
    mean = series.mean()
    std = series.std()
    if pd.isna(std) or std == 0:
        return pd.Series(np.nan, index=series.index)
    return ((series - mean) / std * 10 + 50)

# レース内偏差値
merged_df1['レイティング偏差値'] = (merged_df1.groupby('target_raceid')['レイティング'].transform(deviation_in_race).round(1))
merged_df1['ZI指数偏差値'] = (merged_df1.groupby('target_raceid')['ZI指数'].transform(deviation_in_race).round(1))
merged_df1['マイニング偏差値'] = (merged_df1.groupby('target_raceid')['マイニング'].transform(deviation_in_race).round(1))
merged_df1['対戦型マイニング偏差値'] = (merged_df1.groupby('target_raceid')['対戦型マイニング'].transform(deviation_in_race).round(1))
merged_df1['前半3F偏差値'] = merged_df1.groupby('target_raceid')['前半3F'].transform(lambda s: deviation_in_race(-s)).round(1)
merged_df1['上り3F偏差値'] = merged_df1.groupby('target_raceid')['上り3F'].transform(lambda s: deviation_in_race(-s)).round(1)
merged_df1['Ave-3F偏差値'] = merged_df1.groupby('target_raceid')['Ave-3F'].transform(lambda s: deviation_in_race(-s)).round(1)
merged_df1['テン指数偏差値'] = (merged_df1.groupby('target_raceid')['テン指数'].transform(deviation_in_race).round(1))
merged_df1['上り指数偏差値'] = (merged_df1.groupby('target_raceid')['上り指数'].transform(deviation_in_race).round(1))
merged_df1['スピード指数偏差値'] = (merged_df1.groupby('target_raceid')['スピード指数'].transform(deviation_in_race).round(1))
merged_df1['総合指数偏差値'] = (merged_df1.groupby('target_raceid')['総合指数'].transform(deviation_in_race).round(1))

# レース内順位（dense）
merged_df1['前半3F順位'] = (merged_df1.groupby('target_raceid')['前半3F'].rank(ascending=True, method='dense').astype('Int64'))
merged_df1['テン指数順位'] = (merged_df1.groupby('target_raceid')['テン指数'].rank(ascending=False, method='dense').astype('Int64'))
merged_df1['上り指数順位'] = (merged_df1.groupby('target_raceid')['上り指数'].rank(ascending=False, method='dense').astype('Int64'))
merged_df1['Ave-3F順位'] = (merged_df1.groupby('target_raceid')['Ave-3F'].rank(ascending=True, method='dense').astype('Int64'))
merged_df1['スピード指数順位'] = (merged_df1.groupby('target_raceid')['スピード指数'].rank(ascending=False, method='dense').astype('Int64'))
merged_df1['総合指数順位'] = (merged_df1.groupby('target_raceid')['総合指数'].rank(ascending=False, method='dense').astype('Int64'))

# 順位分布の計算
merged_df1['レイティング順位分布'] = (merged_df1['レイティング順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['ZI指数順位分布'] = (merged_df1['ZI指数順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['マイニング順位分布'] = (merged_df1['マイニング順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['対戦型マイニング順位分布'] = (merged_df1['対戦型マイニング順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['前半3F順位分布'] = (merged_df1['前半3F順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['テン指数順位分布'] = (merged_df1['テン指数順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['上り3F順位分布'] = (merged_df1['上り3F順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['上り指数順位分布'] = (merged_df1['上り指数順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['Ave-3F順位分布'] = (merged_df1['Ave-3F順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['スピード指数順位分布'] = (merged_df1['スピード指数順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['総合指数順位分布'] = (merged_df1['総合指数順位'] - 1) / (merged_df1['頭数'] - 1)

# 33ラップ判定用関数
def assign_label(row):
    lap_value = row['33ラップ']
    if pd.isna(lap_value):
        return np.nan

    candidates = {
        -2: row['33ラップ-2_33lap'],
        -1: row['33ラップ-1_33lap'],
         0: row['33ラップ±0_33lap'],
         1: row['33ラップ+1_33lap'],
         2: row['33ラップ+2_33lap'],
    }

    # 参照側が全部NaNなら判定不能
    if all(pd.isna(v) for v in candidates.values()):
        return np.nan

    # 差分（絶対値）が最小のスケールを選ぶ
    best_scale = min(
        candidates.keys(),
        key=lambda k: abs(lap_value - candidates[k]) if pd.notna(candidates[k]) else np.inf
    )

    # 0スケール
    if best_scale == 0:
        if lap_value < 0:
            return '持0'
        elif lap_value > 0:
            return '瞬0'
        else:
            return '総'

    prefix = '瞬' if lap_value > 0 else '持'
    suffix = f'+{best_scale}' if best_scale > 0 else f'{best_scale}'
    return prefix + suffix

# 33ラップを判定
merged_df1['33ラップ判定'] = merged_df1.apply(assign_label, axis=1)

# レースタイプラベル列の追加
insert_pos = merged_df1.columns.get_loc('33ラップ判定') + 1

# '33ラップ判定' の値 → レースタイプ の対応表
race_type_map = {
    # 瞬発力戦
    '瞬+2': '瞬発力戦',
    '瞬+1': '瞬発力戦',
    '瞬0':  '瞬発力戦',

    # 総合力戦
    '瞬-1': '総合力戦',
    '瞬-2': '総合力戦',
    '総':   '総合力戦',
    '持-1': '総合力戦',
    '持-2': '総合力戦',

    # 持久力戦
    '持+2': '持久力戦',
    '持+1': '持久力戦',
    '持+0': '持久力戦',
}

ref = merged_df1['33ラップ判定']
merged_df1.insert(insert_pos, 'レースタイプ', ref.map(race_type_map))

# ラベル書き換え用のマッピング辞書
label_mapping = {
    '持-2': '0T持-2',
    '持-1': '0T持-1',
    '持0': '0U持0',
    '持+1': '0V持+1',
    '持+2': '0V持+2',
    '瞬-2': '0S瞬-2',
    '瞬-1': '0S瞬-1',
    '瞬0': '0R瞬0',
    '瞬+1': '0Q瞬+1',
    '瞬+2': '0Q瞬+2',
    '総': '02総'
}

# 書き換え
merged_df1['レース印２'] = merged_df1['33ラップ判定'].replace(label_mapping)

# レース強度指数の計算
first_load  = merged_df1['通過3F_stdtime1'].fillna(merged_df1['通過3F_stdtime2']) - merged_df1['通過3F']

middle_diff1 = merged_df1['中盤ラップ1_stdtime1'].fillna(merged_df1['中盤ラップ1_stdtime2']) - merged_df1['中盤ラップ1']
middle_diff2 = merged_df1['中盤ラップ2_stdtime1'].fillna(merged_df1['中盤ラップ2_stdtime2']) - merged_df1['中盤ラップ2']

middle_load = pd.concat([middle_diff1, middle_diff2], axis=1).mean(axis=1, skipna=True)

last_load   = merged_df1['レース上り3F_stdtime1'].fillna(merged_df1['レース上り3F_stdtime2']) - merged_df1['レース上り3F']
spurt_load  = merged_df1['最大加速ラップ_stdtime1'].fillna(merged_df1['最大加速ラップ_stdtime2']) - merged_df1['最大加速ラップ']
goal_load   = merged_df1['ゴール前ラップ差_stdtime1'].fillna(merged_df1['ゴール前ラップ差_stdtime2']) - merged_df1['ゴール前ラップ差']

merged_df1['レース強度指数'] = (
    100
    + first_load.fillna(0)
    + middle_load.fillna(0)
    + last_load.fillna(0)
    + spurt_load.fillna(0)
    + goal_load.fillna(0)
    + merged_df1['馬場補正値'].fillna(0)
)

# サフィックス付き列を削除
# 削除するサフィックスを列挙
suffixes_to_drop = ('_1着','_stdtime1', '_stdtime2', '_paceindex', '_33lap', '_centralscore1', '_centralscore2')

# サフィックスで判定して残す列だけ抜き出し
merged_df1 = merged_df1.loc[:, [c for c in merged_df1.columns
                              if not any(c.endswith(suf) for suf in suffixes_to_drop)]].copy()

#---ステップ３：基準タイムファイル１・２の再集計
# --- 集計設定 ---
agg_cols3 = {
    'レースPCI': 'median',
    'PCI3': 'median',
    '通過3F': 'median',
    '通過4F': 'median',
    '通過5F': 'median',
    'レース上り3F': 'median',
    'レース上り4F': 'median',
    'レース上り5F': 'median',
    '中盤ラップ1': 'median',
    '中盤ラップ2': 'median',
    '33ラップ': 'median',
    '最大加速ラップ': 'median',
    'ゴール前ラップ差': 'median',
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'レイティング偏差値': 'median',
    'レイティング順位': 'median',
    'レイティング順位分布':'median',
    'ZI指数': 'median',
    'ZI指数順位': 'median',
    'ZI指数偏差値': 'median',
    'ZI指数順位分布': 'median',
    'マイニング': 'median',
    'マイニング偏差値': 'median',
    'マイニング順位': 'median',
    'マイニング順位分布': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング偏差値': 'median',
    '対戦型マイニング順位': 'median',
    '対戦型マイニング順位分布': 'median',
    'PCI': 'median',
    'RPCI差': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '前半3F偏差値': 'median',
    '前半3F順位': 'median',
    '前半3F順位分布': 'median',
    '上り3F': 'median',
    '上り3F偏差値': 'median',
    '上り3F順位': 'median',
    '上り3F順位分布': 'median',
    'Ave-3F': 'median',
    'Ave-3F偏差値': 'median',
    'Ave-3F順位': 'median',
    'Ave-3F順位分布': 'median',
    '-3F差': 'median',
    'テン指数': 'median',
    'テン指数偏差値': 'median',
    'テン指数順位': 'median',
    'テン指数順位分布': 'median',
    '上り指数': 'median',
    '上り指数偏差値': 'median',
    '上り指数順位': 'median',
    '上り指数順位分布': 'median',
    '補正タイム': 'median',
    '補9': 'median',
    'スピード指数': 'median',
    'スピード指数偏差値': 'median',
    'スピード指数順位': 'median',
    'スピード指数順位分布': 'median',
    '総合指数': 'median',
    '総合指数偏差値': 'median',
    '総合指数順位': 'median',
    '総合指数順位分布': 'median',
    'レース強度指数': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '初角位置': 'median',
    '初角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}

agg_cols4 = {
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'レイティング偏差値': 'median',
    'レイティング順位': 'median',
    'レイティング順位分布':'median',
    'ZI指数': 'median',
    'ZI指数順位': 'median',
    'ZI指数偏差値': 'median',
    'ZI指数順位分布': 'median',
    'マイニング': 'median',
    'マイニング偏差値': 'median',
    'マイニング順位': 'median',
    'マイニング順位分布': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング偏差値': 'median',
    '対戦型マイニング順位': 'median',
    '対戦型マイニング順位分布': 'median',
    'PCI': 'median',
    'RPCI差': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '前半3F偏差値': 'median',
    '前半3F順位': 'median',
    '前半3F順位分布': 'median',
    '上り3F': 'median',
    '上り3F偏差値': 'median',
    '上り3F順位': 'median',
    '上り3F順位分布': 'median',
    'Ave-3F': 'median',
    'Ave-3F偏差値': 'median',
    'Ave-3F順位': 'median',
    'Ave-3F順位分布': 'median',
    '-3F差': 'median',
    'テン指数': 'median',
    'テン指数偏差値': 'median',
    'テン指数順位': 'median',
    'テン指数順位分布': 'median',
    '上り指数': 'median',
    '上り指数偏差値': 'median',
    '上り指数順位': 'median',
    '上り指数順位分布': 'median',
    'スピード指数': 'median',
    'スピード指数偏差値': 'median',
    'スピード指数順位': 'median',
    'スピード指数順位分布': 'median',
    '補正タイム': 'median',
    '補9': 'median',
    '総合指数': 'median',
    '総合指数偏差値': 'median',
    '総合指数順位': 'median',
    '総合指数順位分布': 'median',
    'レース強度指数': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '初角位置': 'median',
    '初角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}

# --- 基準タイム算出処理2 ---
# 1勝クラス・2勝クラスのみ対象
base_df1 = merged_df1[merged_df1['クラスコード'].isin([23, 43])].copy()

# 1着～3着と1着馬のデータ抽出
df_tops1 = base_df1[base_df1['入線順位'].between(1, 3)].copy()
df_1st1 = base_df1[base_df1['入線順位'] == 1].copy()

# クラス分類の統一
df_tops1.loc[:, 'クラス分類'] = '1勝・2勝クラス'
df_1st1.loc[:, 'クラス分類'] = '1勝・2勝クラス'

# --- 集計処理 ---
base_time_tops1 = df_tops1.groupby(group_cols1, dropna=False).agg(agg_cols3).reset_index()
base_time_1st1 = df_1st1.groupby(group_cols1, dropna=False).agg(agg_cols4).reset_index()
base_time_1st1 = base_time_1st1.rename(columns={col: f"{col}_1着" for col in agg_cols4.keys()})

# --- 結合 ---
base_time_df1 = pd.merge(base_time_tops1, base_time_1st1, on=group_cols1, how="left")

# --- 距離係数（速度換算） ---
base_time_df1['距離係数'] = (1 / base_time_df1['タイムS']) * 100

# --- 基準タイム算出処理2 ---
df_tops2 = merged_df1[merged_df1['入線順位'].between(1, 3)].copy()
df_1st2 = merged_df1[merged_df1['入線順位'] == 1].copy()

base_time_tops2 = df_tops2.groupby(group_cols2).agg(agg_cols3).reset_index()
base_time_1st2 = df_1st2.groupby(group_cols2).agg(agg_cols4).reset_index()
base_time_1st2 = base_time_1st2.rename(columns={col: f"{col}_1着" for col in agg_cols4.keys()})
base_time_df2 = pd.merge(base_time_tops2, base_time_1st2, on=group_cols2, how="left")

# --- 距離係数（速度換算） ---
base_time_df2['距離係数'] = (1 / base_time_df2['タイムS']) * 100

#---ステップ４：成績データへレースレベル指数の追加
# 再集計した基準DFを suffix 付きで準備
std1_pre = prepare_ref_df(base_time_df1, merge_key1, '_stdtime1_new')
std2_pre = prepare_ref_df(base_time_df2, merge_key2, '_stdtime2_new')

# merged_df1にマージ
merged_df1 = pd.merge(merged_df1, std1_pre, on=merge_key1, how='left')
merged_df1 = pd.merge(merged_df1, std2_pre, on=merge_key2, how='left')

# 不要なデータフレームのクリア
del std1_pre, std2_pre

# ベース値（Std2優先 → なければ Std1）
rating_base = merged_df1['レイティング_stdtime2_new'].fillna(
    merged_df1['レイティング_stdtime1_new']
)
speed_base = merged_df1['総合指数_stdtime2_new'].fillna(
    merged_df1['総合指数_stdtime1_new']
)
strength_base = merged_df1['レース強度指数_stdtime2_new'].fillna(
    merged_df1['レース強度指数_stdtime1_new']
)

# 差分（NaN は 0 扱い）
rating_diff = (merged_df1['レイティング平均値'] - rating_base).fillna(0)
speed_diff  = (merged_df1['Top3総合指数'] - speed_base).fillna(0)
strength_diff = (merged_df1['レース強度指数'] - strength_base).fillna(0)

# レースレベル指数
merged_df1['レースレベル指数'] = (rating_diff + speed_diff + strength_diff + 100).round(1)

#---ステップ５：各指数の差分を成績データへ追加
# Top3総合指数と総合指数の差分を成績データへ追加
merged_df1['Top3総合指数差分'] = merged_df1['総合指数'] - merged_df1['Top3総合指数']

# レイティング平均値とレイティングの差分を追加
merged_df1['平均レイティング差分'] = merged_df1['レイティング'] - merged_df1['レイティング平均値']

# レース強度指数：基準タイム１差分
merged_df1['レース強度指数上位差分']=merged_df1['レース強度指数']-merged_df1['レース強度指数_stdtime1_new'].fillna(merged_df1['レース強度指数_stdtime2_new'])

# PCI3：基準タイム１差分
merged_df1['PCI3差分']=merged_df1['PCI3']-merged_df1['PCI3_stdtime1_new'].fillna(merged_df1['PCI3_stdtime2_new'])

# PCI：基準タイム１差分
merged_df1['PCI上位差分']=merged_df1['PCI']-merged_df1['PCI_stdtime1_new'].fillna(merged_df1['PCI_stdtime2_new'])
merged_df1['PCI勝馬差分']=merged_df1['PCI']-merged_df1['PCI_1着_stdtime1_new'].fillna(merged_df1['PCI_1着_stdtime2_new'])

# 生後日数：基準タイム２差分
merged_df1['生後日数上位差分']=merged_df1['生後日数']-merged_df1['生後日数_stdtime2_new'].fillna(merged_df1['生後日数_stdtime1_new'])
merged_df1['生後日数勝馬差分']=merged_df1['生後日数']-merged_df1['生後日数_1着_stdtime2_new'].fillna(merged_df1['生後日数_1着_stdtime1_new'])

# 馬体重：基準タイム２差分
merged_df1['体重上位差分']=merged_df1['体重']-merged_df1['体重_stdtime2_new'].fillna(merged_df1['体重_stdtime1_new'])
merged_df1['体重勝馬差分']=merged_df1['体重']-merged_df1['体重_1着_stdtime2_new'].fillna(merged_df1['体重_1着_stdtime1_new'])

# 馬体重斤量比：基準タイム２差分
merged_df1['斤量馬体重比上位差分']=merged_df1['斤量馬体重比']-merged_df1['斤量馬体重比_stdtime2_new'].fillna(merged_df1['斤量馬体重比_stdtime1_new'])
merged_df1['斤量馬体重比勝馬差分']=merged_df1['斤量馬体重比']-merged_df1['斤量馬体重比_1着_stdtime2_new'].fillna(merged_df1['斤量馬体重比_1着_stdtime1_new'])

# 前走距離：基準タイム１差分
merged_df1['前走距離上位差分']=merged_df1['前走距離']-merged_df1['前走距離_stdtime1_new'].fillna(merged_df1['前走距離_stdtime2_new'])
merged_df1['前走距離勝馬差分']=merged_df1['前走距離']-merged_df1['前走距離_1着_stdtime1_new'].fillna(merged_df1['前走距離_1着_stdtime2_new'])

# 前走距離差：基準タイム１差分
merged_df1['前走距離差上位差分']=merged_df1['前走距離差']-merged_df1['前走距離差_stdtime1_new'].fillna(merged_df1['前走距離差_stdtime2_new'])
merged_df1['前走距離差勝馬差分']=merged_df1['前走距離差']-merged_df1['前走距離差_1着_stdtime1_new'].fillna(merged_df1['前走距離差_1着_stdtime2_new'])

# レイティング：基準タイム１差分
merged_df1['レイティング上位差分']=merged_df1['レイティング']-merged_df1['レイティング_stdtime1_new'].fillna(merged_df1['レイティング_stdtime2_new'])
merged_df1['レイティング勝馬差分']=merged_df1['レイティング']-merged_df1['レイティング_1着_stdtime1_new'].fillna(merged_df1['レイティング_1着_stdtime2_new'])

# レイティング偏差値：基準タイム１差分
merged_df1['レイティング偏差値上位差分']=merged_df1['レイティング偏差値']-merged_df1['レイティング偏差値_stdtime1_new'].fillna(merged_df1['レイティング偏差値_stdtime2_new'])
merged_df1['レイティング偏差値勝馬差分']=merged_df1['レイティング偏差値']-merged_df1['レイティング偏差値_1着_stdtime1_new'].fillna(merged_df1['レイティング偏差値_1着_stdtime2_new'])

# レイティング順位分布：基準タイム１差分
merged_df1['レイティング順位分布上位差分']=merged_df1['レイティング順位分布']-merged_df1['レイティング順位分布_stdtime1_new'].fillna(merged_df1['レイティング順位分布_stdtime2_new'])
merged_df1['レイティング順位分布勝馬差分']=merged_df1['レイティング順位分布']-merged_df1['レイティング順位分布_1着_stdtime1_new'].fillna(merged_df1['レイティング順位分布_1着_stdtime2_new'])

# ZI指数：基準タイム１差分
merged_df1['ZI指数上位差分']=merged_df1['ZI指数']-merged_df1['ZI指数_stdtime1_new'].fillna(merged_df1['ZI指数_stdtime2_new'])
merged_df1['ZI指数勝馬差分']=merged_df1['ZI指数']-merged_df1['ZI指数_1着_stdtime1_new'].fillna(merged_df1['ZI指数_1着_stdtime2_new'])

# ZI指数偏差値：基準タイム１差分
merged_df1['ZI指数偏差値上位差分']=merged_df1['ZI指数偏差値']-merged_df1['ZI指数偏差値_stdtime1_new'].fillna(merged_df1['ZI指数偏差値_stdtime2_new'])
merged_df1['ZI指数偏差値勝馬差分']=merged_df1['ZI指数偏差値']-merged_df1['ZI指数偏差値_1着_stdtime1_new'].fillna(merged_df1['ZI指数偏差値_1着_stdtime2_new'])

# ZI指数順位分布：基準タイム１差分
merged_df1['ZI指数順位分布上位差分']=merged_df1['ZI指数順位分布']-merged_df1['ZI指数順位分布_stdtime1_new'].fillna(merged_df1['ZI指数順位分布_stdtime2_new'])
merged_df1['ZI指数順位分布勝馬差分']=merged_df1['ZI指数順位分布']-merged_df1['ZI指数順位分布_1着_stdtime1_new'].fillna(merged_df1['ZI指数順位分布_1着_stdtime2_new'])

# マイニング：基準タイム１差分
merged_df1['マイニング上位差分']=merged_df1['マイニング']-merged_df1['マイニング_stdtime1_new'].fillna(merged_df1['マイニング_stdtime2_new'])
merged_df1['マイニング勝馬差分']=merged_df1['マイニング']-merged_df1['マイニング_1着_stdtime1_new'].fillna(merged_df1['マイニング_1着_stdtime2_new'])

# マイニング偏差値：基準タイム１差分
merged_df1['マイニング偏差値上位差分']=merged_df1['マイニング偏差値']-merged_df1['マイニング偏差値_stdtime1_new'].fillna(merged_df1['マイニング偏差値_stdtime2_new'])
merged_df1['マイニング偏差値勝馬差分']=merged_df1['マイニング偏差値']-merged_df1['マイニング偏差値_1着_stdtime1_new'].fillna(merged_df1['マイニング偏差値_1着_stdtime2_new'])

# マイニング順位分布：基準タイム１差分
merged_df1['マイニング順位分布上位差分']=merged_df1['マイニング順位分布']-merged_df1['マイニング順位分布_stdtime1_new'].fillna(merged_df1['マイニング順位分布_stdtime2_new'])
merged_df1['マイニング順位分布勝馬差分']=merged_df1['マイニング順位分布']-merged_df1['マイニング順位分布_1着_stdtime1_new'].fillna(merged_df1['マイニング順位分布_1着_stdtime2_new'])

# 対戦型マイニング：基準タイム１差分
merged_df1['対戦型マイニング上位差分']=merged_df1['対戦型マイニング']-merged_df1['対戦型マイニング_stdtime1_new'].fillna(merged_df1['対戦型マイニング_stdtime2_new'])
merged_df1['対戦型マイニング勝馬差分']=merged_df1['対戦型マイニング']-merged_df1['対戦型マイニング_1着_stdtime1_new'].fillna(merged_df1['対戦型マイニング_1着_stdtime2_new'])

# 対戦型マイニング偏差値：基準タイム１差分
merged_df1['対戦型マイニング偏差値上位差分']=merged_df1['対戦型マイニング偏差値']-merged_df1['対戦型マイニング偏差値_stdtime1_new'].fillna(merged_df1['対戦型マイニング偏差値_stdtime2_new'])
merged_df1['対戦型マイニング偏差値勝馬差分']=merged_df1['対戦型マイニング偏差値']-merged_df1['対戦型マイニング偏差値_1着_stdtime1_new'].fillna(merged_df1['対戦型マイニング偏差値_1着_stdtime2_new'])

# 対戦型マイニング順位分布：基準タイム１差分
merged_df1['対戦型マイニング順位分布上位差分']=merged_df1['対戦型マイニング順位分布']-merged_df1['対戦型マイニング順位分布_stdtime1_new'].fillna(merged_df1['対戦型マイニング順位分布_stdtime2_new'])
merged_df1['対戦型マイニング順位分布勝馬差分']=merged_df1['対戦型マイニング順位分布']-merged_df1['対戦型マイニング順位分布_1着_stdtime1_new'].fillna(merged_df1['対戦型マイニング順位分布_1着_stdtime2_new'])

# タイムS：基準タイム１差分
merged_df1['タイムS上位差分']=merged_df1['タイムS']-merged_df1['タイムS_stdtime1_new'].fillna(merged_df1['タイムS_stdtime2_new'])
merged_df1['タイムS勝馬差分']=merged_df1['タイムS']-merged_df1['タイムS_1着_stdtime1_new'].fillna(merged_df1['タイムS_1着_stdtime2_new'])

# 補正走破タイム：基準タイム１差分
merged_df1['補正走破タイム上位差分']=merged_df1['補正走破タイム']-merged_df1['補正走破タイム_stdtime1_new'].fillna(merged_df1['補正走破タイム_stdtime2_new'])
merged_df1['補正走破タイム勝馬差分']=merged_df1['補正走破タイム']-merged_df1['補正走破タイム_1着_stdtime1_new'].fillna(merged_df1['補正走破タイム_1着_stdtime2_new'])

# -3Fタイム：基準タイム１差分
merged_df1['-3Fタイム上位差分']=merged_df1['-3Fタイム']-merged_df1['-3Fタイム_stdtime1_new'].fillna(merged_df1['-3Fタイム_stdtime2_new'])
merged_df1['-3Fタイム勝馬差分']=merged_df1['-3Fタイム']-merged_df1['-3Fタイム_1着_stdtime1_new'].fillna(merged_df1['-3Fタイム_1着_stdtime2_new'])

# 前半3F：基準タイム１差分
merged_df1['前半3F上位差分']=merged_df1['前半3F']-merged_df1['前半3F_stdtime1_new'].fillna(merged_df1['前半3F_stdtime2_new'])
merged_df1['前半3F勝馬差分']=merged_df1['前半3F']-merged_df1['前半3F_1着_stdtime1_new'].fillna(merged_df1['前半3F_1着_stdtime2_new'])

# 前半3F偏差値：基準タイム１差分
merged_df1['前半3F偏差値上位差分']=merged_df1['前半3F偏差値']-merged_df1['前半3F偏差値_stdtime1_new'].fillna(merged_df1['前半3F偏差値_stdtime2_new'])
merged_df1['前半3F偏差値勝馬差分']=merged_df1['前半3F偏差値']-merged_df1['前半3F偏差値_1着_stdtime1_new'].fillna(merged_df1['前半3F偏差値_1着_stdtime2_new'])

# 前半3F順位：基準タイム１差分
merged_df1['前半3F順位上位差分']=merged_df1['前半3F順位']-merged_df1['前半3F順位_stdtime1_new'].fillna(merged_df1['前半3F順位_stdtime2_new'])
merged_df1['前半3F順位勝馬差分']=merged_df1['前半3F順位']-merged_df1['前半3F順位_1着_stdtime1_new'].fillna(merged_df1['前半3F順位_1着_stdtime2_new'])

# 前半3F順位分布：基準タイム１差分
merged_df1['前半3F順位分布上位差分']=merged_df1['前半3F順位分布']-merged_df1['前半3F順位分布_stdtime1_new'].fillna(merged_df1['前半3F順位分布_stdtime2_new'])
merged_df1['前半3F順位分布勝馬差分']=merged_df1['前半3F順位分布']-merged_df1['前半3F順位分布_1着_stdtime1_new'].fillna(merged_df1['前半3F順位分布_1着_stdtime2_new'])

# 上り3F：基準タイム１差分
merged_df1['上り3F上位差分']=merged_df1['上り3F']-merged_df1['上り3F_stdtime1_new'].fillna(merged_df1['上り3F_stdtime2_new'])
merged_df1['上り3F勝馬差分']=merged_df1['上り3F']-merged_df1['上り3F_1着_stdtime1_new'].fillna(merged_df1['上り3F_1着_stdtime2_new'])

# 上り3F偏差値：基準タイム１差分
merged_df1['上り3F偏差値上位差分']=merged_df1['上り3F偏差値']-merged_df1['上り3F偏差値_stdtime1_new'].fillna(merged_df1['上り3F偏差値_stdtime2_new'])
merged_df1['上り3F偏差値勝馬差分']=merged_df1['上り3F偏差値']-merged_df1['上り3F偏差値_1着_stdtime1_new'].fillna(merged_df1['上り3F偏差値_1着_stdtime2_new'])

# 上り3F順位：基準タイム１差分
merged_df1['上り3F順位上位差分']=merged_df1['上り3F順位']-merged_df1['上り3F順位_stdtime1_new'].fillna(merged_df1['上り3F順位_stdtime2_new'])
merged_df1['上り3F順位勝馬差分']=merged_df1['上り3F順位']-merged_df1['上り3F順位_1着_stdtime1_new'].fillna(merged_df1['上り3F順位_1着_stdtime2_new'])

# 上り3F順位分布：基準タイム１差分
merged_df1['上り3F順位分布上位差分']=merged_df1['上り3F順位分布']-merged_df1['上り3F順位分布_stdtime1_new'].fillna(merged_df1['上り3F順位分布_stdtime2_new'])
merged_df1['上り3F順位分布勝馬差分']=merged_df1['上り3F順位分布']-merged_df1['上り3F順位分布_1着_stdtime1_new'].fillna(merged_df1['上り3F順位分布_1着_stdtime2_new'])

# Ave-3F：基準タイム１差分
merged_df1['Ave-3F上位差分']=merged_df1['Ave-3F']-merged_df1['Ave-3F_stdtime1_new'].fillna(merged_df1['Ave-3F_stdtime2_new'])
merged_df1['Ave-3F勝馬差分']=merged_df1['Ave-3F']-merged_df1['Ave-3F_1着_stdtime1_new'].fillna(merged_df1['Ave-3F_1着_stdtime2_new'])

# Ave-3F偏差値：基準タイム１差分
merged_df1['Ave-3F偏差値上位差分']=merged_df1['Ave-3F偏差値']-merged_df1['Ave-3F偏差値_stdtime1_new'].fillna(merged_df1['Ave-3F偏差値_stdtime2_new'])
merged_df1['Ave-3F偏差値勝馬差分']=merged_df1['Ave-3F偏差値']-merged_df1['Ave-3F偏差値_1着_stdtime1_new'].fillna(merged_df1['Ave-3F偏差値_1着_stdtime2_new'])

# Ave-3F順位：基準タイム１差分
merged_df1['Ave-3F順位上位差分']=merged_df1['Ave-3F順位']-merged_df1['Ave-3F順位_stdtime1_new'].fillna(merged_df1['Ave-3F順位_stdtime2_new'])
merged_df1['Ave-3F順位勝馬差分']=merged_df1['Ave-3F順位']-merged_df1['Ave-3F順位_1着_stdtime1_new'].fillna(merged_df1['Ave-3F順位_1着_stdtime2_new'])

# Ave-3F順位分布：基準タイム１差分
merged_df1['Ave-3F順位分布上位差分']=merged_df1['Ave-3F順位分布']-merged_df1['Ave-3F順位分布_stdtime1_new'].fillna(merged_df1['Ave-3F順位分布_stdtime2_new'])
merged_df1['Ave-3F順位分布勝馬差分']=merged_df1['Ave-3F順位分布']-merged_df1['Ave-3F順位分布_1着_stdtime1_new'].fillna(merged_df1['Ave-3F順位分布_1着_stdtime2_new'])

# -3F差：基準タイム１差分
merged_df1['-3F差上位差分']=merged_df1['-3F差']-merged_df1['-3F差_stdtime1_new'].fillna(merged_df1['-3F差_stdtime2_new'])
merged_df1['-3F差勝馬差分']=merged_df1['-3F差']-merged_df1['-3F差_1着_stdtime1_new'].fillna(merged_df1['-3F差_1着_stdtime2_new'])

# テン指数：基準タイム１差分
merged_df1['テン指数上位差分']=merged_df1['テン指数']-merged_df1['テン指数_stdtime1_new'].fillna(merged_df1['テン指数_stdtime2_new'])
merged_df1['テン指数勝馬差分']=merged_df1['テン指数']-merged_df1['テン指数_1着_stdtime1_new'].fillna(merged_df1['テン指数_1着_stdtime2_new'])

# テン指数偏差値：基準タイム１差分
merged_df1['テン指数偏差値上位差分']=merged_df1['テン指数偏差値']-merged_df1['テン指数偏差値_stdtime1_new'].fillna(merged_df1['テン指数偏差値_stdtime2_new'])
merged_df1['テン指数偏差値勝馬差分']=merged_df1['テン指数偏差値']-merged_df1['テン指数偏差値_1着_stdtime1_new'].fillna(merged_df1['テン指数偏差値_1着_stdtime2_new'])

# テン指数順位：基準タイム１差分
merged_df1['テン指数順位上位差分']=merged_df1['テン指数順位']-merged_df1['テン指数順位_stdtime1_new'].fillna(merged_df1['テン指数順位_stdtime2_new'])
merged_df1['テン指数順位勝馬差分']=merged_df1['テン指数順位']-merged_df1['テン指数順位_1着_stdtime1_new'].fillna(merged_df1['テン指数順位_1着_stdtime2_new'])

# テン指数順位分布：基準タイム１差分
merged_df1['テン指数順位分布上位差分']=merged_df1['テン指数順位分布']-merged_df1['テン指数順位分布_stdtime1_new'].fillna(merged_df1['テン指数順位分布_stdtime2_new'])
merged_df1['テン指数順位分布勝馬差分']=merged_df1['テン指数順位分布']-merged_df1['テン指数順位分布_1着_stdtime1_new'].fillna(merged_df1['テン指数順位分布_1着_stdtime2_new'])

# 上り指数：基準タイム１差分
merged_df1['上り指数上位差分']=merged_df1['上り指数']-merged_df1['上り指数_stdtime1_new'].fillna(merged_df1['上り指数_stdtime2_new'])
merged_df1['上り指数勝馬差分']=merged_df1['上り指数']-merged_df1['上り指数_1着_stdtime1_new'].fillna(merged_df1['上り指数_1着_stdtime2_new'])

# 上り指数偏差値：基準タイム１差分
merged_df1['上り指数偏差値上位差分']=merged_df1['上り指数偏差値']-merged_df1['上り指数偏差値_stdtime1_new'].fillna(merged_df1['上り指数偏差値_stdtime2_new'])
merged_df1['上り指数偏差値勝馬差分']=merged_df1['上り指数偏差値']-merged_df1['上り指数偏差値_1着_stdtime1_new'].fillna(merged_df1['上り指数偏差値_1着_stdtime2_new'])

# 上り指数順位：基準タイム１差分
merged_df1['上り指数順位上位差分']=merged_df1['上り指数順位']-merged_df1['上り指数順位_stdtime1_new'].fillna(merged_df1['上り指数順位_stdtime2_new'])
merged_df1['上り指数順位勝馬差分']=merged_df1['上り指数順位']-merged_df1['上り指数順位_1着_stdtime1_new'].fillna(merged_df1['上り指数順位_1着_stdtime2_new'])

# 上り指数順位分布：基準タイム１差分
merged_df1['上り指数順位分布上位差分']=merged_df1['上り指数順位分布']-merged_df1['上り指数順位分布_stdtime1_new'].fillna(merged_df1['上り指数順位分布_stdtime2_new'])
merged_df1['上り指数順位分布勝馬差分']=merged_df1['上り指数順位分布']-merged_df1['上り指数順位分布_1着_stdtime1_new'].fillna(merged_df1['上り指数順位分布_1着_stdtime2_new'])

# スピード指数：基準タイム１差分
merged_df1['スピード指数上位差分']=merged_df1['スピード指数']-merged_df1['スピード指数_stdtime1_new'].fillna(merged_df1['スピード指数_stdtime2_new'])
merged_df1['スピード指数勝馬差分']=merged_df1['スピード指数']-merged_df1['スピード指数_1着_stdtime1_new'].fillna(merged_df1['スピード指数_1着_stdtime2_new'])

# スピード指数偏差値：基準タイム１差分
merged_df1['スピード指数偏差値上位差分']=merged_df1['スピード指数偏差値']-merged_df1['スピード指数偏差値_stdtime1_new'].fillna(merged_df1['スピード指数偏差値_stdtime2_new'])
merged_df1['スピード指数偏差値勝馬差分']=merged_df1['スピード指数偏差値']-merged_df1['スピード指数偏差値_1着_stdtime1_new'].fillna(merged_df1['スピード指数偏差値_1着_stdtime2_new'])

# スピード指数順位：基準タイム１差分
merged_df1['スピード指数順位上位差分']=merged_df1['スピード指数順位']-merged_df1['スピード指数順位_stdtime1_new'].fillna(merged_df1['スピード指数順位_stdtime2_new'])
merged_df1['スピード指数順位勝馬差分']=merged_df1['スピード指数順位']-merged_df1['スピード指数順位_1着_stdtime1_new'].fillna(merged_df1['スピード指数順位_1着_stdtime2_new'])

# スピード指数順位分布：基準タイム１差分
merged_df1['スピード指数順位分布上位差分']=merged_df1['スピード指数順位分布']-merged_df1['スピード指数順位分布_stdtime1_new'].fillna(merged_df1['スピード指数順位分布_stdtime2_new'])
merged_df1['スピード指数順位分布勝馬差分']=merged_df1['スピード指数順位分布']-merged_df1['スピード指数順位分布_1着_stdtime1_new'].fillna(merged_df1['スピード指数順位分布_1着_stdtime2_new'])

# 総合指数：基準タイム１差分
merged_df1['総合指数上位差分']=merged_df1['総合指数']-merged_df1['総合指数_stdtime1_new'].fillna(merged_df1['総合指数_stdtime2_new'])
merged_df1['総合指数勝馬差分']=merged_df1['総合指数']-merged_df1['総合指数_1着_stdtime1_new'].fillna(merged_df1['総合指数_1着_stdtime2_new'])

# 総合指数偏差値：基準タイム１差分
merged_df1['総合指数偏差値上位差分']=merged_df1['総合指数偏差値']-merged_df1['総合指数偏差値_stdtime1_new'].fillna(merged_df1['総合指数偏差値_stdtime2_new'])
merged_df1['総合指数偏差値勝馬差分']=merged_df1['総合指数偏差値']-merged_df1['総合指数偏差値_1着_stdtime1_new'].fillna(merged_df1['総合指数偏差値_1着_stdtime2_new'])

# 総合指数順位：基準タイム１差分
merged_df1['総合指数順位上位差分']=merged_df1['総合指数順位']-merged_df1['総合指数順位_stdtime1_new'].fillna(merged_df1['総合指数順位_stdtime2_new'])
merged_df1['総合指数順位勝馬差分']=merged_df1['総合指数順位']-merged_df1['総合指数順位_1着_stdtime1_new'].fillna(merged_df1['総合指数順位_1着_stdtime2_new'])

# 総合指数順位分布：基準タイム１差分
merged_df1['総合指数順位分布上位差分']=merged_df1['総合指数順位分布']-merged_df1['総合指数順位分布_stdtime1_new'].fillna(merged_df1['総合指数順位分布_stdtime2_new'])
merged_df1['総合指数順位分布勝馬差分']=merged_df1['総合指数順位分布']-merged_df1['総合指数順位分布_1着_stdtime1_new'].fillna(merged_df1['総合指数順位分布_1着_stdtime2_new'])

# PCI：基準タイム１差分
merged_df1['PCI上位差分']=merged_df1['PCI']-merged_df1['PCI_stdtime1_new'].fillna(merged_df1['PCI_stdtime2_new'])
merged_df1['PCI勝馬差分']=merged_df1['PCI']-merged_df1['PCI_1着_stdtime1_new'].fillna(merged_df1['PCI_1着_stdtime2_new'])

# 初角_4角差：基準タイム１差分
merged_df1['初角_4角差上位差分']=merged_df1['初角_4角差']-merged_df1['初角_4角差_stdtime1_new'].fillna(merged_df1['初角_4角差_stdtime2_new'])
merged_df1['初角_4角差勝馬差分']=merged_df1['初角_4角差']-merged_df1['初角_4角差_1着_stdtime1_new'].fillna(merged_df1['初角_4角差_1着_stdtime2_new'])

# 4角_入線順位差：基準タイム１差分
merged_df1['4角_入線順位差上位差分']=merged_df1['4角_入線順位差']-merged_df1['4角_入線順位差_stdtime1_new'].fillna(merged_df1['4角_入線順位差_stdtime2_new'])
merged_df1['4角_入線順位差勝馬差分']=merged_df1['4角_入線順位差']-merged_df1['4角_入線順位差_1着_stdtime1_new'].fillna(merged_df1['4角_入線順位差_1着_stdtime2_new'])

#---ステップ６：レースレベル基準の作成
# グループ化キー
group_cols3 = ['場所','芝・ダート','距離','トラックコード(JV)','競走種別','クラス分類']

# --- 基準レースレベル算出処理 ---
racelevel_df = (
    merged_df1
      .dropna(subset=['レースレベル指数'])  # 欠損を除外
      .groupby(group_cols3, dropna=False)['レースレベル指数']
      .agg(['mean', 'std'])  # mean と std を同時に集計
      .reset_index()
).copy()

# 近似Zスコア用の基準値を計算
racelevel_df['RL±0'] = racelevel_df['mean'].round(2)
racelevel_df['RL+1'] = (racelevel_df['mean'] + racelevel_df['std']).round(2)
racelevel_df['RL-1'] = (racelevel_df['mean'] - racelevel_df['std']).round(2)
racelevel_df['RL+2'] = (racelevel_df['mean'] + 2 * racelevel_df['std']).round(2)
racelevel_df['RL-2'] = (racelevel_df['mean'] - 2 * racelevel_df['std']).round(2)

# 必要なカラムだけ残す
racelevel_df = racelevel_df[
    group_cols3 + ['RL-2','RL-1','RL±0','RL+1','RL+2','mean','std']
].copy()

#---ステップ７：レースレベル判定
# レースレベル基準のデータフレームにサフィックスを付ける
merged_df2 = prepare_ref_df(racelevel_df, group_cols3, '_racelevel')

merged_df1 = pd.merge(merged_df1, merged_df2, on=group_cols3, how='left')

# 不要なデータフレームのクリア
del merged_df2

# 差分
rl_m2 = (merged_df1['レースレベル指数'] - merged_df1['RL-2_racelevel']).abs()
rl_m1 = (merged_df1['レースレベル指数'] - merged_df1['RL-1_racelevel']).abs()
rl_0  = (merged_df1['レースレベル指数'] - merged_df1['RL±0_racelevel']).abs()
rl_p1 = (merged_df1['レースレベル指数'] - merged_df1['RL+1_racelevel']).abs()
rl_p2 = (merged_df1['レースレベル指数'] - merged_df1['RL+2_racelevel']).abs()

# 5本を横に並べて「最小の列名」を取る（行ごと）
diff_df = pd.concat([rl_m2, rl_m1, rl_0, rl_p1, rl_p2], axis=1)
diff_df.columns = [-2, -1, 0, 1, 2]  # そのまま判定値にする

# 「基準が無い行」判定（5本すべてNaN）
no_ref = diff_df.isna().all(axis=1)

# idxminを安定させるため NaN は無限大扱いにして最小を取る
band = diff_df.fillna(np.inf).idxmin(axis=1)

# レースレベル判定 列を追加
insert_pos = merged_df1.columns.get_loc('レースレベル指数') + 1
merged_df1.insert(insert_pos, 'レースレベル判定', band.where(~no_ref, np.nan).astype('Int64'))

# レースレベル評価 列を追加
band_to_grade = {2: 'A', 1: 'B', 0: 'C', -1: 'D', -2: 'E'}
insert_pos = merged_df1.columns.get_loc('レースレベル判定') + 1
merged_df1.insert(insert_pos, 'レースレベル評価', merged_df1['レースレベル判定'].map(band_to_grade))

# レース印３ 列を追加
rank_to_label = {'A': '05A', 'B': '07B', 'C': '01C', 'D': '00D', 'E': '03E'}
merged_df1['レース印３'] = merged_df1['レースレベル評価'].replace(rank_to_label)

# サフィックス付き列を削除
# 削除するサフィックスを列挙
suffixes_to_drop = ('_stdtime1_new','_stdtime2_new', '_racelevel')

# サフィックスで判定して残す列だけ抜き出し
merged_df1 = merged_df1.loc[:, [c for c in merged_df1.columns
                              if not any(c.endswith(suf) for suf in suffixes_to_drop)]].copy()

#---ステップ８：馬の持久力/瞬発力タイプ判定
# merged_df1 をコピーして、1着～3着だけにする
group_cols4 = ['場所', '芝・ダート', '距離', 'トラックコード(JV)','馬場分類']

horsetype_df = merged_df1[merged_df1['入線順位'].between(1, 3)].copy()

horsetype_df= horsetype_df[['target_raceid', '血統登録番号','PCI3', 'PCI'] + group_cols4].copy()

# 基準タイムファイル１から基準PCI3を取ってマージ
std1_df = base_time_df1[group_cols4 + ['PCI3']].copy()
std1_df = std1_df.rename(columns={'PCI3': 'PCI3_stdtime1'})

horsetype_df = pd.merge(horsetype_df, std1_df, on=group_cols4, how='left')

# 不要なデータフレームのクリア
del std1_df

# レース性質（基準比）と、馬のレース内差分を合算
horsetype_df['基準PCI3差分'] = horsetype_df['PCI3'] - horsetype_df['PCI3_stdtime1']
horsetype_df['レースPCI3差分'] = horsetype_df['PCI'] - horsetype_df['PCI3']
horsetype_df['PCI判定スコア'] = horsetype_df['基準PCI3差分'] + horsetype_df['レースPCI3差分']

# 馬ごとに PCI3差分 の中央値を取る
horsetype_df = horsetype_df.groupby('血統登録番号', as_index=False)['PCI判定スコア'].median()

# 'Ｃ' 列を追加して判定（マイナス=持、プラス=瞬）
horsetype_df['Ｃ'] = ''
horsetype_df.loc[horsetype_df['PCI判定スコア'] < 0, 'Ｃ'] = '0'
horsetype_df.loc[horsetype_df['PCI判定スコア'] > 0, 'Ｃ'] = '1'

# 成績データへ馬タイプを代入する
c_map = horsetype_df.set_index('血統登録番号')['Ｃ']
merged_df1['Ｃ'] = merged_df1['血統登録番号'].map(c_map)

# 今日の日付を yymmdd で作る
today_yymmdd = datetime.now().strftime("%y%m%d")

# 判定済み horsetype_df（= 血統登録番号 + Ｃ）に馬名を付ける
horse_name_df = (
    merged_df1[['血統登録番号', '馬名']]
    .dropna(subset=['馬名'])
    .drop_duplicates(subset=['血統登録番号'], keep='last')
)

checkhorse_df = pd.merge(horsetype_df, horse_name_df, on='血統登録番号', how='left')

# 不要なデータフレームのクリア
del horsetype_df
del horse_name_df

# ★「0/1が入った馬だけ」出力（中央値0や判定不能は除外）
checkhorse_df = checkhorse_df[checkhorse_df['Ｃ'].isin(['0', '1'])].copy()

# TARGET仕様の列名に変換
checkhorse_df = checkhorse_df.rename(columns={'Ｃ': 'タイプ'})

# 登録日
checkhorse_df['登録日'] = today_yymmdd

# 血統登録番号の整形（UX～なら zfill しない方が安全）
checkhorse_df['血統登録番号'] = checkhorse_df['血統登録番号'].astype(str).str.strip()

# 列順
checkhorse_df = checkhorse_df[['馬名', 'タイプ', '登録日', '血統登録番号']]

#---ステップ９：csvファイル保存
# csvファイル保存
merged_df1.to_csv(output_filepath1, index=False, encoding='cp932')
base_time_df1.to_csv(output_filepath2, index=False, encoding='cp932')
base_time_df2.to_csv(output_filepath3, index=False, encoding='cp932')
pace_medians_df.to_csv(output_filepath4, index=False, encoding='cp932')
lap33_df.to_csv(output_filepath5, index=False, encoding='cp932')
racelevel_df.to_csv(output_filepath6, index=False, encoding='cp932')
checkhorse_df.to_csv(output_filepath13, index=False, encoding='cp932')

# 各指数をインポート用ファイルに加工して保存
# テン指数
imp_df1 = merged_df1[['target_horseid','テン指数']]
# 上り指数
imp_df2 = merged_df1[['target_horseid','上り指数']]
# スピード指数
imp_df3 = merged_df1[['target_horseid','スピード指数']]
# 総合指数
imp_df4 = merged_df1[['target_horseid','総合指数']]
# 33ラップ判定
imp_df5 = merged_df1[['target_raceid','レース印２']].drop_duplicates('target_raceid')
# レースレベル判定
imp_df6 = merged_df1[['target_raceid','レース印３']].drop_duplicates('target_raceid')

# 年度列を追加（target_raceid先頭4桁が年）
imp_df1['year'] = pd.to_datetime(
    imp_df1['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

imp_df2['year'] = pd.to_datetime(
    imp_df2['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

imp_df3['year'] = pd.to_datetime(
    imp_df3['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

imp_df4['year'] = pd.to_datetime(
    imp_df4['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

imp_df5['year'] = pd.to_datetime(
    imp_df5['target_raceid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

imp_df6['year'] = pd.to_datetime(
    imp_df6['target_raceid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

# 年度別にファイル分割
for year, df in imp_df1.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_filepath7}First_Score_Master_{year}.csv',
        index=False, encoding='cp932'
    )

for year, df in imp_df2.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_filepath8}Spurt_Score_Master_{year}.csv',
        index=False, encoding='cp932'
    )

for year, df in imp_df3.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_filepath9}Speed_Score_Master_{year}.csv',
        index=False, encoding='cp932'
    )

for year, df in imp_df4.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_filepath10}Total_Score_Master_{year}.csv',
        index=False, encoding='cp932'
    )

for year, df in imp_df5.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_filepath11}33Lap_Category_Master_{year}.csv',
        index=False, encoding='cp932'
    )

for year, df in imp_df6.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_filepath12}Race_Level_Master_{year}.csv',
        index=False, encoding='cp932'
    )

# 不要なデータフレームのクリア
del merged_df1
del base_time_df1
del base_time_df2
del pace_medians_df
del lap33_df
del racelevel_df
del imp_df1
del imp_df2
del imp_df3
del imp_df4
del imp_df5
del imp_df6

C:\Users\rotte\AppData\Local\Temp\ipykernel_21172\2103797910.py:59: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(master_file_path, encoding='cp932')
C:\Users\rotte\AppData\Local\Temp\ipykernel_21172\2103797910.py:67: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df9 = pd.read_csv(merge_filepath8, encoding='cp932')
C:\Users\rotte\AppData\Local\Temp\ipykernel_21172\2103797910.py:172: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[53.6 47.8 54.6 ... '61.9' '61.5' '58.9']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  left.update(right, overwrite=True)
C:\Users\rotte\AppData\Local\Temp\ipykernel_21172\2103797910.py:660: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version

# 追切指数を出力するコード

## ・追切指数算出用の基準タイムファイルを作成する

In [5]:
from datetime import datetime
import pandas as pd
import numpy as np
import re
import os

# ファイルパスの指定
traning_filepath = r'D:\Keiba\00_ImportData\50_Training_Comments\Training_Comments_Master.csv'
master_filepath = r'D:\Keiba\10_ExportData\ResultData\Processed\Race_Result_Master.csv'
output_filepath = r'D:\Keiba\20_IndexData\Training\\'

# ファイルを読みこむ
df1 = pd.read_csv(traning_filepath, encoding='cp932' ).copy()
df2 = pd.read_csv(master_filepath, encoding='cp932').copy()

# 不要な列を削除
def clean_training_data(df):
    df = df.copy()

    mask_invalid = df["日付"].astype(str).str.contains("■|◇", na=False)
    df = df[~mask_invalid].copy()

    check_cols = [c for c in ['8F','7F','6F','5F(4F)','4F(3F)','3F(2F)'] if c in df.columns]
    for c in check_cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')

    is_not_hill = ~df['コース'].astype(str).str.contains('坂', na=False)
    has_10sec_like = np.column_stack([df[c].between(10.0, 19.9) for c in check_cols]).any(axis=1)

    drop_mask = is_not_hill & has_10sec_like
    df = df[~drop_mask].copy()

    return df

df1 = clean_training_data(df1)

# 追切コースの書き換え
replace_map = {'美Ｄ': '美ダ'}
df1['コース'] = df1['コース'].replace(replace_map)

# 馬場状態を書き換え
df1['馬場状態'] = df1['馬場状態'].replace({'良': '良・稍重','稍': '良・稍重','重': '重・不良','不': '重・不良'})

# 回り位置を数値化（変換できないものは NaN に）
df1["回り位置"] = pd.to_numeric(df1["回り位置"], errors="coerce")

# 対象の距離カラム
distance_columns = [col for col in ["8F", "7F", "6F", "5F(4F)", "4F(3F)", "3F(2F)", "1F"] if col in df1.columns]

# まず、補正後の列を作成しておく（NaNで初期化）
for col in distance_columns:
    df1[f"{col}_補"] = np.nan

# 補正ループ開始（ここは一切変更しない）
for index, row in df1.iterrows():
    if not pd.isna(row["回り位置"]):
        correction_value = (9 - row["回り位置"]) * 0.1

        valid_times = [col for col in distance_columns if not pd.isna(row[col])]
        if valid_times:
            leftmost_col = valid_times[0]
            leftmost_index = distance_columns.index(leftmost_col)

            df1.at[index, f"{leftmost_col}_補"] = round(row[leftmost_col] + correction_value, 1)

            remaining_cols = distance_columns[leftmost_index + 1:]
            if remaining_cols:
                equal_correction = correction_value / len(remaining_cols)
                for col in remaining_cols:
                    if not pd.isna(row[col]):
                        df1.at[index, f"{col}_補"] = round(row[col] + equal_correction, 1)
        else:
            continue
    else:
        for col in distance_columns:
            if not pd.isna(row[col]):
                df1.at[index, f"{col}_補"] = row[col]

# 成績データで必要な列を定義
merge_cols = [
    'target_horseid',
    '場所',
    '芝・ダート',
    '距離',
    'トラックコード(JV)',
    '競走種別',
    'クラス分類',
    '調教師',
    '調教師コード',
    '血統登録番号',
    '馬名',
    '入線順位'
]
df2_subset = df2[merge_cols].copy()

# マージする
merged_df1 = pd.merge(df1, df2_subset, on="target_horseid", how="inner")

# 不要なデータフレームのクリア
del df1, df2, df2_subset

# 1着から3着馬のデータを抽出する
merged_df1 = merged_df1[merged_df1['入線順位'].between(1, 3)].copy()

# 追切基準タイムファイルの列を定義する（ここも現状維持）
time_columns = ['8F_補', '7F_補', '6F_補', '5F(4F)_補', '4F(3F)_補', '3F(2F)_補', '1F_補']

# ==============================
# ここから「最小改修」：median + (MAD由来の標準偏差相当) + 外れ値除外(任意)
# ==============================
K_MAD = 1.4826   # MAD -> std相当
K_OUT = 3.0      # 外れ値除外のしきい（3σ相当）

def _robust_stats_for_group(g):
    out = {}
    for c in time_columns:
        s = g[c].dropna()
        if s.empty:
            out[c] = np.nan
            out[f'標準偏差_{c}'] = np.nan
            continue

        m = s.median()
        mad = (s - m).abs().median()
        rs = K_MAD * mad if pd.notna(mad) else np.nan

        # 外れ値除外（rsが0/NaNなら除外しない）
        if pd.notna(rs) and rs > 0:
            s2 = s[(s - m).abs() <= K_OUT * rs]
        else:
            s2 = s

        m2 = s2.median() if not s2.empty else np.nan
        mad2 = (s2 - m2).abs().median() if (pd.notna(m2) and not s2.empty) else np.nan
        rs2 = K_MAD * mad2 if pd.notna(mad2) else np.nan

        out[c] = m2
        out[f'標準偏差_{c}'] = round(rs2, 2) if pd.notna(rs2) else np.nan

    return pd.Series(out)

# ===== A. course基準（コース×馬場状態）：中央値＋(robust sigma)＋Sample_Count =====
course_median = merged_df1.groupby(['コース','馬場状態']).apply(_robust_stats_for_group).reset_index()

course_median = pd.merge(
    course_median,
    merged_df1.groupby(['コース','馬場状態']).size().reset_index(name='Sample_Count'),
    on=['コース','馬場状態'],
    how='left'
)

# ===== B. class基準（競走種別×クラス分類×コース×馬場状態）：中央値＋(robust sigma)＋Sample_Count =====
class_median = merged_df1.groupby(['競走種別','クラス分類','コース','馬場状態']).apply(_robust_stats_for_group).reset_index()

class_median = pd.merge(
    class_median,
    merged_df1.groupby(['競走種別','クラス分類','コース','馬場状態']).size().reset_index(name='Sample_Count'),
    on=['競走種別','クラス分類','コース','馬場状態'],
    how='left'
)

# ===== C. course top10 / top20（コース×馬場状態）※inlier(外れ値除外後)でp10/p20を作る =====
def _top_mean_for_group(g, q):
    out = {}
    for c in time_columns:
        s = g[c].dropna()
        if s.empty:
            out[c] = np.nan
            continue

        m = s.median()
        mad = (s - m).abs().median()
        rs = K_MAD * mad if pd.notna(mad) else np.nan

        if pd.notna(rs) and rs > 0:
            s = s[(s - m).abs() <= K_OUT * rs]

        if s.empty:
            out[c] = np.nan
            continue

        thr = s.quantile(q)
        out[c] = s[s <= thr].mean()

    return pd.Series(out)

course_top20_mean = merged_df1.groupby(['コース','馬場状態']).apply(lambda g: _top_mean_for_group(g, 0.2)).round(1).reset_index()
course_top10_mean = merged_df1.groupby(['コース','馬場状態']).apply(lambda g: _top_mean_for_group(g, 0.1)).round(1).reset_index()

# ===== ファイル出力（A,B,Cのみ） =====
course_median.to_csv(f"{output_filepath}traning_std_course.csv", index=False, encoding="cp932")
class_median.to_csv(f"{output_filepath}traning_std_class.csv", index=False, encoding="cp932")
course_top20_mean.to_csv(f"{output_filepath}traning_std_course_top20.csv", index=False, encoding="cp932")
course_top10_mean.to_csv(f"{output_filepath}traning_std_course_top10.csv", index=False, encoding="cp932")

# 不要なデータフレームのクリア
del merged_df1, course_median, class_median, course_top20_mean, course_top10_mean


C:\Users\rotte\AppData\Local\Temp\ipykernel_3296\401544643.py:143: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  course_median = merged_df1.groupby(['コース','馬場状態']).apply(_robust_stats_for_group).reset_index()
C:\Users\rotte\AppData\Local\Temp\ipykernel_3296\401544643.py:153: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  class_median = merged_df1.groupby(['競走種別','クラス分類','コース','馬場状態']).apply(_robust_stats_for_group).re

## ・追切指数を成績データへ追加する

In [6]:
import pandas as pd
import numpy as np

# =========================
# 読み込みファイルパス
# =========================
traning_filepath = r'D:\Keiba\00_ImportData\50_Training_Comments\Training_Comments_Master.csv'
master_filepath  = r'D:\Keiba\10_ExportData\ResultData\Processed\Race_Result_Master.csv'

# =========================
# 基準ファイル（A/B/Cのみ）
#  A: course median(+std)
#  B: class  median(+std)
#  C: course top10 / top20
# =========================
p_course_median  = r'D:\Keiba\20_IndexData\Training\traning_std_course.csv'
p_class_median   = r'D:\Keiba\20_IndexData\Training\traning_std_class.csv'

p_course_top20   = r'D:\Keiba\20_IndexData\Training\traning_std_course_top20.csv'
p_course_top10   = r'D:\Keiba\20_IndexData\Training\traning_std_course_top10.csv'

# =========================
# 出力ファイル
# =========================
output_filepath1 = r'D:\Keiba\00_ImportData\210_Training_Score\Training_Score_Master.csv'
output_filepath2 = r'D:\Keiba\20_IndexData\Times\StdTime1.csv'
output_filepath3 = r'D:\Keiba\20_IndexData\Times\StdTime2.csv'
output_dir       = r'D:\Keiba\00_ImportData\210_Training_Score\\'

# =========================
# 読み込み
# =========================
df1 = pd.read_csv(traning_filepath, encoding='cp932').copy()
df2 = pd.read_csv(master_filepath,  encoding='cp932').copy()

# =========================
# 前処理（追切）
# =========================
def clean_training_data(df):
    df = df.copy()

    # 特殊文字を含む行を削除
    mask_invalid = df["日付"].astype(str).str.contains("■|◇", na=False)
    df = df[~mask_invalid].copy()

    # 周回コースに紛れた「10秒台タイム」を削除
    check_cols = [c for c in ['8F','7F','6F','5F(4F)','4F(3F)','3F(2F)'] if c in df.columns]
    for c in check_cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')

    is_not_hill = ~df['コース'].astype(str).str.contains('坂', na=False)
    has_10sec_like = np.column_stack([df[c].between(10.0, 19.9) for c in check_cols]).any(axis=1)

    drop_mask = is_not_hill & has_10sec_like
    df = df[~drop_mask].copy()

    return df

df1 = clean_training_data(df1)

# 追切コースの書き換え
df1['コース'] = df1['コース'].replace({'美Ｄ': '美ダ'})

# 馬場状態を書き換え
df1['馬場状態'] = df1['馬場状態'].replace({'良': '良・稍重','稍': '良・稍重','重': '重・不良','不': '重・不良'})

# 回り位置
df1["回り位置"] = pd.to_numeric(df1["回り位置"], errors="coerce")

# 対象距離カラム
distance_columns = [col for col in ["8F", "7F", "6F", "5F(4F)", "4F(3F)", "3F(2F)", "1F"] if col in df1.columns]

# 補正後列
for col in distance_columns:
    df1[f"{col}_補"] = np.nan

# =========================
# 補正ループ（変更なし）
# =========================
for index, row in df1.iterrows():
    if not pd.isna(row["回り位置"]):
        correction_value = (9 - row["回り位置"]) * 0.1

        valid_times = [col for col in distance_columns if not pd.isna(row[col])]
        if valid_times:
            leftmost_col = valid_times[0]
            leftmost_index = distance_columns.index(leftmost_col)

            df1.at[index, f"{leftmost_col}_補"] = round(row[leftmost_col] + correction_value, 1)

            remaining_cols = distance_columns[leftmost_index + 1:]
            if remaining_cols:
                equal_correction = correction_value / len(remaining_cols)
                for col in remaining_cols:
                    if not pd.isna(row[col]):
                        df1.at[index, f"{col}_補"] = round(row[col] + equal_correction, 1)
        else:
            continue
    else:
        for col in distance_columns:
            if not pd.isna(row[col]):
                df1.at[index, f"{col}_補"] = row[col]

# 評価列
time_columns = [f"{c}_補" for c in distance_columns]

# =========================
# 成績側の必要列 → マージ
# =========================
merge_cols = [
    'target_horseid','場所','芝・ダート','距離','トラックコード(JV)',
    '競走種別','クラス分類','調教師','調教師コード','血統登録番号','馬名','入線順位'
]
df2_subset = df2[merge_cols].copy()

merged = pd.merge(df1, df2_subset, on='target_horseid', how='inner').copy()

# 坂路判定
merged['is_saka'] = merged['コース'].astype(str).str.contains('坂', na=False)

# =========================
# 基準テーブル読み込み＆列名タグ付け
# =========================
def load_and_tag(path, suffix, has_std):
    t = pd.read_csv(path, encoding='cp932').copy()
    rename_map = {}
    for c in time_columns:
        if c in t.columns:
            rename_map[c] = f"{c}_{suffix}"
        stdc = f"標準偏差_{c}"
        if has_std and (stdc in t.columns):
            rename_map[stdc] = f"{stdc}_{suffix}"
    return t.rename(columns=rename_map)

# 偏差値用（A/Bのみ）
df3 = load_and_tag(p_course_median, 'course', True)
df5 = load_and_tag(p_class_median,  'class',  True)

# 加点用（Cのみ）
df7 = load_and_tag(p_course_top20, 'course_20', False)
df8 = load_and_tag(p_course_top10, 'course_10', False)

# =========================
# マージ（A/B/Cのみ）
# =========================
key_course = ['コース','馬場状態']
key_class  = ['競走種別','クラス分類','コース','馬場状態']

def smerge(left, right, keys):
    exist_keys = [k for k in keys if (k in left.columns and k in right.columns)]
    return pd.merge(left, right, on=exist_keys, how='left')

merged = smerge(merged, df3, key_course)
merged = smerge(merged, df5, key_class)
merged = smerge(merged, df7, key_course)
merged = smerge(merged, df8, key_course)

# =========================
# 偏差値（course/class 平均）
# =========================
def dev_vec(v, m, s):
    ok = (~pd.isna(v)) & (~pd.isna(m)) & (~pd.isna(s)) & (s != 0)
    return np.where(ok, 50 + 10 * (m - v) / s, np.nan)

bases_for_dev = ['course', 'class']

for c in time_columns:
    dev_cols = []
    for b in bases_for_dev:
        mean_col = f"{c}_{b}"
        std_col  = f"標準偏差_{c}_{b}"
        colname = f"偏差値_{c}_{b}"
        merged[colname] = dev_vec(merged[c], merged[mean_col], merged[std_col])
        dev_cols.append(colname)

    merged[f"偏差値_統合_{c}"] = merged[dev_cols].mean(axis=1, skipna=True)

merged['総合偏差値スコア'] = merged[[f"偏差値_統合_{c}" for c in time_columns]].mean(axis=1, skipna=True)

# =========================
# 加点（course_top10/20 のみ）
# =========================
bonus_targets = {
    '4F': '4F(3F)_補',
    '2F': '3F(2F)_補',
    '1F': '1F_補'
}

def calculate_bonus(value, mean, is_saka, col_short):
    if pd.isna(value) or pd.isna(mean):
        return 0.0
    if is_saka:
        if col_short == '4F': return 0.5 if value < mean else 0.0
        if col_short == '2F': return 1.5 if value < mean else 0.0
        if col_short == '1F': return 1.0 if value < mean else 0.0
    else:
        if col_short == '4F': return 0.5 if value < mean else 0.0
        if col_short == '2F': return 1.0 if value < mean else 0.0
        if col_short == '1F': return 1.5 if value < mean else 0.0
    return 0.0

th_sfx_list = ['course_20','course_10']

bonus_cols = []
for col_short, base_col in bonus_targets.items():
    for sfx in th_sfx_list:
        th_col = f"{base_col}_{sfx}"
        if th_col in merged.columns:
            out_col = f"加点_{col_short}_{sfx}"
            merged[out_col] = merged.apply(
                lambda r: calculate_bonus(
                    r.get(base_col, np.nan),
                    r.get(th_col, np.nan),
                    bool(r.get('is_saka', False)),
                    col_short
                ),
                axis=1
            )
            bonus_cols.append(out_col)

merged['総合加点スコア'] = merged[bonus_cols].sum(axis=1, skipna=True) if bonus_cols else 0.0

# =========================
# 係数
# =========================
def rider_coef(x):
    s = '' if pd.isna(x) else str(x)
    if '助手' in s: return 1.0
    if '見習' in s: return 0.8
    return 0.9

def leg_coef(x):
    s = '' if pd.isna(x) else str(x)
    if '馬なり' in s: return 1.1
    if ('Ｇ' in s) or ('G' in s) or ('強' in s): return 1.0
    if '一杯' in s: return 0.8
    if 'ヨレ' in s: return 0.7
    if 'バテ' in s: return 0.6
    return 0.9

merged['騎乗者係数'] = merged['乗り役'].apply(rider_coef)
merged['脚色係数']   = merged['脚色'].apply(leg_coef)

# =========================
# 最終指数
# =========================
merged['追切指数'] = (
    (merged['総合偏差値スコア'].fillna(50) + merged['総合加点スコア'].fillna(0.0))
    * merged['騎乗者係数'].fillna(0.9)
    * merged['脚色係数'].fillna(0.9)
)

merged['追切指数'] = (
    merged['追切指数']
    .replace([np.inf, -np.inf], np.nan)
    .fillna(1)
    .round(1)
)

# target_horseidごとに最大の追切指数だけ残す
merged_unique = merged.loc[merged.groupby('target_horseid')['追切指数'].idxmax()].reset_index(drop=True).copy()
merged_unique = merged_unique[['target_horseid','追切指数']].copy()

# =========================
# 以降の処理（保存・master更新・StdTime1/2算出）は元コードのまま
# ただし del は安全にする
# =========================
def safe_del(*names):
    g = globals()
    for n in names:
        if n in g:
            del g[n]

safe_del('df1','df2','df2_subset','df3','df5','df7','df8','merged')

# 既存ファイルを開く
combined_df = pd.read_csv(output_filepath1, encoding='cp932').copy()

# 既存ファイルに追切指数データを追加
combined_df = pd.concat([combined_df, merged_unique], ignore_index=True)

# target_horseidで重複削除する
combined_df = combined_df.drop_duplicates(subset='target_horseid', keep='last')

# マスタファイルにも追切指数を追加
master_df = pd.read_csv(master_filepath, encoding='cp932').copy()

left  = master_df.set_index('target_horseid')
right = (
    merged_unique[['target_horseid', '追切指数']]
    .dropna(subset=['追切指数'])
    .drop_duplicates(subset=['target_horseid'], keep='last')
    .set_index('target_horseid')
)
left.update(right, overwrite=True)
master_df = left.reset_index()

# 追切指数の偏差値を計算
def deviation_in_race(series):
    mean = series.mean()
    std = series.std()
    if std == 0 or pd.isna(std):
        return pd.Series(50, index=series.index)
    return ((series - mean) / std * 10 + 50)

master_df['追切指数偏差値'] = master_df.groupby('target_raceid')['追切指数'].transform(deviation_in_race).round(1)

# 追切指数順位
master_df['追切指数順位'] = master_df.groupby('target_raceid')['追切指数'].rank(ascending=False, method='dense').astype('Int64')

# 追切指数順位分布
master_df['追切指数順位分布'] = (master_df['追切指数順位'] - 1) / (master_df['頭数'] - 1)

# --- 集計設定 ---
agg_cols1 = {
    'レースPCI': 'median',
    'PCI3': 'median',
    '通過3F': 'median',
    '通過4F': 'median',
    '通過5F': 'median',
    'レース上り3F': 'median',
    'レース上り4F': 'median',
    'レース上り5F': 'median',
    '中盤ラップ1': 'median',
    '中盤ラップ2': 'median',
    '33ラップ': 'median',
    '最大加速ラップ': 'median',
    'ゴール前ラップ差': 'median',
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'レイティング偏差値': 'median',
    'レイティング順位': 'median',
    'レイティング順位分布': 'median',
    'ZI指数': 'median',
    'ZI指数順位': 'median',
    'ZI指数偏差値': 'median',
    'ZI指数順位分布': 'median',
    '追切指数': 'median',
    '追切指数偏差値': 'median',
    '追切指数順位': 'median',
    '追切指数順位分布': 'median',
    'マイニング': 'median',
    'マイニング偏差値': 'median',
    'マイニング順位': 'median',
    'マイニング順位分布': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング偏差値': 'median',
    '対戦型マイニング順位': 'median',
    '対戦型マイニング順位分布': 'median',
    'PCI': 'median',
    'RPCI差': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '前半3F偏差値': 'median',
    '前半3F順位': 'median',
    '前半3F順位分布': 'median',
    '上り3F': 'median',
    '上り3F偏差値': 'median',
    '上り3F順位': 'median',
    '上り3F順位分布': 'median',
    'Ave-3F': 'median',
    'Ave-3F偏差値': 'median',
    'Ave-3F順位': 'median',
    'Ave-3F順位分布': 'median',
    '-3F差': 'median',
    'テン指数': 'median',
    'テン指数偏差値': 'median',
    'テン指数順位': 'median',
    'テン指数順位分布': 'median',
    '上り指数': 'median',
    '上り指数偏差値': 'median',
    '上り指数順位': 'median',
    '上り指数順位分布': 'median',
    '補正タイム': 'median',
    '補9': 'median',
    'スピード指数': 'median',
    'スピード指数偏差値': 'median',
    'スピード指数順位': 'median',
    'スピード指数順位分布': 'median',
    '総合指数': 'median',
    '総合指数偏差値': 'median',
    '総合指数順位': 'median',
    '総合指数順位分布': 'median',
    'レース強度指数': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '初角位置': 'median',
    '初角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}

agg_cols2 = {
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'レイティング偏差値': 'median',
    'レイティング順位': 'median',
    'レイティング順位分布': 'median',
    'ZI指数': 'median',
    'ZI指数順位': 'median',
    'ZI指数偏差値': 'median',
    'ZI指数順位分布': 'median',
    '追切指数': 'median',
    '追切指数偏差値': 'median',
    '追切指数順位': 'median',
    '追切指数順位分布': 'median',
    'マイニング': 'median',
    'マイニング偏差値': 'median',
    'マイニング順位': 'median',
    'マイニング順位分布': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング偏差値': 'median',
    '対戦型マイニング順位': 'median',
    '対戦型マイニング順位分布': 'median',
    'PCI': 'median',
    'RPCI差': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '前半3F偏差値': 'median',
    '前半3F順位': 'median',
    '前半3F順位分布': 'median',
    '上り3F': 'median',
    '上り3F偏差値': 'median',
    '上り3F順位': 'median',
    '上り3F順位分布': 'median',
    'Ave-3F': 'median',
    'Ave-3F偏差値': 'median',
    'Ave-3F順位': 'median',
    'Ave-3F順位分布': 'median',
    '-3F差': 'median',
    'テン指数': 'median',
    'テン指数偏差値': 'median',
    'テン指数順位': 'median',
    'テン指数順位分布': 'median',
    '上り指数': 'median',
    '上り指数偏差値': 'median',
    '上り指数順位': 'median',
    '上り指数順位分布': 'median',
    'スピード指数': 'median',
    'スピード指数偏差値': 'median',
    'スピード指数順位': 'median',
    'スピード指数順位分布': 'median',
    '補正タイム': 'median',
    '補9': 'median',
    '総合指数': 'median',
    '総合指数偏差値': 'median',
    '総合指数順位': 'median',
    '総合指数順位分布': 'median',
    'レース強度指数': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '初角位置': 'median',
    '初角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}


# --- 基準タイム算出処理1（元コードそのまま） ---
base_df1 = master_df[master_df['クラスコード'].isin([23, 43])].copy()

df_tops1 = base_df1[base_df1['入線順位'].between(1, 3)].copy()
df_1st1  = base_df1[base_df1['入線順位'] == 1].copy()

df_tops1.loc[:, 'クラス分類'] = '1勝・2勝クラス'
df_1st1.loc[:,  'クラス分類'] = '1勝・2勝クラス'

group_cols1 = ['場所', '芝・ダート', '距離', 'トラックコード(JV)','馬場分類', 'クラス分類']

base_time_tops1 = df_tops1.groupby(group_cols1, dropna=False).agg(agg_cols1).reset_index()
base_time_1st1  = df_1st1.groupby(group_cols1, dropna=False).agg(agg_cols2).reset_index()
base_time_1st1  = base_time_1st1.rename(columns={col: f"{col}_1着" for col in agg_cols2.keys()})

base_time_df1 = pd.merge(base_time_tops1, base_time_1st1, on=group_cols1, how="left")
base_time_df1['距離係数'] = (1 / base_time_df1['タイムS']) * 100

# --- 基準タイム算出処理2（元コードそのまま） ---
df_tops2 = master_df[master_df['入線順位'].between(1, 3)].copy()
df_1st2  = master_df[master_df['入線順位'] == 1].copy()

group_cols2 = ['場所', '芝・ダート', '距離','トラックコード(JV)', '競走種別', 'クラス分類', '馬場分類']

base_time_tops2 = df_tops2.groupby(group_cols2).agg(agg_cols1).reset_index()
base_time_1st2  = df_1st2.groupby(group_cols2).agg(agg_cols2).reset_index()
base_time_1st2  = base_time_1st2.rename(columns={col: f"{col}_1着" for col in agg_cols2.keys()})

base_time_df2 = pd.merge(base_time_tops2, base_time_1st2, on=group_cols2, how="left")
base_time_df2['距離係数'] = (1 / base_time_df2['タイムS']) * 100

# --- マージ（元コードそのまま） ---
def prepare_ref_df(df_ref, merge_keys, suffix):
    df_ref = df_ref.copy()
    rename_map = {c: f"{c}{suffix}" for c in df_ref.columns if c not in merge_keys}
    return df_ref.rename(columns=rename_map)

ref_df1 = prepare_ref_df(base_time_df1, group_cols1, '_stdtime1')
ref_df2 = prepare_ref_df(base_time_df2, group_cols2, '_stdtime2')

master_df = pd.merge(master_df, ref_df1, on=group_cols1, how='left')
master_df = pd.merge(master_df, ref_df2, on=group_cols2, how='left')

master_df['追切指数上位差分'] = master_df['追切指数'] - master_df['追切指数_stdtime1'].fillna(master_df['追切指数_stdtime2'])
master_df['追切指数勝馬差分'] = master_df['追切指数'] - master_df['追切指数_1着_stdtime1'].fillna(master_df['追切指数_1着_stdtime2'])
master_df['追切指数偏差値上位差分'] = master_df['追切指数偏差値'] - master_df['追切指数偏差値_stdtime1'].fillna(master_df['追切指数偏差値_stdtime2'])
master_df['追切指数偏差値勝馬差分'] = master_df['追切指数偏差値'] - master_df['追切指数偏差値_1着_stdtime1'].fillna(master_df['追切指数偏差値_1着_stdtime2'])
master_df['追切指数順位上位差分'] = master_df['追切指数順位'] - master_df['追切指数順位_stdtime1'].fillna(master_df['追切指数順位_stdtime2'])
master_df['追切指数順位勝馬差分'] = master_df['追切指数順位'] - master_df['追切指数順位_1着_stdtime1'].fillna(master_df['追切指数順位_1着_stdtime2'])
master_df['追切指数順位分布上位差分'] = master_df['追切指数順位分布'] - master_df['追切指数順位分布_stdtime1'].fillna(master_df['追切指数順位分布_stdtime2'])
master_df['追切指数順位分布勝馬差分'] = master_df['追切指数順位分布'] - master_df['追切指数順位分布_1着_stdtime1'].fillna(master_df['追切指数順位分布_1着_stdtime2'])

suffixes_to_drop = ['_stdtime1', '_1着_stdtime1','_stdtime2', '_1着_stdtime2']
master_df = master_df.loc[:, [c for c in master_df.columns if not any(c.endswith(suf) for suf in suffixes_to_drop)]].copy()

# 不要DF削除
safe_del(
    'base_df1','df_tops1','df_1st1','base_time_tops1','base_time_1st1',
    'df_tops2','df_1st2','base_time_tops2','base_time_1st2',
    'ref_df1','ref_df2'
)

# 保存
combined_df.to_csv(output_filepath1, index=False, encoding='cp932')
master_df.to_csv(master_filepath, index=False, encoding='cp932')
base_time_df1.to_csv(output_filepath2, index=False, encoding='cp932')
base_time_df2.to_csv(output_filepath3, index=False, encoding='cp932')

# 年度別ファイル分割
combined_df['year'] = pd.to_datetime(
    combined_df['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

for year, dfy in combined_df.groupby('year'):
    dfy.drop(columns='year').to_csv(
        f'{output_dir}Training_Score_Master_{year}.csv',
        index=False,
        encoding='cp932'
    )

safe_del('combined_df','master_df','base_time_df1','base_time_df2')

print("完了：追切指数（A/B/C基準）を出力して、マスタ＆StdTimeも更新しました。")


完了：追切指数（A/B/C基準）を出力して、マスタ＆StdTimeも更新しました。


# 成績データの列の並びを整える

In [10]:
import pandas as pd

in_path  = r"D:\Keiba\10_ExportData\ResultData\Processed\Race_Result_Master.csv"
out_path = r"D:\Keiba\10_ExportData\ResultData\Processed\Race_Result_Master.csv"

cols = [
    "target_horseid",
    "target_raceid",
    "日付S",
    "場所",
    "芝・ダート",
    "距離",
    "コース区分",
    "コーナー回数",
    "コースマーク",
    "コースグループ名1",
    "コースラベル",
    "トラックコード(JV)",
    "年齢限定(競走種別コード)",
    "競走記号コード",
    "競走種別",
    "重量コード",
    "クラスコード",
    "クラス分類",
    "グレードコード",
    "天候コード",
    "天候",
    "馬場状態コード",
    "馬場状態",
    "馬場分類",
    "頭数",
    "レース印",
    "レース印２",
    "レース印３",
    "馬場指数",
    "通過3F",
    "通過4F",
    "通過5F",
    "レース上り3F",
    "レース上り4F",
    "レース上り5F",
    "前後3F差",
    "前後4F差",
    "前後5F差",
    "1着入線タイム(秒)",
    "1-5着平均タイム(秒)",
    "2-5着平均タイム(秒)",
    "全馬平均タイム(秒)",
    "最速上3F",
    "基準タイム(秒)",
    "レースPCI",
    "PCI3",
    "33ラップ",
    "33ラップ判定",
    "中盤ラップ1",
    "中盤ラップ2",
    "最大加速ラップ",
    "ゴール前ラップ差",
    "Lap01",
    "Lap02",
    "Lap03",
    "Lap04",
    "Lap05",
    "Lap06",
    "Lap07",
    "Lap08",
    "Lap09",
    "Lap10",
    "Lap11",
    "Lap12",
    "Lap13",
    "Lap14",
    "Lap15",
    "Lap16",
    "Lap17",
    "Lap18",
    "Lap19",
    "Lap20",
    "Lap21",
    "Lap22",
    "Lap23",
    "Lap24",
    "Lap25",
    "血統登録番号",
    "馬名",
    "欧字馬名",
    "性別",
    "年齢",
    "誕生日",
    "生後日数",
    "生後日数上位差分",
    "生後日数勝馬差分",
    "種牡馬",
    "種牡馬タイプ名",
    "母馬名",
    "母の父馬名",
    "母の父タイプ名",
    "父×母の父タイプ名",
    "父タイプ名×母の父タイプ名",
    "母母父タイプ名",
    "生産者",
    "馬主コード",
    "馬主",
    "調教師コード",
    "多頭出し",
    "所属",
    "調教師",
    "騎手コード",
    "騎手",
    "生産者×馬主",
    "生産者×調教師",
    "生産者×騎手",
    "馬主×調教師",
    "馬主×騎手",
    "調教師×騎手",
    "調教師×コースラベル",
    "騎手×コースラベル",
    "ブリンカー",
    "枠番",
    "馬番",
    "体重",
    "増減",
    "斤量",
    "馬齢斤量",
    "馬齢斤量差",
    "斤量馬体重比",
    "基準斤量",
    "体重上位差分",
    "体重勝馬差分",
    "斤量馬体重比上位差分",
    "斤量馬体重比勝馬差分",
    "Ｃ",
    "去勢初戦",
    "転厩初戦",
    "キャリア",
    "間隔",
    "前走芝・ダ",
    "前走距離",
    "前走距離差",
    "前走距離上位差分",
    "前走距離勝馬差分",
    "前走距離差上位差分",
    "前走距離差勝馬差分",
    "レイティング",
    "レイティング順位",
    "レイティング偏差値",
    "レイティング順位分布",
    "レイティング上位差分",
    "レイティング勝馬差分",
    "レイティング偏差値上位差分",
    "レイティング偏差値勝馬差分",
    "レイティング順位分布上位差分",
    "レイティング順位分布勝馬差分",
    "レイティング平均値",
    "平均レイティング差分",
    "ZI指数",
    "ZI指数順位",
    "ZI指数順位分布",
    "ZI指数偏差値",
    "ZI指数上位差分",
    "ZI指数勝馬差分",
    "ZI指数偏差値上位差分",
    "ZI指数偏差値勝馬差分",
    "ZI指数順位分布上位差分",
    "ZI指数順位分布勝馬差分",
    "追切指数",
    "追切指数順位",
    "追切指数偏差値",
    "追切指数順位分布",
    "追切指数上位差分",
    "追切指数勝馬差分",
    "追切指数偏差値上位差分",
    "追切指数偏差値勝馬差分",
    "追切指数順位上位差分",
    "追切指数順位勝馬差分",
    "追切指数順位分布上位差分",
    "追切指数順位分布勝馬差分",
    "外部指数4",
    "外部指数順4",
    "マイニング",
    "マイニング順位",
    "マイニング偏差値",
    "マイニング順位分布",
    "マイニング上位差分",
    "マイニング勝馬差分",
    "マイニング偏差値上位差分",
    "マイニング偏差値勝馬差分",
    "マイニング順位分布上位差分",
    "マイニング順位分布勝馬差分",
    "対戦型マイニング",
    "対戦型マイニング順位",
    "対戦型マイニング偏差値",
    "対戦型マイニング順位分布",
    "対戦型マイニング上位差分",
    "対戦型マイニング勝馬差分",
    "対戦型マイニング偏差値上位差分",
    "対戦型マイニング偏差値勝馬差分",
    "対戦型マイニング順位分布上位差分",
    "対戦型マイニング順位分布勝馬差分",
    "人気",
    "単勝オッズ",
    "複勝人気",
    "複勝下限",
    "複勝上限",
    "単勝票数",
    "複勝票数",
    "単複票数比",
    "単勝シェア",
    "複勝シェア",
    "連絡みシェア",
    "ワイド絡みシェア",
    "馬単絡みシェア",
    "馬単1着軸シェア",
    "馬単2着軸シェア",
    "3連複絡みシェア",
    "レースタイプ",
    "レース強度指数",
    "レース強度指数上位差分",
    "レースレベル指数",
    "レースレベル判定",
    "レースレベル評価",
    "RPCI差",
    "PCI",
    "PCI3差分",
    "PCI上位差分",
    "PCI勝馬差分",
    "タイムS",
    "タイムS上位差分",
    "タイムS勝馬差分",
    "-3Fタイム",
    "-3F差",
    "-3Fタイム上位差分",
    "-3Fタイム勝馬差分",
    "-3F差上位差分",
    "-3F差勝馬差分",
    "1着差タイム",
    "時速(km/h)",
    "平均1Fタイム",
    "Ave-3F",
    "Ave-3F偏差値",
    "Ave-3F順位",
    "Ave-3F順位分布",
    "Ave-3F上位差分",
    "Ave-3F勝馬差分",
    "Ave-3F偏差値上位差分",
    "Ave-3F偏差値勝馬差分",
    "Ave-3F順位上位差分",
    "Ave-3F順位勝馬差分",
    "Ave-3F順位分布上位差分",
    "Ave-3F順位分布勝馬差分",
    "前半3F",
    "前半3F順位",
    "前半3F偏差値",
    "前半3F順位分布",
    "前半3F上位差分",
    "前半3F勝馬差分",
    "前半3F偏差値上位差分",
    "前半3F偏差値勝馬差分",
    "前半3F順位上位差分",
    "前半3F順位勝馬差分",
    "前半3F順位分布上位差分",
    "前半3F順位分布勝馬差分",
    "上り3F",
    "上り3F順位",
    "上り3F偏差値",
    "上り3F順位分布",
    "上り3F上位差分",
    "上り3F勝馬差分",
    "上り3F偏差値上位差分",
    "上り3F偏差値勝馬差分",
    "上り3F順位上位差分",
    "上り3F順位勝馬差分",
    "上り3F順位分布上位差分",
    "上り3F順位分布勝馬差分",
    "テン指数",
    "テン指数偏差値",
    "テン指数順位",
    "テン指数順位分布",
    "テン指数上位差分",
    "テン指数勝馬差分",
    "テン指数偏差値上位差分",
    "テン指数偏差値勝馬差分",
    "テン指数順位上位差分",
    "テン指数順位勝馬差分",
    "テン指数順位分布上位差分",
    "テン指数順位分布勝馬差分",
    "上り指数",
    "上り指数偏差値",
    "上り指数順位",
    "上り指数順位分布",
    "上り指数上位差分",
    "上り指数勝馬差分",
    "上り指数偏差値上位差分",
    "上り指数偏差値勝馬差分",
    "上り指数順位上位差分",
    "上り指数順位勝馬差分",
    "上り指数順位分布上位差分",
    "上り指数順位分布勝馬差分",
    "補正タイム",
    "補9",
    "コーナーロス",
    "補正走破タイム",
    "補正走破タイム上位差分",
    "補正走破タイム勝馬差分",
    "スローorハイ関数",
    "スローorハイ関数範囲",
    "馬場補正値",
    "スピード指数",
    "スピード指数偏差値",
    "スピード指数順位",
    "スピード指数順位分布",
    "スピード指数上位差分",
    "スピード指数勝馬差分",
    "スピード指数偏差値上位差分",
    "スピード指数偏差値勝馬差分",
    "スピード指数順位上位差分",
    "スピード指数順位勝馬差分",
    "スピード指数順位分布上位差分",
    "スピード指数順位分布勝馬差分",
    "総合指数",
    "総合指数偏差値",
    "総合指数順位",
    "総合指数順位分布",
    "総合指数上位差分",
    "総合指数勝馬差分",
    "総合指数偏差値上位差分",
    "総合指数偏差値勝馬差分",
    "総合指数順位上位差分",
    "総合指数順位勝馬差分",
    "総合指数順位分布上位差分",
    "総合指数順位分布勝馬差分",
    "Top3総合指数",
    "Top3総合指数差分",
    "通過1",
    "通過2",
    "通過3",
    "通過4",
    "初角サイドポジション",
    "2角サイドポジション",
    "3角サイドポジション",
    "4角サイドポジション",
    "4角位置",
    "初角位置",
    "初角_4角差",
    "初角_4角差上位差分",
    "初角_4角差勝馬差分",
    "4角_入線順位差",
    "4角_入線順位差上位差分",
    "4角_入線順位差勝馬差分",
    "サイドポジション平均",
    "入線順位",
    "決め手",
    "賞金",
    "付加賞金",
    "獲得賞金",
]

df = pd.read_csv(in_path, encoding='cp932')
df = df[cols]
df.to_csv(out_path, index=False, encoding='cp932')

# 過去n走分の特徴量を出力する

In [4]:
import sys
import time
import importlib.util
from pathlib import Path
from tqdm import tqdm

script_path = Path(r"D:\Keiba\Python\scripts\add_past_features.py")
spec = importlib.util.spec_from_file_location("add_past_features", script_path)
mod = importlib.util.module_from_spec(spec)

sys.modules[spec.name] = mod
spec.loader.exec_module(mod)

mod.run(
    input_path=r"D:\Keiba\10_ExportData\ResultData\Processed\Race_Result_Master.csv",
    surface_col="芝・ダート",
    surface_turf_value="芝",
    surface_dirt_value="ダート",
    course_label_col="コースラベル",
)


[info] input: D:\Keiba\10_ExportData\ResultData\Processed\Race_Result_Master.csv


past_numeric_chunks: 100%|██████████| 4/4 [19:33<00:00, 293.37s/it]


[info] saved: D:\Keiba\30_FeaturesData\Horse\horse_past_numeric_snapshot.csv
[info] saved: D:\Keiba\30_FeaturesData\Horse\horse_runningstyle_snapshot.csv
[info] saved: D:\Keiba\30_FeaturesData\Horse\horse_blinker_flags_snapshot.csv


horse_last5: 100%|██████████| 39833/39833 [04:25<00:00, 149.82it/s]


[info] saved: D:\Keiba\30_FeaturesData\Horse\horse_last5_perf_snapshot.csv
[info] saved: D:\Keiba\30_FeaturesData\Blinker\Horse\blinker_performance_snapshot.csv


...

[info] saved: D:\Keiba\30_FeaturesData\Blinker\Trainer\blinker_performance_snapshot.csv


blinker_rel_keys:  67%|██████▋   | 2/3 [00:00<00:00,  2.48it/s]

[info] saved: D:\Keiba\30_FeaturesData\Blinker\Jockey\blinker_performance_snapshot.csv


blinker_rel_keys: 100%|██████████| 3/3 [00:01<00:00,  2.32it/s]

[info] saved: D:\Keiba\30_FeaturesData\Blinker\Trainer_Jockey\blinker_performance_snapshot.csv
[info] done


# ↓アーカイブ

In [33]:
import pandas as pd
import numpy as np
import re
import os

from tkinter import Tk, filedialog
from IPython.display import display
from datetime import datetime

# 元になる成績データファイルのパス
master_filipath = r'D:\Keiba\10_ExportData\Race_Result_Master_org.csv'

# 成績データにマッピングするデータファイルのパス
# 初角位置
mapping_filepath1 = r'D:\Keiba\00_ImportData\80_First_Corner_Position\First_Corner_Position_Master.csv'
# 2角位置
mapping_filepath2 = r'D:\Keiba\00_ImportData\90_Second_Corner_Position\Second_Corner_Position_Master.csv'
# 3角位置
mapping_filepath3 = r'D:\Keiba\00_ImportData\100_Third_Corner_Position\Third_Corner_Position_Master.csv'
# 4角位置
mapping_filepath4 = r'D:\Keiba\00_ImportData\110_Fourth_Corner_Position\Fourth_Corner_Position_Master.csv'
# 上り位置
mapping_filepath5 = r'D:\Keiba\00_ImportData\120_Spurt_Position\Spurt_Position_Master.csv'
# 馬場指数
mapping_filepath6 = r'D:\Keiba\00_ImportData\140_Track_Condition\Track_Condition_Master.csv'
# 前半3Fタイム
mapping_filepath7 = r'D:\Keiba\00_ImportData\180_First3F_Lap\First3F_Lap_Master.csv'
# レイティング
mapping_filepath8 = r'D:\Keiba\00_ImportData\170_Rating_Score\Rating_Score_Master.csv'

# 保存先フォルダのパス
# 加工後の成績データファイルの保存先パス
output_filepath1 = r'D:\Keiba\10_ExportData\Race_Result_Master.csv'
# 基準タイムデータファイル1の保存先パス
output_filepath2 = r'D:\Keiba\20_IndexData\Base_Time_Master1.csv'
# 基準タイムデータファイル2の保存先パス
output_filepath3 = r'D:\Keiba\20_IndexData\Base_Time_Master2.csv'
# ペース係数データファイルの保存先パス
output_filepath4 = r'D:\Keiba\20_IndexData\Pace_Coefficient_Master.csv'
# 基準33ラップファイルの保存先パス
output_filepath5 = r'D:\Keiba\20_IndexData\Base_33Lap_Master.csv'

# 成績データファイルとマッピングファイルの読み込み
df1 = pd.read_csv(master_filipath, encoding='cp932').copy()
df2 = pd.read_csv(mapping_filepath1, encoding='cp932').copy()
df3 = pd.read_csv(mapping_filepath2, encoding='cp932').copy()
df4 = pd.read_csv(mapping_filepath3, encoding='cp932').copy()
df5 = pd.read_csv(mapping_filepath4, encoding='cp932').copy()
df6 = pd.read_csv(mapping_filepath5, encoding='cp932').copy()
df7 = pd.read_csv(mapping_filepath6, encoding='cp932').copy()
df8 = pd.read_csv(mapping_filepath7, encoding='cp932').copy()
df9 = pd.read_csv(mapping_filepath8, encoding='cp932').copy()

# カラムの整理
# df1の列名を変更
rename_map = {
    'レースID(新)': 'target_raceid',
    'レースID(新).1': 'target_horseid',
    '上り3F': 'レース上り3F',
    '上り4F': 'レース上り4F',
    '上り5F': 'レース上り5F',
    '外部指数1':'レイティング',
    '外部指数順1':'レイティング順位',
    '外部指数2':'テン指数',
    '外部指数順2':'テン指数順位',
    '外部指数3':'上り指数',
    '外部指数順3':'上り指数順位',
    '外部指数4':'スピード指数',
    '外部指数順4':'スピード指数順位',
    '上り3F.1':'上り3F'
}

# df1の列名に対してrename_mapを適用
cols = list(df1.columns)
seen = set()
for i, c in enumerate(cols):
    if c in rename_map and c not in seen:
        cols[i] = rename_map[c]
        seen.add(c)
df1.columns = cols

# 配当金額の列を作成する
# 列から取り出す金額の数を定義
target_cols = {
    '単勝配当表記': ('単勝', 1),
    '複勝配当表記': ('複勝', 3),
    '枠連配当表記': ('枠連', 1),
    '馬連配当表記': ('馬連', 1),
    'ワイド配当表記': ('ワイド', 3),
    '馬単配当表記': ('馬単', 1),
    '３連複配当表記': ('３連複', 1),
    '３連単配当表記': ('３連単', 1),
}

# 金額を取り出す正規表現パターン
pattern = re.compile(r'[\\¥]\s*([0-9,]+)(?=\s*(?:\(|/|$))')

def pick_amounts(text, take=1):
    if pd.isna(text):
        return [np.nan]*take
    s = str(text)
    found = pattern.findall(s)
    nums = []
    for x in found:
        try:
            nums.append(int(x.replace(',', '')))
        except:
            # 変な値が来てもスルー
            continue
    # 必要な個数だけ先頭から取り、足りなければNaNで埋める
    nums = nums[:take]
    if len(nums) < take:
        nums += [np.nan]*(take - len(nums))
    return nums

for src_col, (base_name, take) in target_cols.items():
    if src_col not in df1.columns:
        # その列が無い場合はスキップ
        continue

    # 値を取り出す
    values = df1[src_col].apply(lambda x: pick_amounts(x, take))

    # 1個だけなら「単勝」のように1列、3個なら「複勝1, 複勝2, 複勝3」の3列を作る
    if take == 1:
        col_name = base_name
        df1[col_name] = values.apply(lambda v: v[0] if isinstance(v, list) else np.nan)
        # Convert to integer, coercing errors to NaN
        df1[col_name] = pd.to_numeric(df1[col_name], errors='coerce').astype('Int64')
    else:
        for i in range(take):
            col_name = f'{base_name}{i+1}'
            df1[col_name] = values.apply(lambda v: v[i] if isinstance(v, list) and len(v) > i else np.nan)
            # Convert to integer, coercing errors to NaN
            df1[col_name] = pd.to_numeric(df1[col_name], errors='coerce').astype('Int64')

#　種牡馬名のspace削除
df1['種牡馬'] = df1['種牡馬'].astype(str).str.replace(r'\s+', '', regex=True)

# 血統データの列を追加
df1['父×母父系統'] = df1['種牡馬'].astype(str) + '×' + df1['母の父タイプ名'].astype(str)
df1['父系統×母父系統'] = df1['種牡馬タイプ名'].astype(str) + '×' + df1['母の父タイプ名'].astype(str)

# マッピング対象の値をdf1の別列に代入
if '前半3F' not in df1.columns:
    df1['前半3F'] = pd.NA
OVERWRITE = True  # 既存も上書きする → Falseなら空欄だけ埋める

# キーの型を揃える（数字/ゼロ埋めブレ防止）
for c in ['target_horseid', 'target_raceid']:
    if c in df1: df1[c] = df1[c].astype(str)

for d in [df2, df3, df4, df5, df6, df7, df8, df9]:
    for c in ['target_horseid', 'target_raceid']:
        if c in d: d[c] = d[c].astype(str)

# 初角位置の代入
left  = df1.set_index('target_horseid')
right = df2[['target_horseid','馬印4']].dropna(subset=['馬印4']).set_index('target_horseid')
left.update(right, overwrite=OVERWRITE)
df1 = left.reset_index()
# 2角位置の代入
left  = df1.set_index('target_horseid')
right = df3[['target_horseid','馬印5']].dropna(subset=['馬印5']).set_index('target_horseid')
left.update(right, overwrite=OVERWRITE)
df1 = left.reset_index()
# 3角位置の代入
left  = df1.set_index('target_horseid')
right = df4[['target_horseid','馬印6']].dropna(subset=['馬印6']).set_index('target_horseid')
left.update(right, overwrite=OVERWRITE)
df1 = left.reset_index()
# 4角位置の代入
left  = df1.set_index('target_horseid')
right = df5[['target_horseid','馬印7']].dropna(subset=['馬印7']).set_index('target_horseid')
left.update(right, overwrite=OVERWRITE)
df1 = left.reset_index()
# 上り位置の代入
left  = df1.set_index('target_horseid')
right = df6[['target_horseid','馬印3']].dropna(subset=['馬印3']).set_index('target_horseid')
left.update(right, overwrite=OVERWRITE)
df1 = left.reset_index()
# 馬場指数の代入
left  = df1.set_index('target_raceid')
right = df7[['target_raceid','レース印']].dropna(subset=['レース印']).set_index('target_raceid')
left.update(right, overwrite=OVERWRITE)
df1 = left.reset_index()
# 前半3Fタイムの代入
left  = df1.set_index('target_horseid')
right = df8[['target_horseid','前半3F']].dropna(subset=['前半3F']).set_index('target_horseid')
left.update(right, overwrite=OVERWRITE)
df1 = left.reset_index()
# レイティングの代入
left  = df1.set_index('target_horseid')
right = df9[['target_horseid','レイティング']].dropna(subset=['レイティング']).set_index('target_horseid')
left.update(right, overwrite=OVERWRITE)
df1 = left.reset_index()

# --- 関数群 ---
def calculate_33_lap(df1):
    """距離ごとに33ラップを計算する"""
    rap_33 = []
    distance_to_lap_range = {
        1000: (0, 3),
        1150: (0, 3),  # 1150mは後で補正
        1200: (0, 3),
        1300: (1, 4),
        1400: (1, 4),
        1500: (2, 5),
        1600: (2, 5),
        1700: (3, 6),
        1800: (3, 6),
        1900: (4, 7),
        2000: (4, 7),
        2100: (5, 8),
        2200: (5, 8),
        2300: (6, 9),
        2400: (6, 9),
        2500: (7, 10),
        2600: (7, 10),
        2700: (8, 11),
        2800: (8, 11),
        2900: (9, 12),
        3000: (9, 12),
        3100: (10, 13),
        3200: (10, 13),
        3300: (11, 14),
        3400: (11, 14),
        3500: (12, 15),
        3600: (12, 15),
    }
    for _, row in df1.iterrows():
        distance = row['距離']
        lap_times = [row[f'Lap{str(i+1).zfill(2)}'] for i in range(25)]
        lap_times = [t for t in lap_times if not pd.isna(t)]
        if distance in distance_to_lap_range:
            start, end = distance_to_lap_range[distance]
            sum_6to4 = sum(lap_times[start:end])
        elif distance == 1150:
            sum_6to4 = round(lap_times[0] * 1.25, 1) + lap_times[1] + lap_times[2] if len(lap_times) >= 3 else None
        else:
            sum_6to4 = None
        sum_3to1 = row['レース上り3F'] if 'レース上り3F' in row and not pd.isna(row['レース上り3F']) else None
        rap_33_value = sum_6to4 - sum_3to1 if not pd.isna(sum_6to4) and not pd.isna(sum_3to1) else None
        rap_33.append(rap_33_value)
    df1['33ラップ'] = rap_33
    return df1

def calculate_middle_lap(df1):
    """距離ごとに中盤ラップ1・2を計算する"""
    middle_lap1 = []
    middle_lap2 = []
    distance_to_mid_lap = {
        1000: (2, 4, None, None),
        1150: (2, 4, None, None),
        1200: (2, 4, None, None),
        1300: (2, 4, None, None),
        1400: (2, 4, None, None),
        1500: (3, 5, None, None),
        1600: (3, 5, None, None),
        1700: (3, 6, None, None),
        1800: (3, 6, None, None),
        1900: (3, 7, None, None),
        2000: (3, 7, None, None),
        2100: (3, 5, 5, 8),
        2200: (3, 5, 5, 8),
        2300: (3, 5, 5, 8),
        2400: (3, 6, 6, 9),
        2500: (3, 6, 6, 9),
        2600: (3, 7, 7, 10),
        3000: (3, 8, 8, 12),
        3200: (3, 8, 8, 13),
        3400: (3, 9, 9, 14),
        3600: (3, 9, 9, 15),
    }
    for _, row in df1.iterrows():
        distance = row['距離']
        lap_times = [row[f'Lap{str(i+1).zfill(2)}'] for i in range(25)]
        lap_times = [t for t in lap_times if not pd.isna(t)]
        if distance in distance_to_mid_lap:
            mid1_start, mid1_end, mid2_start, mid2_end = distance_to_mid_lap[distance]
            mid1 = sum(lap_times[mid1_start:mid1_end]) if mid1_start is not None else None
            mid2 = sum(lap_times[mid2_start:mid2_end]) if mid2_start is not None else None
        else:
            mid1 = None
            mid2 = None
        middle_lap1.append(mid1)
        middle_lap2.append(mid2)
    df1['中盤ラップ1'] = middle_lap1
    df1['中盤ラップ2'] = middle_lap2
    return df1

def calculate_lap_features(df1):
    """
    Lap01～Lap25 を使って
      ・最大加速（隣接ラップ差分の最小値）
      ・ゴール前ラップ差（ラスト1F - ラスト2F）
    を計算して df に列を追加する。
    """
    lap_cols = [f'Lap{str(i).zfill(2)}' for i in range(1, 26)]

    def _calc_row(row):
        # その馬のラップ一覧（NaN は除外）
        laps = []
        for c in lap_cols:
            if c in row.index and pd.notna(row[c]):
                laps.append(row[c])

        # ラップが1つ以下ならどっちも計算不能
        if len(laps) < 2:
            return pd.Series({'最大加速ラップ': np.nan, 'ゴール前ラップ差': np.nan})

        laps = np.array(laps, dtype=float)

        # 隣り合う差分（後ろ - 前）
        diffs = np.diff(laps)   # 例：Lap02-Lap01, Lap03-Lap02, ...

        # 最大加速 = 最もマイナスが大きい差分（＝最小値）
        max_accel = diffs.min() if len(diffs) > 0 else np.nan

        # 終盤ラップ差 = ラスト1F - ラスト2F
        last_diff = laps[-1] - laps[-2] if len(laps) >= 2 else np.nan

        return pd.Series({
            '最大加速ラップ': max_accel if pd.notna(max_accel) else np.nan,
            'ゴール前ラップ差': last_diff if pd.notna(last_diff) else np.nan
        })

    new_cols = df1.apply(_calc_row, axis=1)
    df1['最大加速ラップ'] = new_cols['最大加速ラップ'].round(1)
    df1['ゴール前ラップ差'] = new_cols['ゴール前ラップ差'].round(1)

    return df1

def calculate_days_since_birth(df1):
    """生後日数を計算する"""
    birth_days = []
    for _, row in df1.iterrows():
        race_date = datetime.strptime(row['日付S'], '%Y.%m.%d')
        birth_str = row['誕生日'].replace(" ", "").replace("日", "").replace("-", "")
        birth_month, birth_day = map(int, birth_str.replace("月", " ").split())
        birth_year = race_date.year - row['年齢']
        try:
            birth_date = datetime(birth_year, birth_month, birth_day)
        except ValueError:
            birth_date = datetime(birth_year, 2, 28)
        days_old = (race_date - birth_date).days
        birth_days.append(days_old)
    df1['生後日数'] = birth_days
    return df1

def calculate_distance_diff(df1):
    """前走距離との差を計算する"""
    df1['前走距離差'] = df1['距離'] - df1['前走距離']
    return df1

def calculate_firsthalf_diff(df1):
    """1角から4角位置の差分を計算"""
    df1['1角_4角差'] = df1.apply(lambda row: row['通過3'] - row['通過4'] if pd.isna(row['通過1']) else row['通過1'] - row['通過4'], axis=1).fillna(0)
    return df1

def calculate_goal_diff(df1):
    """4角位置から入線順位の差分を計算"""
    df1['入線順位'] = df1['入線順位'].apply(lambda x: df1['頭数'][df1['入線順位'] == x].values[0] if x == 0 else x)
    df1['4角_入線順位差'] = df1['通過4'].fillna(0) - df1['入線順位'].fillna(0)
    return df1

def calculate_sideposition(df1):
    """サイドポジションの平均を計算する"""
    cols = ['馬印4', '馬印5', '馬印6', '馬印7']
    df1['サイドポジション平均'] = df1[cols].mean(axis=1)
    return df1

def calculate_totalprize(df1):
    """獲得賞金を計算する"""
    df1['獲得賞金'] = df1['賞金'].fillna(0) + df1['付加賞金'].fillna(0)
    return df1

def convert_time_to_seconds(time_str):
    """タイム表記を秒数に変換"""
    try:
        parts = time_str.split(".")
        if len(parts) == 3:
            minutes, seconds, tenths = map(int, parts)
            total_seconds = minutes * 60 + seconds + tenths * 0.1
        else:
            return np.nan
        return total_seconds
    except:
        return np.nan

def remove_plus_sign(value):
    """数値データから `+` を削除して変換"""
    try:
        return float(str(value).replace("+", ""))
    except:
        return np.nan

def extract_weight(value):
    """斤量の数値部分だけを抽出"""
    try:
        match = re.search(r'\d+', str(value))
        return int(match.group()) if match else np.nan
    except:
        return np.nan

def calculate_corner_loss(row):
    """コーナーロスを計算"""
    corner_positions = [
        row.get('馬印4', 1) - 1,
        row.get('馬印5', 1) - 1,
        row.get('馬印6', 1) - 1,
        row.get('馬印7', 1) - 1
    ]
    total_distance_loss = sum(corner_positions) * 1.5  # m単位
    finish_time_seconds = row['タイムS']
    distance_m = row['距離']
    if pd.isna(finish_time_seconds) or pd.isna(distance_m) or finish_time_seconds == 0:
        return np.nan
    avg_speed = distance_m / finish_time_seconds if finish_time_seconds > 0 else np.nan
    corner_loss = total_distance_loss / avg_speed if avg_speed > 0 else 0
    return round(corner_loss, 2)

# 各列の型変換
df1['レース印'] = df1['レース印'].astype(str).str.extract(r'(-?\d+)')[0].astype('Int64')
df1['体重'] = pd.to_numeric(df1['体重'], errors='coerce')
df1['Ave-3F'] = pd.to_numeric(df1['Ave-3F'], errors='coerce')
df1['上り3F'] = pd.to_numeric(df1['上り3F'], errors='coerce')

for col in ['前後3F差', '前後4F差', '前後5F差', '増減']:
    df1[col] = df1[col].apply(remove_plus_sign)

for corner_col in ['馬印4', '馬印5', '馬印6', '馬印7']:
    df1[corner_col] = pd.to_numeric(df1[corner_col], errors='coerce').fillna(1).astype(int)

df1['タイムS'] = df1['タイムS'].apply(convert_time_to_seconds)

df1['斤量'] = df1['斤量'].apply(extract_weight)

# 各種計算関数を順次実行
df1 = calculate_33_lap(df1)
df1 = calculate_middle_lap(df1)
df1 = calculate_lap_features(df1)
df1 = calculate_days_since_birth(df1)
df1 = calculate_distance_diff(df1)
df1 = calculate_firsthalf_diff(df1)
df1 = calculate_goal_diff(df1)
df1 = calculate_sideposition(df1)
df1 = calculate_totalprize(df1)

df1['-3Fタイム'] = df1['-3Fタイム'].apply(convert_time_to_seconds)
df1['基準斤量'] = df1['斤量'] - df1['馬齢斤量差']
df1['コーナーロス'] = df1.apply(calculate_corner_loss, axis=1)
df1['補正走破タイム'] = df1['タイムS'] - df1['コーナーロス']
df1['スローorハイ関数'] = df1['Ave-3F'] - df1['上り3F']

# マージ用の列作成処理
def categorize_race_type(race_type):
    if race_type == 11:
        return 'サラブレッド系2歳'
    elif race_type == 12:
        return 'サラブレッド系3歳'
    elif race_type >= 13:
        return 'サラブレッド系3歳以上'
    else:
        return np.nan

def categorize_class_code(class_code):
    if 7 <= class_code <= 15:
        return '新馬・未勝利'
    elif class_code == 23:
        return '1勝クラス'
    elif class_code == 43:
        return '2勝クラス'
    elif class_code == 67:
        return '3勝クラス'
    elif class_code >= 115:
        return 'OP・重賞'
    else:
        return np.nan

# 範囲定義
ranges = [
    (-np.inf, -4.6), (-4.6, -3.6), (-3.6, -2.6), (-2.6, -1.6), (-1.6, -0.6),
    (-0.6, 0.6), (0.6, 1.6), (1.6, 2.6), (2.6, 3.6), (3.6, 4.6), (4.6, np.inf)
]

# 区間ラベル
def assign_range(value):
    if pd.isna(value):
        return np.nan
    for i, (lower, upper) in enumerate(ranges):
        if i < len(ranges) - 1:
            if lower <= value < upper:
                return f"{lower}～{upper}"
        else:
            # 最終区間（4.6～inf）は右も含める
            if lower <= value <= upper:
                return f"{lower}～{upper}"
    return np.nan  # 念のため

# 馬場分類・競走種別・クラス分類の列を作成する
df1['馬場分類'] = df1['馬場状態'].replace({'良': '良・稍重','稍': '良・稍重','重': '重・不良','不': '重・不良'})
df1['競走種別'] = df1['年齢限定(競走種別コード)'].apply(categorize_race_type)
df1['クラス分類'] = df1['クラスコード'].apply(categorize_class_code)
df1['スローorハイ関数範囲'] = df1['スローorハイ関数'].apply(assign_range)

# --- 集計設定 ---
agg_cols1 = {
    'レースPCI': 'median',
    'PCI3': 'median',
    'PCI': 'median',
    '通過3F': 'median',
    '通過4F': 'median',
    '通過5F': 'median',
    'レース上り3F': 'median',
    'レース上り4F': 'median',
    'レース上り5F': 'median',
    '中盤ラップ1': 'median',
    '中盤ラップ2': 'median',
    '33ラップ': 'median',
    '最大加速ラップ': 'median',
    'ゴール前ラップ差': 'median',
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'マイニング': 'median',
    'マイニング順位': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング順位': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '上り3F': 'median',
    '上り3F順位': 'median',
    'Ave-3F': 'median',
    '-3F差': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '1角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}

agg_cols2 = {
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'マイニング': 'median',
    'マイニング順位': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング順位': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '上り3F': 'median',
    '上り3F順位': 'median',
    'Ave-3F': 'median',
    '-3F差': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '1角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}

# --- 基準タイム算出処理1 ---
# 1勝クラス・2勝クラスのみ対象
base_df1 = df1[df1['クラスコード'].isin([23, 43])].copy()

# 1着～3着と1着馬のデータ抽出
df_tops1 = base_df1[base_df1['入線順位'].between(1, 3)].copy()
df_1st1 = base_df1[base_df1['入線順位'] == 1].copy()

# クラス分類の統一
df_tops1.loc[:, 'クラス分類'] = '1勝・2勝クラス'
df_1st1.loc[:, 'クラス分類'] = '1勝・2勝クラス'

# グループ化キー
group_cols1 = ['場所', '芝・ダート', '距離', 'トラックコード(JV)','馬場分類', 'クラス分類']

# --- 集計処理 ---
base_time_tops1 = df_tops1.groupby(group_cols1, dropna=False).agg(agg_cols1).reset_index()
base_time_1st1 = df_1st1.groupby(group_cols1, dropna=False).agg(agg_cols2).reset_index()
base_time_1st1 = base_time_1st1.rename(columns={col: f"{col}_1着" for col in agg_cols2.keys()})

# --- 結合 ---
base_time_df1 = pd.merge(base_time_tops1, base_time_1st1, on=group_cols1, how="left")

# --- 距離係数（速度換算） ---
base_time_df1['距離係数'] = (1 / base_time_df1['タイムS']) * 100

# --- 基準タイム算出処理2 ---

df_tops2 = df1[df1['入線順位'].between(1, 3)].copy()
df_1st2 = df1[df1['入線順位'] == 1].copy()

# グループ化キー
group_cols2 = ['場所', '芝・ダート', '距離','トラックコード(JV)', '競走種別', 'クラス分類', '馬場分類']

base_time_tops2 = df_tops2.groupby(group_cols2).agg(agg_cols1).reset_index()
base_time_1st2 = df_1st2.groupby(group_cols2).agg(agg_cols2).reset_index()
base_time_1st2 = base_time_1st2.rename(columns={col: f"{col}_1着" for col in agg_cols2.keys()})
base_time_df2 = pd.merge(base_time_tops2, base_time_1st2, on=group_cols2, how="left")

# --- ペース係数算出処理 ---

# 外れ値除外（IQR）
def remove_outliers(group):
    Q1 = group['タイムS'].quantile(0.25)
    Q3 = group['タイムS'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return group[(group['タイムS'] >= lower_bound) & (group['タイムS'] <= upper_bound)]

# 区間ラベル
def assign_range(value):
    if pd.isna(value):
        return np.nan
    for i, (lower, upper) in enumerate(ranges):
        if i < len(ranges) - 1:
            if lower <= value < upper:
                return f"{lower}～{upper}"
        else:
            # 最終区間（4.6～inf）は右も含める
            if lower <= value <= upper:
                return f"{lower}～{upper}"
    return np.nan  # 念のため

# グルーピング＆IQR除外
pace_df = (
    df1.groupby(['場所', '芝・ダート', '距離','トラックコード(JV)'])
       .apply(remove_outliers)
       .reset_index(drop=True)
       .copy()
)

# 区間ラベリング
pace_df['スローorハイ関数範囲'] = pace_df['スローorハイ関数'].apply(assign_range)

# 区間別の中央値（タイムS & スローorハイ関数）
grouped_medians = pace_df.groupby(
    ['場所', '芝・ダート', '距離','トラックコード(JV)', 'スローorハイ関数範囲']
).agg({
    'タイムS': 'median',
    'スローorハイ関数': 'median'
}).reset_index()

# --- ベースライン作成 ---
# 優先：-0.6～0.6（中庸帯）
# 代用：0.6～1.6 / -1.6～-0.6 の平均 → どちらか片方だけでも可
baseline_medians = {}
for (location, turf_dirt, distance,track_code), group_df in grouped_medians.groupby(['場所', '芝・ダート', '距離','トラックコード(JV)']):
    baseline_row = group_df[group_df['スローorハイ関数範囲'] == '-0.6～0.6']
    if not baseline_row.empty:
        baseline_median = float(baseline_row['タイムS'].iloc[0])
    else:
        range_negative = group_df[group_df['スローorハイ関数範囲'] == '-1.6～-0.6']
        range_positive = group_df[group_df['スローorハイ関数範囲'] == '0.6～1.6']
        if not range_negative.empty and not range_positive.empty:
            baseline_median = (float(range_negative['タイムS'].median()) +
                               float(range_positive['タイムS'].median())) / 2.0
        elif not range_negative.empty:
            baseline_median = float(range_negative['タイムS'].median())
        elif not range_positive.empty:
            baseline_median = float(range_positive['タイムS'].median())
        else:
            baseline_median = np.nan
    baseline_medians[(location, turf_dirt, distance,track_code)] = baseline_median

# ベースラインとの差（スローorハイ関数差指数）
def calculate_difference(row):
    key = (row['場所'], row['芝・ダート'], row['距離'],row['トラックコード(JV)'])
    baseline_time = baseline_medians.get(key, np.nan)
    return row['タイムS'] - baseline_time if not pd.isna(baseline_time) else np.nan

grouped_medians['スローorハイ関数差指数'] = grouped_medians.apply(calculate_difference, axis=1)

# 係数計算（0除算・NaN安全＋クリップ）
def calculate_pace_adjustment_coefficient(row):
    val = row['スローorハイ関数']
    diff = row['スローorハイ関数差指数']
    if pd.notna(val) and val != 0 and pd.notna(diff):
        coefficient = round(diff / val, 2)
        return float(np.clip(coefficient, -3, 3))
    return 0.0

grouped_medians['ペース補正係数'] = grouped_medians.apply(calculate_pace_adjustment_coefficient, axis=1)

# --- 基準33ラップ算出処理 ---
lap33_df = (
    df1
      .dropna(subset=['33ラップ'])  # まず欠損を除外
      .groupby(['場所', '芝・ダート', '距離', 'トラックコード(JV)'], dropna=False)['33ラップ']
      .agg(['mean', 'std'])  # mean と std を同時に集計
      .reset_index()
).copy()

# 近似Zスコア用の基準値を計算
lap33_df['33ラップ±0'] = lap33_df['mean'].round(2)
lap33_df['33ラップ+1'] = (lap33_df['mean'] + lap33_df['std']).round(2)
lap33_df['33ラップ-1'] = (lap33_df['mean'] - lap33_df['std']).round(2)
lap33_df['33ラップ+2'] = (lap33_df['mean'] + 2 * lap33_df['std']).round(2)
lap33_df['33ラップ-2'] = (lap33_df['mean'] - 2 * lap33_df['std']).round(2)

# 必要なカラムだけ残す
lap33_df = lap33_df[
    ['場所', '芝・ダート', '距離', 'トラックコード(JV)',
     '33ラップ-2', '33ラップ-1', '33ラップ±0', '33ラップ+1', '33ラップ+2',
     'mean', 'std']
]

# CSV出力
df1.to_csv(output_filepath1, index=False, encoding='cp932')
base_time_df1.to_csv(output_filepath2, index=False, encoding='cp932')
base_time_df2.to_csv(output_filepath3, index=False, encoding='cp932')
grouped_medians.to_csv(output_filepath4, index=False, encoding='cp932')
lap33_df.to_csv(output_filepath5, index=False, encoding='cp932')

C:\Users\rotte\AppData\Local\Temp\ipykernel_19984\1735542871.py:44: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(master_filipath, encoding='cp932').copy()
C:\Users\rotte\AppData\Local\Temp\ipykernel_19984\1735542871.py:185: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['OH-4' 'OH-4' 'OH-4' ... 'OI-12' 'OI-12' 'OI-12']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  left.update(right, overwrite=OVERWRITE)
c:\Users\rotte\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\rotte\AppData\Local\Programs\Python\Python313\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdim

## 成績データへ指数の追加

In [34]:
import pandas as pd
import numpy as np
import re
import os

# 指数追加用成績データファイルのパス
master_filepath = r'D:\Keiba\10_ExportData\Race_Result_Master.csv'

# 参照用基準タイムファイルのパス
ref_filepath1 = r'D:\Keiba\20_IndexData\Base_Time_Master1.csv'
ref_filepath2 = r'D:\Keiba\20_IndexData\Base_Time_Master2.csv'
ref_filepath3 = r'D:\Keiba\20_IndexData\Pace_Coefficient_Master.csv'
ref_filepath4 = r'D:\Keiba\20_IndexData\Base_33Lap_Master.csv'

# 派生マスタcsvファイルのパス
output_filepath1 = r'D:\Keiba\00_ImportData\160_33Lap_Category\33Lap_Category_Master.csv'

# 共通関数：参照側DFのキー以外の列にサフィックスを付ける
def prepare_ref_df(df_ref, merge_keys, suffix):
    """
    マージ用の参照DFの列名に suffix を付ける（キー列以外すべて）
    """
    df_ref = df_ref.copy()
    rename_map = {
        c: f"{c}{suffix}"
        for c in df_ref.columns
        if c not in merge_keys
    }
    df_ref = df_ref.rename(columns=rename_map)
    return df_ref

# ファイルを開く
df1 = pd.read_csv(master_filepath, encoding='CP932').copy()
df2 = pd.read_csv(ref_filepath1, encoding='CP932').copy()
df3 = pd.read_csv(ref_filepath2, encoding='CP932').copy()
df4 = pd.read_csv(ref_filepath3, encoding='CP932').copy()
df5 = pd.read_csv(ref_filepath4, encoding='CP932').copy()

# マージキー
merge_key1 = ['場所','芝・ダート','距離','トラックコード(JV)','馬場分類']
merge_key2 = ['場所','芝・ダート','距離','トラックコード(JV)','競走種別','クラス分類','馬場分類']
merge_key3 = ['場所','芝・ダート','距離','トラックコード(JV)','スローorハイ関数範囲']
merge_key4 = ['場所','芝・ダート','距離','トラックコード(JV)']

# 参照側の列名にsuffixを付けてからマージ
df2_pre = prepare_ref_df(df2, merge_key1, '_stdtime1')
df3_pre = prepare_ref_df(df3, merge_key2, '_stdtime2')
df4_pre = prepare_ref_df(df4, merge_key3, '_paceindex')
df5_pre = prepare_ref_df(df5, merge_key4, '_33lap')

# df1へその他データフレームをマージ
merged_df1 = pd.merge(df1, df2_pre, on=merge_key1, how='left')
merged_df1 = pd.merge(merged_df1, df3_pre, on=merge_key2, how='left')
merged_df1 = pd.merge(merged_df1, df4_pre, on=merge_key3, how='left')
merged_df1 = pd.merge(merged_df1, df5_pre, on=merge_key4, how='left')

# 各指数算出の前処理
merged_df1['平均3F補正値'] = merged_df1['Ave-3F_stdtime1'] - merged_df1['Ave-3F']
merged_df1['馬場指数'] = merged_df1['レース印'] / 10
merged_df1['斤量補正値'] = merged_df1['斤量'] - merged_df1['基準斤量']
merged_df1['クラス補正値'] = merged_df1['タイムS_stdtime1'] - merged_df1['タイムS_stdtime2']
merged_df1['ペース補正値'] = merged_df1['スローorハイ関数'] * merged_df1['ペース補正係数_paceindex']

merged_df1['初角位置'] = merged_df1['通過1'].fillna(merged_df1['通過2']) # 1角がない場合は2角で補完
merged_df1['初角位置'] = merged_df1['初角位置'].fillna(merged_df1['通過3']) # 2角がない場合は3角で補完
merged_df1['初角位置'] = merged_df1['初角位置'].fillna(merged_df1['通過4']) # 3角がない場合は4角で補完

# テン指数の計算
merged_df1['テン指数'] = round((
(merged_df1['前半3F_stdtime1'].fillna(merged_df1['前半3F_stdtime2']) - merged_df1['前半3F']) # 基準タイムとの比較評価
+ ((merged_df1['頭数'] - merged_df1['初角位置'] + 1) / merged_df1['頭数']) # 初角の位置取り評価
+ 50
), 1)

# 上り指数の計算
merged_df1['上り指数'] = round((
(merged_df1['レース上り3F_stdtime1'].fillna(merged_df1['レース上り3F_stdtime2']) - merged_df1['レース上り3F']) # 基準タイムとの比較評価
+ ((merged_df1['頭数']) - merged_df1['入線順位'] + 1) / merged_df1['頭数'] # 着順評価
+ (merged_df1['通過4'] - merged_df1['入線順位']) / merged_df1['頭数'] # ポジション押し上げ力評価
+ (merged_df1['頭数'] - merged_df1['通過4'] + 1) / merged_df1['頭数'] # 4角の位置取り評価
+ 50
), 1)

# スピード指数の計算
merged_df1['スピード指数'] = round((
    (((merged_df1['タイムS'].fillna(0))
     - (merged_df1['補正走破タイム'].fillna(0)))
      * merged_df1['距離係数_stdtime1'].fillna(0))
  + merged_df1['馬場指数'].fillna(0)
  + merged_df1['平均3F補正値'].fillna(0)
  + merged_df1['斤量補正値'].fillna(0)
  + merged_df1['クラス補正値'].fillna(0)
  + merged_df1['ペース補正値'].fillna(0)
  + 100
), 1)

# 総合指数の計算
merged_df1['総合指数'] = round(
    (merged_df1['スピード指数'].fillna(0) + merged_df1['補正タイム'].fillna(0) + merged_df1['補9'].fillna(0)) / 3, 1
)

# 総合指数の代表値(1着～3着)をdf1へ追加
central_score_df1 = merged_df1[merged_df1['入線順位'].between(1, 3)].copy()
central_score_df1 = central_score_df1.groupby('target_raceid' ,as_index = False)['総合指数'].mean()
central_score_df1.rename(columns={'総合指数': 'Top3総合指数'}, inplace=True)
central_score_df1['Top3総合指数'] = round(central_score_df1['Top3総合指数'], 1)
merged_df1 = pd.merge(merged_df1, central_score_df1, on='target_raceid', how='left',suffixes=('', '_centralscore1')).copy()

# レイティングの平均値をdf1へ追加
merged_df1['レイティング'] = pd.to_numeric(merged_df1['レイティング'], errors='coerce')
central_score_df2 = df1.groupby('target_raceid' ,as_index = False)['レイティング'].mean().copy()
central_score_df2.rename(columns={'レイティング': 'レイティング平均値'}, inplace=True)
central_score_df2['レイティング平均値'] = round(central_score_df2['レイティング平均値'], 1)
merged_df1 = pd.merge(merged_df1, central_score_df2, on='target_raceid', how='left' ,suffixes=('', '_centralscore2')).copy()

# 基準総合指数と総合指数の差分をdf1へ追加
merged_df1['基準総合指数差分'] = merged_df1['総合指数'] - merged_df1['Top3総合指数']

# レイティング平均値とレイティングの差分を追加
merged_df1['平均レイティング差分'] = merged_df1['レイティング'] - merged_df1['レイティング平均値']

# レースごと（target_raceidごと）に大きい順で順位をつける
# 同点は同順位にして、次は飛ばさない（例：1位,1位,2位,3位…）＝ "dense" 方式
merged_df1['レイティング順位'] = merged_df1.groupby('target_raceid')['レイティング'].rank(ascending=False, method='dense').astype('Int64')
merged_df1['テン指数順位']     = merged_df1.groupby('target_raceid')['テン指数']    .rank(ascending=False, method='dense').astype('Int64')
merged_df1['上り指数順位']   = merged_df1.groupby('target_raceid')['上り指数']  .rank(ascending=False, method='dense').astype('Int64')
merged_df1['スピード指数順位'] = merged_df1.groupby('target_raceid')['スピード指数'].rank(ascending=False, method='dense').astype('Int64')
merged_df1['総合指数順位']     = merged_df1.groupby('target_raceid')['総合指数']    .rank(ascending=False, method='dense').astype('Int64')

# 33ラップの判定処理
merged_df1['33ラップ-2_差分'] = abs(merged_df1['33ラップ'] - merged_df1['33ラップ-2_33lap'])
merged_df1['33ラップ-1_差分'] = abs(merged_df1['33ラップ'] - merged_df1['33ラップ-1_33lap'])
merged_df1['33ラップ±0_差分'] = abs(merged_df1['33ラップ'] - merged_df1['33ラップ±0_33lap'])
merged_df1['33ラップ+1_差分'] = abs(merged_df1['33ラップ'] - merged_df1['33ラップ+1_33lap'])
merged_df1['33ラップ+2_差分'] = abs(merged_df1['33ラップ'] - merged_df1['33ラップ+2_33lap'])

# 33ラップ判定用関数
def assign_label(row):
  lap_value = row['33ラップ']

  # 33ラップが欠損なら何も判定しない
  if pd.isna(lap_value):
    return np.nan

  # 差分辞書
  diffs = {
      '33ラップ-2_差分': row['33ラップ-2_差分'],
      '33ラップ-1_差分': row['33ラップ-1_差分'],
      '33ラップ±0_差分': row['33ラップ±0_差分'],
      '33ラップ+1_差分': row['33ラップ+1_差分'],
      '33ラップ+2_差分': row['33ラップ+2_差分']
  }

  # 全部NaNならこれも判定不能にする
  if all(pd.isna(v) for v in diffs.values()):
      return np.nan

  # 最小差分キーを取得
  min_key = min(diffs, key=diffs.get)

  # キーの数値マッピング
  mapping = {
      '33ラップ-2_差分':-2,
      '33ラップ-1_差分':-1,
      '33ラップ±0_差分':0,
      '33ラップ+1_差分':1,
      '33ラップ+2_差分':2
  }

  scale = mapping[min_key]

  # 0スケールの場合
  if scale == 0:
        if lap_value < 0:
            return '持0'
        elif lap_value > 0:
            return '瞬0'
        else:
            return '総'

  # 0以外の場合
  prefix = '瞬' if lap_value > 0 else '持'
  if scale > 0:
      suffix = f'+{scale}'
  else:
      suffix = f'{scale}'

  return prefix + suffix

# 33ラップを判定
merged_df1['33ラップ判定'] = merged_df1.apply(assign_label, axis=1)

# ラベル書き換え用のマッピング辞書
label_mapping = {
    '持-2': '0T持-2',
    '持-1': '0T持-1',
    '持0': '0U持0',
    '持+1': '0V持+1',
    '持+2': '0V持+2',
    '瞬-2': '0S瞬-2',
    '瞬-1': '0S瞬-1',
    '瞬0': '0R瞬0',
    '瞬+1': '0Q瞬+1',
    '瞬+2': '0Q瞬+2',
    '総': '02総'
}

# 書き換え
merged_df1['レース印２'] = merged_df1['33ラップ判定'].replace(label_mapping)

# レース強度指数の計算
merged_df1['前半負荷'] = merged_df1['通過3F_stdtime1'].fillna(merged_df1['通過3F_stdtime2']) - merged_df1['通過3F']
merged_df1['中盤1差'] = merged_df1['中盤ラップ1_stdtime1'].fillna(merged_df1['中盤ラップ1_stdtime2']) - merged_df1['中盤ラップ1']
merged_df1['中盤2差'] = merged_df1['中盤ラップ2_stdtime1'].fillna(merged_df1['中盤ラップ2_stdtime2']) - merged_df1['中盤ラップ2']
merged_df1['中盤負荷'] = merged_df1[['中盤1差', '中盤2差']].mean(axis=1, skipna=True)
merged_df1['終盤負荷'] = merged_df1['レース上り3F_stdtime1'].fillna(merged_df1['レース上り3F_stdtime2']) - merged_df1['レース上り3F']
merged_df1['加速度負荷'] = merged_df1['最大加速ラップ_stdtime1'].fillna(merged_df1['最大加速ラップ_stdtime2']) - merged_df1['最大加速ラップ']
merged_df1['ゴール前負荷'] = merged_df1['ゴール前ラップ差_stdtime1'].fillna(merged_df1['ゴール前ラップ差_stdtime2']) - merged_df1['ゴール前ラップ差']
merged_df1['馬場補正値'] = merged_df1['馬場指数'].fillna(0) * 0.1

merged_df1['レース強度指数'] = (
    100
    + merged_df1['前半負荷'].fillna(0)
    + merged_df1['中盤負荷'].fillna(0)
    + merged_df1['終盤負荷'].fillna(0)
    + merged_df1['加速度負荷'].fillna(0)
    + merged_df1['ゴール前負荷'].fillna(0)
    + merged_df1['馬場補正値'].fillna(0)
)

# サフィックス付き列を削除して保存
# 削除するサフィックスを列挙
suffixes_to_drop = ('_1着','_stdtime1', '_stdtime2', '_paceindex', '_33lap', '_centralscore1', '_centralscore2','_差分')

# サフィックスで判定して残す列だけ抜き出し
clean_df = merged_df1.loc[:, [c for c in merged_df1.columns
                              if not any(c.endswith(suf) for suf in suffixes_to_drop)]].copy()

# インポート用ファイルの定義（レース単位）
import_columns = ['target_raceid', 'レース印２']
import_df = clean_df[import_columns].copy()

# target_raceid で重複削除（最後の値を残す）
import_df = import_df.drop_duplicates(subset='target_raceid', keep='last')

# year列を追加（target_raceid の先頭4桁）
import_df['year'] = pd.to_datetime(
    import_df['target_raceid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

# 成績データを上書き保存
clean_df.to_csv(master_filepath, index=False, encoding='cp932')

# レース印２のインポート用まとめファイル
import_df.to_csv(output_filepath1, index=False, encoding='cp932')

# 年度別ファイルに分割して保存
for year, df_y in import_df.groupby('year'):
    df_y.drop(columns='year').to_csv(
        rf"D:\Keiba\00_ImportData\160_33Lap_Category\33Lap_Category_Master_{int(year)}.csv",
        index=False,
        encoding='cp932'
    )

C:\Users\rotte\AppData\Local\Temp\ipykernel_19984\171397495.py:33: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(master_filepath, encoding='CP932').copy()


## ・基準タイムファイルへ列追加1

In [35]:
import pandas as pd
import numpy as np
import re
import os

# 参照用成績データファイルのパス
master_filipath = r'D:\Keiba\10_ExportData\Race_Result_Master.csv'

# 更新ファイルのパス
output_filepath1 = r'D:\Keiba\20_IndexData\Base_Time_Master1.csv'
output_filepath2 = r'D:\Keiba\20_IndexData\Base_Time_Master2.csv'

# ファイルを開く
df1 = pd.read_csv(master_filipath, encoding='CP932').copy()

# --- 集計設定 ---
agg_cols1 = {
    'レースPCI': 'median',
    'PCI3': 'median',
    'PCI': 'median',
    '通過3F': 'median',
    '通過4F': 'median',
    '通過5F': 'median',
    'レース上り3F': 'median',
    'レース上り4F': 'median',
    'レース上り5F': 'median',
    '中盤ラップ1': 'median',
    '中盤ラップ2': 'median',
    '33ラップ': 'median',
    '最大加速ラップ': 'median',
    'ゴール前ラップ差': 'median',
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'レイティング順位':'median',
    'マイニング': 'median',
    'マイニング順位': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング順位': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '上り3F': 'median',
    '上り3F順位': 'median',
    'Ave-3F': 'median',
    '-3F差': 'median',
    'テン指数': 'median',
    'テン指数順位': 'median',
    '上り指数': 'median',
    '上り指数順位': 'median',
    'スピード指数': 'median',
    'スピード指数順位': 'median',
    '総合指数': 'median',
    '総合指数順位': 'median',
    'レース強度指数': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '1角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}

agg_cols2 = {
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'レイティング順位':'median',
    'マイニング': 'median',
    'マイニング順位': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング順位': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '上り3F': 'median',
    '上り3F順位': 'median',
    'Ave-3F': 'median',
    '-3F差': 'median',
    'テン指数': 'median',
    'テン指数順位': 'median',
    '上り指数': 'median',
    '上り指数順位': 'median',
    'スピード指数': 'median',
    'スピード指数順位': 'median',
    '総合指数': 'median',
    '総合指数順位': 'median',
    'レース強度指数': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '1角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}

# --- 基準タイム算出処理1 ---
# 1勝クラス・2勝クラスのみ対象
base_df1 = df1[df1['クラスコード'].isin([23, 43])].copy()

# 1着～3着と1着馬のデータ抽出
df_tops1 = base_df1[base_df1['入線順位'].between(1, 3)].copy()
df_1st1 = base_df1[base_df1['入線順位'] == 1].copy()

# クラス分類の統一
df_tops1.loc[:, 'クラス分類'] = '1勝・2勝クラス'
df_1st1.loc[:, 'クラス分類'] = '1勝・2勝クラス'

# グループ化キー
group_cols1 = ['場所', '芝・ダート', '距離','トラックコード(JV)', '馬場分類', 'クラス分類']

# ---集計対象列（存在するものだけ）---
num_cols1 = [c for c in agg_cols1.keys() if c in df_tops1.columns]
num_cols1_1st = [c for c in agg_cols2.keys() if c in df_1st1.columns]

# ---数値化：変換失敗は NaN に---
df_tops1[num_cols1] = df_tops1[num_cols1].apply(pd.to_numeric, errors='coerce')
df_1st1[num_cols1_1st] = df_1st1[num_cols1_1st].apply(pd.to_numeric, errors='coerce')

# --- 集計処理 ---
base_time_tops1 = df_tops1.groupby(group_cols1, dropna=False).agg(agg_cols1).reset_index()
base_time_1st1 = df_1st1.groupby(group_cols1, dropna=False).agg(agg_cols2).reset_index()
base_time_1st1 = base_time_1st1.rename(columns={col: f"{col}_1着" for col in agg_cols2.keys()})

# --- 結合 ---
base_time_df1 = pd.merge(base_time_tops1, base_time_1st1, on=group_cols1, how="left")

# --- 距離係数（速度換算） ---
base_time_df1['距離係数'] = (1 / base_time_df1['タイムS']) * 100

# --- 基準タイム算出処理2 ---
# 1着～3着と1着馬のデータ抽出
df_tops2 = df1[df1['入線順位'].between(1, 3)].copy()
df_1st2 = df1[df1['入線順位'] == 1].copy()

# グループ化キー
group_cols2 = ['場所', '芝・ダート', '距離','トラックコード(JV)', '競走種別', 'クラス分類', '馬場分類']

base_time_tops2 = df_tops2.groupby(group_cols2).agg(agg_cols1).reset_index()
base_time_1st2 = df_1st2.groupby(group_cols2).agg(agg_cols2).reset_index()
base_time_1st2 = base_time_1st2.rename(columns={col: f"{col}_1着" for col in agg_cols2.keys()})
base_time_df2 = pd.merge(base_time_tops2, base_time_1st2, on=group_cols2, how="left")

# CSV出力(基準タイムファイルの更新)
base_time_df1.to_csv(output_filepath1, index=False, encoding='cp932')
base_time_df2.to_csv(output_filepath2, index=False, encoding='cp932')

C:\Users\rotte\AppData\Local\Temp\ipykernel_19984\868928031.py:14: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(master_filipath, encoding='CP932').copy()


## レースレベル指数の算出

In [36]:
import pandas as pd
import numpy as np
import re
import os

# 指数追加用成績データファイルのパス
master_filepath = r'D:\Keiba\10_ExportData\Race_Result_Master.csv'

# 参照用基準タイムファイルのパス
ref_filepath1 = r'D:\Keiba\20_IndexData\Base_Time_Master1.csv'
ref_filepath2 = r'D:\Keiba\20_IndexData\Base_Time_Master2.csv'

#  共通関数：参照側DFのキー以外の列にサフィックスを付ける
def prepare_ref_df(df_ref, merge_keys, suffix):
    """
    マージ用の参照DFの列名に suffix を付ける（キー列以外すべて）
    """
    df_ref = df_ref.copy()
    rename_map = {
        c: f"{c}{suffix}"
        for c in df_ref.columns
        if c not in merge_keys
    }
    df_ref = df_ref.rename(columns=rename_map)
    return df_ref

# ファイルを開く
df1 = pd.read_csv(master_filepath, encoding='CP932').copy()
df2 = pd.read_csv(ref_filepath1, encoding='CP932').copy()
df3 = pd.read_csv(ref_filepath2, encoding='CP932').copy()

# マージキー
merge_key1 = ['場所','芝・ダート','距離','トラックコード(JV)','馬場分類']
merge_key2 = ['場所','芝・ダート','距離','トラックコード(JV)','競走種別','クラス分類','馬場分類']

# 参照側の列名にsuffixを付けてからマージ
df2_pre = prepare_ref_df(df2, merge_key1, '_stdtime1')
df3_pre = prepare_ref_df(df3, merge_key2, '_stdtime2')

# df1へその他データフレームをマージ
merged_df1 = pd.merge(df1, df2_pre, on=merge_key1, how='left')
merged_df1 = pd.merge(merged_df1, df3_pre, on=merge_key2, how='left')

# レースレベル指数の算出
rating_base = merged_df1['レイティング_stdtime2'].fillna(merged_df1['レイティング_stdtime1'])
speed_base  = merged_df1['総合指数_stdtime2'].fillna(merged_df1['総合指数_stdtime1'])
strength_base = merged_df1['レース強度指数_stdtime2'].fillna(merged_df1['レース強度指数_stdtime1'])

rating_diff = (merged_df1['レイティング平均値'] - rating_base).fillna(0)
speed_diff  = (merged_df1['Top3総合指数'] - speed_base).fillna(0)
strength_diff = (merged_df1['レース強度指数'] - strength_base).fillna(0)

merged_df1['レースレベル指数'] = (rating_diff + speed_diff + strength_diff + 100).round(1)

# サフィックス付き列を削除して保存
# 削除するサフィックスを列挙
suffixes_to_drop = ('_1着','_stdtime1', '_stdtime2')

# サフィックスで判定して残す列だけ抜き出し
clean_df = merged_df1.loc[:, [c for c in merged_df1.columns
                              if not any(c.endswith(suf) for suf in suffixes_to_drop)]].copy()

# csvファイル保存
clean_df.to_csv(master_filepath, index=False, encoding='cp932')

C:\Users\rotte\AppData\Local\Temp\ipykernel_19984\2702594063.py:28: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(master_filepath, encoding='CP932').copy()


## レースレベル基準ファイルの作成

In [37]:
import pandas as pd
import numpy as np

# 成績マスタの読み込み（レースレベル指数まで追加済みの状態）
master_filepath = r'D:\Keiba\10_ExportData\Race_Result_Master.csv'
df1 = pd.read_csv(master_filepath, encoding='cp932').copy()

# レースレベル指数を数値にしておく（念のため）
df1['レースレベル指数'] = pd.to_numeric(df1['レースレベル指数'], errors='coerce')

# 分類キー
group_key = ['場所','芝・ダート','距離','トラックコード(JV)','競走種別','クラス分類']

# --- 基準レースレベル算出処理 ---
racelevel_df = (
    df1
      .dropna(subset=['レースレベル指数'])  # 欠損を除外
      .groupby(group_key, dropna=False)['レースレベル指数']
      .agg(['mean', 'std'])  # mean と std を同時に集計
      .reset_index()
).copy()

# 近似Zスコア用の基準値を計算
racelevel_df['RL±0'] = racelevel_df['mean'].round(2)
racelevel_df['RL+1'] = (racelevel_df['mean'] + racelevel_df['std']).round(2)
racelevel_df['RL-1'] = (racelevel_df['mean'] - racelevel_df['std']).round(2)
racelevel_df['RL+2'] = (racelevel_df['mean'] + 2 * racelevel_df['std']).round(2)
racelevel_df['RL-2'] = (racelevel_df['mean'] - 2 * racelevel_df['std']).round(2)

# 必要なカラムだけ残す
racelevel_df = racelevel_df[
    group_key + ['RL-2','RL-1','RL±0','RL+1','RL+2','mean','std']
].copy()

# csvファイル保存
output_filepath_level = r'D:\Keiba\20_IndexData\Race_Level_Master.csv'
racelevel_df.to_csv(output_filepath_level, index=False, encoding='cp932')


C:\Users\rotte\AppData\Local\Temp\ipykernel_19984\1610236460.py:6: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(master_filepath, encoding='cp932').copy()


## レースレベル判定ラベルを追加するコード

In [38]:
import pandas as pd
import numpy as np
import re
import os

# 指数追加用成績データファイルのパス
master_filepath = r'D:\Keiba\10_ExportData\Race_Result_Master.csv'

# 参照用基準タイムファイルのパス
ref_filepath1 = r'D:\Keiba\20_IndexData\Race_Level_Master.csv'

# 派生マスタcsvファイルのパス
output_filepath1 = r'D:\Keiba\00_ImportData\200_Race_Level\Race_Level_Master.csv'

# 共通関数：参照側DFのキー以外の列にサフィックスを付ける
def prepare_ref_df(df_ref, merge_keys, suffix: str):
    """
    マージ用の参照DFの列名に suffix を付ける（キー列以外すべて）
    """
    df_ref = df_ref.copy()
    rename_map = {
        c: f"{c}{suffix}"
        for c in df_ref.columns
        if c not in merge_keys
    }
    return df_ref.rename(columns=rename_map)

# ファイルを開く
df1 = pd.read_csv(master_filepath, encoding='CP932').copy()
df2 = pd.read_csv(ref_filepath1, encoding='CP932').copy()

# マージキー
group_key = ['場所','芝・ダート','距離','トラックコード(JV)','競走種別','クラス分類']

# df2 側にサフィックスを付けてからマージ
df2_pre = prepare_ref_df(df2, group_key, '_racelevel')

merged_df1 = pd.merge(df1, df2_pre, on=group_key, how='left')

merged_df1['RL-2_差分'] = (merged_df1['レースレベル指数'] - merged_df1['RL-2_racelevel']).abs()
merged_df1['RL-1_差分'] = (merged_df1['レースレベル指数'] - merged_df1['RL-1_racelevel']).abs()
merged_df1['RL±0_差分'] = (merged_df1['レースレベル指数'] - merged_df1['RL±0_racelevel']).abs()
merged_df1['RL+1_差分'] = (merged_df1['レースレベル指数'] - merged_df1['RL+1_racelevel']).abs()
merged_df1['RL+2_差分'] = (merged_df1['レースレベル指数'] - merged_df1['RL+2_racelevel']).abs()

def assign_level_band(row):
    value = row['レースレベル指数']

    # 欠損は評価不能
    if pd.isna(value):
        return np.nan

    diffs = {
        'RL-2_差分': row['RL-2_差分'],
        'RL-1_差分': row['RL-1_差分'],
        'RL±0_差分': row['RL±0_差分'],
        'RL+1_差分': row['RL+1_差分'],
        'RL+2_差分': row['RL+2_差分'],
    }

    # 全部 NaN なら判定不能
    if all(pd.isna(v) for v in diffs.values()):
        return np.nan

    # 一番差分が小さいキーを取る
    min_key = min(diffs, key=diffs.get)

    mapping = {
        'RL-2_差分': -2,
        'RL-1_差分': -1,
        'RL±0_差分':  0,
        'RL+1_差分':  1,
        'RL+2_差分':  2,
    }
    return mapping[min_key]

# -2 ~ +2 のレースレベル判定
merged_df1['レースレベル判定'] = merged_df1.apply(assign_level_band, axis=1)

band_to_grade = {
    2: 'A',
    1: 'B',
    0: 'C',
    -1: 'D',
    -2: 'E'
}

merged_df1['レースレベル評価'] = merged_df1['レースレベル判定'].map(band_to_grade)

# 書き換え用のラベル定義
rank_to_label = {'A': '05A', 'B': '07B', 'C': '01C', 'D': '00D', 'E': '03E'}

# インポート用のレースレベル列を書き換え
merged_df1['レース印３'] = merged_df1['レースレベル評価'].replace(rank_to_label)

# サフィックス付き列を削除して保存
# 削除するサフィックスを列挙
suffixes_to_drop = ('_racelevel', '_差分')

# サフィックスで判定して残す列だけ抜き出し
clean_df = merged_df1.loc[:, [c for c in merged_df1.columns
                              if not any(c.endswith(suf) for suf in suffixes_to_drop)]].copy()

# インポート用ファイルの定義（レース単位に変更）
import_columns = ['target_raceid', 'レース印３']
import_df = clean_df[import_columns].copy()

# target_raceid で重複削除（レース単位に1行だけ）
import_df = import_df.drop_duplicates(subset='target_raceid', keep='last')

# 年度列を追加（target_raceid先頭4桁が年）
import_df['year'] = pd.to_datetime(
    import_df['target_raceid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

# 成績データの更新保存
clean_df.to_csv(master_filepath, index=False, encoding='cp932')

# インポート用まとめファイルの保存
import_df.to_csv(output_filepath1, index=False, encoding='cp932')

# 年度別にファイル分割
for year, df in import_df.groupby('year'):
    df.drop(columns='year').to_csv(
        rf"D:\Keiba\00_ImportData\200_Race_Level\Race_Level_Master_{year}.csv",
        index=False, encoding='cp932'
    )


C:\Users\rotte\AppData\Local\Temp\ipykernel_19984\3334226599.py:29: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(master_filepath, encoding='CP932').copy()


## ・基準タイムファイルへ列追加2

In [41]:
import pandas as pd
import numpy as np
import re
import os

# 参照用成績データファイルのパス
master_filipath = r'D:\Keiba\10_ExportData\Race_Result_Master.csv'

# 更新ファイルのパス
output_filepath1 = r'D:\Keiba\20_IndexData\Base_Time_Master1.csv'
output_filepath2 = r'D:\Keiba\20_IndexData\Base_Time_Master2.csv'

# ファイルを開く
df1 = pd.read_csv(master_filipath, encoding='CP932').copy()

# --- 集計設定 ---
agg_cols1 = {
    'レースPCI': 'median',
    'PCI3': 'median',
    'PCI': 'median',
    '通過3F': 'median',
    '通過4F': 'median',
    '通過5F': 'median',
    'レース上り3F': 'median',
    'レース上り4F': 'median',
    'レース上り5F': 'median',
    '中盤ラップ1': 'median',
    '中盤ラップ2': 'median',
    '33ラップ': 'median',
    '最大加速ラップ': 'median',
    'ゴール前ラップ差': 'median',
    'レースレベル指数': 'median',
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'レイティング順位':'median',
    'マイニング': 'median',
    'マイニング順位': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング順位': 'median',
    '追切指数': 'median',
    '追切指数順位': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '上り3F': 'median',
    '上り3F順位': 'median',
    'Ave-3F': 'median',
    '-3F差': 'median',
    'テン指数': 'median',
    'テン指数順位': 'median',
    '上り指数': 'median',
    '上り指数順位': 'median',
    'スピード指数': 'median',
    'スピード指数順位': 'median',
    '総合指数': 'median',
    '総合指数順位': 'median',
    'レース強度指数': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '1角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}

agg_cols2 = {
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'レイティング順位':'median',
    'マイニング': 'median',
    'マイニング順位': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング順位': 'median',
    '追切指数': 'median',
    '追切指数順位': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '上り3F': 'median',
    '上り3F順位': 'median',
    'Ave-3F': 'median',
    '-3F差': 'median',
    'テン指数': 'median',
    'テン指数順位': 'median',
    '上り指数': 'median',
    '上り指数順位': 'median',
    'スピード指数': 'median',
    'スピード指数順位': 'median',
    '総合指数': 'median',
    '総合指数順位': 'median',
    'レース強度指数': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '1角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}

# --- 基準タイム算出処理1 ---
# 1勝クラス・2勝クラスのみ対象
base_df1 = df1[df1['クラスコード'].isin([23, 43])].copy()

# 1着～3着と1着馬のデータ抽出
df_tops1 = base_df1[base_df1['入線順位'].between(1, 3)].copy()
df_1st1 = base_df1[base_df1['入線順位'] == 1].copy()

# クラス分類の統一
df_tops1.loc[:, 'クラス分類'] = '1勝・2勝クラス'
df_1st1.loc[:, 'クラス分類'] = '1勝・2勝クラス'

# グループ化キー
group_cols1 = ['場所', '芝・ダート', '距離','トラックコード(JV)', '馬場分類', 'クラス分類']

# ---集計対象列（存在するものだけ）---
num_cols1 = [c for c in agg_cols1.keys() if c in df_tops1.columns]
num_cols1_1st = [c for c in agg_cols2.keys() if c in df_1st1.columns]

# ---数値化：変換失敗は NaN に---
df_tops1[num_cols1] = df_tops1[num_cols1].apply(pd.to_numeric, errors='coerce')
df_1st1[num_cols1_1st] = df_1st1[num_cols1_1st].apply(pd.to_numeric, errors='coerce')

# --- 集計処理 ---
base_time_tops1 = df_tops1.groupby(group_cols1, dropna=False).agg(agg_cols1).reset_index()
base_time_1st1 = df_1st1.groupby(group_cols1, dropna=False).agg(agg_cols2).reset_index()
base_time_1st1 = base_time_1st1.rename(columns={col: f"{col}_1着" for col in agg_cols2.keys()})

# --- 結合 ---
base_time_df1 = pd.merge(base_time_tops1, base_time_1st1, on=group_cols1, how="left")

# --- 距離係数（速度換算） ---
base_time_df1['距離係数'] = (1 / base_time_df1['タイムS']) * 100

# --- 基準タイム算出処理2 ---
# 1着～3着と1着馬のデータ抽出
df_tops2 = df1[df1['入線順位'].between(1, 3)].copy()
df_1st2 = df1[df1['入線順位'] == 1].copy()

# グループ化キー
group_cols2 = ['場所', '芝・ダート', '距離','トラックコード(JV)', '競走種別', 'クラス分類', '馬場分類']

base_time_tops2 = df_tops2.groupby(group_cols2).agg(agg_cols1).reset_index()
base_time_1st2 = df_1st2.groupby(group_cols2).agg(agg_cols2).reset_index()
base_time_1st2 = base_time_1st2.rename(columns={col: f"{col}_1着" for col in agg_cols2.keys()})
base_time_df2 = pd.merge(base_time_tops2, base_time_1st2, on=group_cols2, how="left")

# CSV出力(基準タイムファイルの更新)
base_time_df1.to_csv(output_filepath1, index=False, encoding='cp932')
base_time_df2.to_csv(output_filepath2, index=False, encoding='cp932')

## ・成績データへ基準タイムファイルとの差分データを追加

In [ ]:
import pandas as pd

# 成績データのファイルパス
master_filepath = r'D:\Keiba\10_ExportData\Race_Result_Master.csv'

# 基準タイムファイルパス
ref_filepath1 = r'D:\Keiba\20_IndexData\Base_Time_Master1.csv'
ref_filepath2 = r'D:\Keiba\20_IndexData\Base_Time_Master2.csv'


# 共通関数：参照DFのキー以外の列に suffix を付ける
def prepare_ref_df(df_ref, merge_keys, suffix):
    df_ref = df_ref.copy()
    rename_map = {
        c: f"{c}{suffix}"
        for c in df_ref.columns
        if c not in merge_keys
    }
    return df_ref.rename(columns=rename_map)

# データ読み込み
df1 = pd.read_csv(master_filepath, encoding='cp932').copy()
df2 = pd.read_csv(ref_filepath1, encoding='cp932').copy()
df3 = pd.read_csv(ref_filepath2, encoding='cp932').copy()

# マージキー
merge_key1 = ['場所', '芝・ダート', '距離', 'トラックコード(JV)', '馬場分類']
merge_key2 = ['場所', '芝・ダート', '距離', 'トラックコード(JV)', '競走種別', 'クラス分類', '馬場分類']

# suffix をつけて準備
df2_pre = prepare_ref_df(df2, merge_key1, '_stdtime1')
df3_pre = prepare_ref_df(df3, merge_key2, '_stdtime2')

# マージ実行
merged_df1 = pd.merge(df1, df2_pre, on=merge_key1, how='left')
merged_df1 = pd.merge(merged_df1, df3_pre, on=merge_key2, how='left')

# 差分を辞書的に一括生成
# 指数系（大きいほど強い → 自分 - 基準）
index_like_cols = [
    'マイニング', '対戦型マイニング', 'レイティング', '追切指数',
    'テン指数', '上り指数', '補正タイム', '補9', 'スピード指数',
    '総合指数', 'レース強度指数', 'レースレベル指数'
]

# タイム系（小さいほど強い → 基準 - 自分）
time_like_cols = [
    'タイムS', '補正走破タイム', '-3Fタイム',
    '前半3F', '上り3F', 'Ave-3F'
]

# 指数：自分 - 基準
for col in index_like_cols:
    if f"{col}_stdtime1" in merged_df1:
        merged_df1[f"{col}差分1"] = merged_df1[col] - merged_df1[f"{col}_stdtime1"]
    if f"{col}_stdtime2" in merged_df1:
        merged_df1[f"{col}差分2"] = merged_df1[col] - merged_df1[f"{col}_stdtime2"]

# タイム：基準 - 自分
for col in time_like_cols:
    if f"{col}_stdtime1" in merged_df1:
        merged_df1[f"{col}差分1"] = merged_df1[f"{col}_stdtime1"] - merged_df1[col]
    if f"{col}_stdtime2" in merged_df1:
        merged_df1[f"{col}差分2"] = merged_df1[f"{col}_stdtime2"] - merged_df1[col]

# 基準タイム列（_stdtime1/_stdtime2）を削除して clean にする
suffixes_to_drop = ('_1着', '_stdtime1', '_stdtime2')

clean_df = merged_df1.loc[
    :,
    [c for c in merged_df1.columns if not any(c.endswith(suf) for suf in suffixes_to_drop)]
].copy()

# CSV 出力
clean_df.to_csv(master_filepath, index=False, encoding='cp932')


## ・特徴量を成績データへ追加

In [ ]:
import pandas as pd

#読み込みファイルパスの指定
result_filepath = r'D:\Keiba\10_ExportData\Race_Result_Master.csv'

#ファイルを開く
df = pd.read_csv(result_filepath, encoding='cp932').copy()

# '日付S'がすでにdatetime型かどうかチェックし、変換が必要な場合のみ変換を実施する
if not pd.api.types.is_datetime64_any_dtype(df['日付S']):

    # format='%Y/%m/%d' を format='%Y-%m-%d'に変更(またはformat='mixed')
    df['日付S'] = pd.to_datetime(df['日付S'].str.replace('.', '/'), format='%Y/%m/%d')

# 過去 n 走特徴量の生成
sort_keys = ['血統登録番号', '日付S', 'target_raceid', 'target_horseid']
df = df.sort_values(sort_keys)

metrics = [
    'レイティング', 'テン指数', 'スピード指数', 'マイニング', '追切指数',
    '総合指数', 'Top3総合指数', 'レース強度指数', 'レースレベル指数',
    'PCI', 'ペース補正値', '賞金', '入線順位'
]

windows = [1, 5, 7]

for n in windows:
    shifted = df.groupby('血統登録番号')[metrics].shift(1)
    rolled = (
        shifted
        .groupby(df['血統登録番号'])
        .rolling(window=n, min_periods=1)
        .agg(['mean', 'max', 'min', 'std'])
    )
    rolled.columns = [f'past{n}_{col}_{stat}' for col, stat in rolled.columns]
    df = pd.concat([df, rolled.reset_index(level=0, drop=True)], axis=1)

In [ ]:
import pandas as pd
import numpy as np
from tkinter import Tk, filedialog
from IPython.display import display
from datetime import datetime

# ファイル選択ダイアログを表示するための設定
root = Tk()
root.withdraw()  # メインウィンドウを非表示にする

# 成績CSVファイルのパスを選択
file_path = filedialog.askopenfilename(title="成績CSVファイルを選択してください", filetypes=[("CSV Files", "*.csv")])
if not file_path:
    print("成績CSVファイルが選択されなかったため、処理を終了します。")
    exit()

# 1. 成績CSVファイルを開く
df = pd.read_csv(file_path, low_memory=False)

# カラム名の辞書
column_mappings = {
    'Location': "場所",
    'Turf_Dirt': "芝・ダート",
    'Distance': "距離",
    'Race_Type': "競争種別コード",
    'Class_Code': "クラスコード",
    'Track_Condition': "馬場状態",
    'RPCI': "レースペースチェンジ指数",
    'PCI3': "上位馬ペースチェンジ指数",
    '33_Lap': "33ラップ",
    'Middle_Lap1': "中盤ラップ1",
    'Middle_Lap2': "中盤ラップ2",
    '1st_Horse_Adjustment': '1着馬補正タイム',
    '2nd_Horse_Adjustment': '2着馬補正タイム',
    '3rd_Horse_Adjustment': '3着馬補正タイム',
    '1st_Horse_Additional9': '1着馬補9',
    '2nd_Horse_Additional9': '2着馬補9',
    '3rd_Horse_Additional9': '3着馬補9',
    '1st_Horse_Speed_Index': '1着馬スピード指数',
    '2nd_Horse_Speed_Index': '2着馬スピード指数',
    '3rd_Horse_Speed_Index': '3着馬スピード指数',
    '1st_Horse_Finish_Time': "1着馬走破タイム",
    '2nd_Horse_Finish_Time': "2着馬走破タイム",
    '3rd_Horse_Finish_Time': "3着馬走破タイム",
    '1st_Horse_PCI': "1着馬ペースチェンジ指数",
    '2nd_Horse_PCI': "2着馬ペースチェンジ指数",
    '3rd_Horse_PCI': "3着馬ペースチェンジ指数",
    '1st_Horse_Average_3F': "1着馬平均3F",
    '2nd_Horse_Average_3F': "2着馬平均3F",
    '3rd_Horse_Average_3F': "3着馬平均3F",
    '1st_Horse_First_3F': "1着馬前半3F",
    '2nd_Horse_First_3F': "2着馬前半3F",
    '3rd_Horse_First_3F': "3着馬前半3F",
    '1st_Horse_Final_3F': "1着馬上り3F",
    '2nd_Horse_Final_3F': "2着馬上り3F",
    '3rd_Horse_Final_3F': "3着馬上り3F"
}

# 必要なカラムを抽出
selected_columns = list(column_mappings.keys())
df_selected = df[selected_columns].copy()

# 競争種別コードの分類関数
def categorize_race_type(race_type):
    if race_type == 11:
        return 'サラブレッド系2歳'
    elif race_type == 12:
        return 'サラブレッド系3歳'
    elif race_type >= 13:
        return 'サラブレッド系3歳以上'
    else:
        return np.nan

# クラスコードの分類関数
def categorize_class_code(class_code):
    if 7 <= class_code <= 15:
        return '新馬・未勝利'
    elif class_code == 23:
        return '1勝クラス'
    elif class_code == 43:
        return '2勝クラス'
    elif class_code == 67:
        return '3勝クラス'
    elif class_code >= 114:
        return 'OP・重賞'
    else:
        return np.nan

# 馬場状態の分類関数
def categorize_track_condition(condition):
    if condition in ['良', '稍']:
        return '良・稍'
    elif condition in ['重', '不']:
        return '重・不'
    else:
        return 'その他'

# 新しいカテゴリカラムを追加
df_selected['Race_Type_Category'] = df_selected['Race_Type'].apply(categorize_race_type)
df_selected['Class_Code_Category'] = df_selected['Class_Code'].apply(categorize_class_code)
df_selected['Track_Condition_Category'] = df_selected['Track_Condition'].apply(categorize_track_condition)

# 1着馬から3着馬の補正タイムの平均を計算する新しいカラムを追加
df_selected['Top3_Adjustment'] = df_selected[['1st_Horse_Adjustment', '2nd_Horse_Adjustment', '3rd_Horse_Adjustment']].mean(axis=1)

# 1着馬から3着馬の補9の平均を計算する新しいカラムを追加
df_selected['Top3_Adjustment9'] = df_selected[['1st_Horse_Additional9', '2nd_Horse_Additional9', '3rd_Horse_Additional9']].mean(axis=1)

# 1着馬から3着馬のスピード指数の平均を計算する新しいカラムを追加
df_selected['Top3_Speed_Index'] = df_selected[['1st_Horse_Speed_Index', '2nd_Horse_Speed_Index', '3rd_Horse_Speed_Index']].mean(axis=1)

# 1着馬から3着馬の走破タイムの平均を計算する新しいカラムを追加
df_selected['Top3_Finish_Time'] = df_selected[['1st_Horse_Finish_Time', '2nd_Horse_Finish_Time', '3rd_Horse_Finish_Time']].mean(axis=1)

# 1着馬から3着馬の平均3Fを計算する新しいカラムを追加
df_selected['Top3_Average_3F'] = df_selected[['1st_Horse_Average_3F', '2nd_Horse_Average_3F', '3rd_Horse_Average_3F']].mean(axis=1)

# 1着馬から3着馬の前半3Fの平均を計算する新しいカラムを追加
df_selected['Top3_First_3F'] = df_selected[['1st_Horse_First_3F', '2nd_Horse_First_3F', '3rd_Horse_First_3F']].mean(axis=1)

# 1着馬から3着馬の後半3Fの平均を計算する新しいカラムを追加
df_selected['Top3_Horse_Final_3F'] = df_selected[['1st_Horse_Final_3F', '2nd_Horse_Final_3F', '3rd_Horse_Final_3F']].mean(axis=1)

# 各指数の重み付けを指定
weights = {
    'Top3_Adjustment': 0.2,
    'Top3_Adjustment9': 0.4,
    'Top3_Speed_Index': 0.4
}

# 各指数の平均を重み付けで計算
df_selected['Unified_Score'] = (
    df_selected['Top3_Adjustment'] * weights['Top3_Adjustment'] +
    df_selected['Top3_Adjustment9'] * weights['Top3_Adjustment9'] +
    df_selected['Top3_Speed_Index'] * weights['Top3_Speed_Index']
)

# グループ化 - 「場所」、「芝・ダート」、「距離」、「馬場状態」、「競争種別カテゴリ」、「クラスカテゴリ」でグループ化
grouped1 = df_selected.groupby(['Location', 'Turf_Dirt', 'Distance', 'Track_Condition', 'Race_Type_Category', 'Class_Code_Category'])

# グループ化 - 「場所」、「芝・ダート」、「距離」、「馬場状態カテゴリ」、「競争種別カテゴリ」、「クラスカテゴリ」でグループ化
grouped2 = df_selected.groupby(['Location', 'Turf_Dirt', 'Distance', 'Track_Condition_Category', 'Race_Type_Category', 'Class_Code_Category'])

# グループ化 - 「芝・ダート」、「距離」、「競争種別カテゴリ」、「クラスカテゴリ」でグループ化
grouped3 = df_selected.groupby(['Turf_Dirt', 'Distance', 'Race_Type_Category', 'Class_Code_Category'])

# 各グループごとのタイム系カラムの中央値をまとめる
def aggregate_group(grouped):
    return grouped.agg({
        'RPCI': 'median',
        '33_Lap': 'median',
        'Middle_Lap1': 'median',
        'Middle_Lap2': 'median',
        '1st_Horse_PCI': 'median',
        'PCI3': 'median',
        '1st_Horse_Adjustment': 'median',
        'Top3_Adjustment': 'median',
        '1st_Horse_Additional9': 'median',
        'Top3_Adjustment9': 'median',
        '1st_Horse_Speed_Index': 'median',
        'Top3_Speed_Index': 'median',
        '1st_Horse_Finish_Time': 'median',
        'Top3_Finish_Time': 'median',
        '1st_Horse_Average_3F': 'median',
        'Top3_Average_3F': 'median',
        '1st_Horse_First_3F': 'median',
        'Top3_First_3F': 'median',
        '1st_Horse_Final_3F': 'median',
        'Top3_Horse_Final_3F': 'median'
    }).reset_index()

grouped_median1 = aggregate_group(grouped1)
grouped_median2 = aggregate_group(grouped2)
grouped_median3 = aggregate_group(grouped3)

# 基準タイムの中央値を計算する関数
def get_median_times(group):
    numeric_cols = group.select_dtypes(include='number').columns.tolist()
    numeric_cols = [col for col in numeric_cols if col != 'Distance']

    class_1 = group[group['Class_Code_Category'] == '1勝クラス']
    class_2 = group[group['Class_Code_Category'] == '2勝クラス']

    if not class_1.empty and not class_2.empty:
        median_1 = class_1[numeric_cols].median()
        median_2 = class_2[numeric_cols].median()
        return (median_1 + median_2) / 2

    return group[numeric_cols].median()

# グループ化して基準データを取得
standard_times = grouped_median2.groupby(['Location', 'Turf_Dirt', 'Distance', 'Track_Condition_Category']).apply(get_median_times).reset_index()

# 基準データに'Distance_Index' カラムを追加
standard_times['Distance_Index'] = (1 / standard_times['Top3_Finish_Time']) * 100

# 基準レースレベル判定用のデータフレームを作成
race_levels = grouped3['Unified_Score'].agg(['mean', 'std']).reset_index()

# スコアに基づく分類基準スコアを計算
def calculate_thresholds(row):
    mean = row['mean']
    std = row['std']
    thresholds = {
        'A': mean + 2 * std,
        'B': mean + 1 * std,
        'C': mean,  # 真ん中
        'D': mean - 1 * std,
        'E': mean - 2 * std
    }
    return pd.Series(thresholds)

thresholds_df = race_levels.apply(calculate_thresholds, axis=1)

# 結合して最終的なデータフレームを作成
race_levels = pd.concat([race_levels, thresholds_df], axis=1)

# カラム名を調整
race_levels = race_levels.rename(columns={
    'mean': 'Unified_Score',
    'A': 'A',
    'B': 'B',
    'C': 'C',
    'D': 'D',
    'E': 'E'
})

# 数値を丸める処理の関数化
def round_columns(df, exclude_cols):
    columns_to_round = df.select_dtypes(include='number').columns.tolist()
    for col in exclude_cols:
        if col in columns_to_round:
            columns_to_round.remove(col)
    df[columns_to_round] = df[columns_to_round].round(2)
    
# 数値の丸めを適用
round_columns(grouped_median1, ['Distance'])
round_columns(grouped_median2, ['Distance'])
round_columns(standard_times, ['Distance'])
round_columns(race_levels, ['Distance'])

# データフレームをcsvファイルで保存
output_path1 = f'D:/Keiba/DataTables/30_IndexFiles/00_StandardTimes1_{datetime.now().strftime("%Y%m%d")}.csv'
grouped_median1.to_csv(output_path1, index=False, encoding='utf_8_sig')
print(f"結果を{output_path1}に保存しました。")

# データフレームをcsvファイルで保存
output_path2 = f'D:/Keiba/DataTables/30_IndexFiles/00_StandardTimes2_{datetime.now().strftime("%Y%m%d")}.csv'
grouped_median2.to_csv(output_path2, index=False, encoding='utf_8_sig')
print(f"結果を{output_path2}に保存しました。")

# データフレームをcsvファイルで保存
output_path3 = f'D:/Keiba/DataTables/30_IndexFiles/00_StandardTimes3_{datetime.now().strftime("%Y%m%d")}.csv'
standard_times.to_csv(output_path3, index=False, encoding='utf_8_sig')
print(f"結果を{output_path3}に保存しました。")

# データフレームをcsvファイルで保存
output_path4 = f'D:/Keiba/DataTables/30_IndexFiles/20_RaceLevels_{datetime.now().strftime("%Y%m%d")}.csv'
race_levels.to_csv(output_path4, index=False, encoding='utf_8_sig')
print(f"結果を{output_path4}に保存しました。")

結果をD:/Keiba/DataTables/30_IndexFiles/00_StandardTimes1_20241207.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/00_StandardTimes2_20241207.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/00_StandardTimes3_20241207.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/20_RaceLevels_20241207.csvに保存しました。


# ペース係数を出力するコード

## ・スピード指数の中で用いるペース補正の算出に使うコース別のペース係数ファイルを出力する

###　処理の概要
1. **csvファイルの読み込み**
    - 'pandas'を使って、ある期間の1着馬から1.5秒以内にゴールした馬の成績が記録されたcsvファイルを読み込む。

### 2. **データの前処理**

#### a. **走破タイムの秒数への変換**
- 走破タイム（`分.秒.10分の1秒`形式）を秒数（浮動小数点数）に変換する関数`convert_time_to_seconds`を適用。

#### b. **馬場状態の分類**
- 馬場状態を「良・稍」「重・不良」「その他」の3つに分類します。

#### c. **スローorハイ関数の計算**
- 新しいカラム`スローorハイ関数`を追加し、以下の計算式で値を算出する

    スローorハイ関数 = 平均3Fタイム - 上り3Fタイム

### 3. **データのグループ化**
- データを以下の項目でグループ化
- 場所
- 芝・ダート
- 距離
- 馬場状態分類

### 4. **スローorハイ関数の範囲によるフィルタリング**
- `スローorハイ関数`の値を以下の範囲で区切り、範囲ごとにデータをフィルタリング

    (-∞, -4.6), (-4.5, -3.6), ..., (3.6, 4.5), (4.6, ∞)

- 各範囲ごとにデータフレームを作成し、リストに追加します。


### 5. **中央値の算出**
- 各グループにおいて、以下のカラムの中央値を計算します：
- `走破タイム`
- `スローorハイ関数`


### 6. **基準タイム（-0.5～0.5の範囲）を基にした補正値の計算**
- `スローorハイ関数範囲`が「-0.5～0.5」のデータを抽出し、その走破タイムの中央値を「基準タイム」として保存する。
- 基準タイムを使用して、各データの`スローorハイ関数差指数`を計算する。

    スローorハイ関数差指数 = 走破タイム - 基準タイム

### 7. **ペース補正係数の算出**
- 新しいカラム`ペース補正係数`を以下の式で計算して追加する。

    ペース補正係数 = スローorハイ関数差指数 / スローorハイ関数
    ※ただし、`スローorハイ関数 = 0`の場合は補正係数を0に設定する。

### 8. **結果のCSVファイル出力**
- 最終的に計算されたデータをCSVファイルとして保存する。
- ファイル名には日付が付与される。

## 出力
- 最終的なデータには以下のカラムが含まれる。
- 場所
- 芝・ダート
- 距離
- 馬場状態分類
- スローorハイ関数範囲
- 走破タイム（中央値）
- スローorハイ関数（中央値）
- スローorハイ関数差指数
- ペース補正係数
- 保存先：  ’D:/Keiba/DataTables/40_IndexFiles/10_PacecorrectionTimes1_YYYYMMDD.csv'



In [16]:
import pandas as pd
import numpy as np
from tkinter import Tk, filedialog
from IPython.display import display
from datetime import datetime

# ファイル選択ダイアログを表示するための設定
root = Tk()
root.withdraw()  # メインウィンドウを非表示にする

# カラム名の辞書
column_mappings = {
    '場所': "場所",
    '芝・ダ': "芝・ダート",
    '距離': "距離",
    '馬場状態': "馬場状態",
    '走破タイム': "走破タイム",
    'Ave-3F': "平均3Fタイム",
    '上り3F': "上り3Fタイム"
}

# 1. 成績CSVファイルのパスを選択して読み込み
file_path = filedialog.askopenfilename(title="ペース補正用成績CSVファイルを選択してください", filetypes=[("CSV Files", "*.csv")])
if not file_path:
    print("ペース補正用成績CSVファイルが選択されなかったため、処理を終了します。")
    exit()

df = pd.read_csv(file_path, low_memory=False, encoding='cp932')

# 必要なカラムを抽出
selected_columns = list(column_mappings.keys())
df_selected = df[selected_columns].copy()

# 追加：走破タイムを時間（秒）に変換する関数
def convert_time_to_seconds(time_str):
    if pd.isna(time_str):
        return np.nan
    parts = time_str.split(".")
    if len(parts) == 3:
        try:
            minutes, seconds, tenths = map(int, parts)
            total_seconds = minutes * 60 + seconds + tenths * 0.1
            return total_seconds
        except ValueError:
            return np.nan
    else:
        return np.nan

# 2. 走破タイムを秒数に変換
df_selected['走破タイム'] = df_selected['走破タイム'].apply(convert_time_to_seconds)

# 3. 走破タイムがNaNでないことを確認
df_selected = df_selected.dropna(subset=['走破タイム'])

# 4. スローorハイ関数用に新しいカラム 'スローorハイ関数' を追加する
df_selected['スローorハイ関数'] = df_selected['Ave-3F'] - df_selected['上り3F']

# 5. 走破タイムの異常値をIQRを使って除外する（修正箇所）
grouped_time = df_selected.groupby(['場所', '芝・ダ', '距離'])

def remove_outliers(group):
    Q1 = group['走破タイム'].quantile(0.25)
    Q3 = group['走破タイム'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # 異常値を除外
    return group[(group['走破タイム'] >= lower_bound) & (group['走破タイム'] <= upper_bound)]

df_filtered = grouped_time.apply(remove_outliers).reset_index(drop=True)

# 6. コース別にグループ化（場所、芝・ダ、距離、馬場状態分類）
grouped = df_filtered.groupby(['場所', '芝・ダ', '距離', '馬場状態'])

# 7. 各コースごとにスローorハイ関数の範囲でデータをフィルタリング
ranges = [
    (-np.inf, -4.6),
    (-4.5, -3.6),
    (-3.5, -2.6),
    (-2.5, -1.6),
    (-1.5, -0.6),
    (-0.5, 0.5),
    (0.6, 1.5),
    (1.6, 2.5),
    (2.6, 3.5),
    (3.6, 4.5),
    (4.6, np.inf)
]

# 8. 各グループごとに範囲ごとのデータフレームを作成
grouped_dataframes = []

for name, group in grouped:
    # コースごとのグループ名
    location, turf_dirt, distance, track_condition = name

    # スローorハイ関数の範囲ごとにデータをフィルタリング
    for lower, upper in ranges:
        filtered_df = group[(group['スローorハイ関数'] > lower) & (group['スローorハイ関数'] <= upper)].copy()
        if not filtered_df.empty:
            filtered_df['スローorハイ関数範囲'] = f"{lower}～{upper}"
            filtered_df['場所'] = location
            filtered_df['芝・ダ'] = turf_dirt
            filtered_df['距離'] = distance
            grouped_dataframes.append(filtered_df)

# 9. 全ての範囲ごとのデータをまとめる
result_df = pd.concat(grouped_dataframes, ignore_index=True)

grouped_medians = result_df.groupby(['場所', '芝・ダ', '距離', 'スローorハイ関数範囲']).agg({
    '走破タイム': 'median',
    'スローorハイ関数': 'median'
}).reset_index()

# 10. 各グループごとに基準タイムを計算
baseline_medians = {}

grouped_by_course = grouped_medians.groupby(['場所', '芝・ダ', '距離'])

for group_name, group_df in grouped_by_course:
    # 各グループで'-0.5～0.5'の範囲をチェック
    baseline_row = group_df[group_df['スローorハイ関数範囲'] == '-0.5～0.5']
    if not baseline_row.empty:
        # 基準タイムを設定
        baseline_median = baseline_row['走破タイム'].iloc[0]
    else:
        # 例外処理：他の範囲を使用
        range_negative = group_df[group_df['スローorハイ関数範囲'] == '-0.6～-1.5']
        range_positive = group_df[group_df['スローorハイ関数範囲'] == '0.6～1.5']
        if not range_negative.empty and not range_positive.empty:
            median_negative = range_negative['走破タイム'].median()
            median_positive = range_positive['走破タイム'].median()
            baseline_median = (median_negative + median_positive) / 2
        elif not range_negative.empty:
            baseline_median = range_negative['走破タイム'].median()
        elif not range_positive.empty:
            baseline_median = range_positive['走破タイム'].median()
        else:
            # 適切な範囲がない場合は、全データの中央値を基準タイムとする
            baseline_median = group_df['走破タイム'].median()
    # グループごとの基準タイムを保存
    baseline_medians[group_name] = baseline_median

# 11. 各行に対して基準タイムを使って補正差を計算
def calculate_difference(row):
    key = (row['場所'], row['芝・ダ'], row['距離'])
    baseline_time = baseline_medians.get(key, None)
    if baseline_time is not None:
        return row['走破タイム'] - baseline_time
    else:
        return 0  # 基準タイムがない場合は0を返す

grouped_medians['スローorハイ関数差指数'] = grouped_medians.apply(calculate_difference, axis=1)

# 12. ペース補正係数のカラムを追加する
def calculate_pace_adjustment_coefficient(row):
    pace_function = row['スローorハイ関数']
    difference_index = row['スローorハイ関数差指数']
    if pace_function != 0:
        coefficient = round(difference_index / pace_function, 2)
        # ペース補正係数の絶対値が10を超える場合、上限値を設定
        if abs(coefficient) > 10:
            return 10 * (coefficient / abs(coefficient))  # 符号を維持
        else:
            return coefficient
    else:
        return 0  # 分母がゼロの場合は0を返す

grouped_medians['ペース補正係数'] = grouped_medians.apply(calculate_pace_adjustment_coefficient, axis=1)

# データフレームをcsvファイルで保存
output_path = f'D:/Keiba/DataTables/30_IndexFiles/10_PacecorrectionTimes1_{datetime.now().strftime("%Y%m%d")}.csv'
grouped_medians.to_csv(output_path, index=False, encoding='utf_8_sig')
print(f"結果を{output_path}に保存しました。")


結果をD:/Keiba/DataTables/30_IndexFiles/10_PacecorrectionTimes1_20241201.csvに保存しました。


# 基準33ラップを出力するコード

In [3]:
import pandas as pd
import numpy as np
from tkinter import Tk, filedialog
from IPython.display import display
from datetime import datetime

# ファイル選択ダイアログを表示するための設定
root = Tk()
root.withdraw()  # メインウィンドウを非表示にする

# 成績CSVファイルのパスを選択
file_path = filedialog.askopenfilename(title="成績CSVファイルを選択してください", filetypes=[("CSV Files", "*.csv")])
if not file_path:
    print("成績CSVファイルが選択されなかったため、処理を終了します。")
    exit()

# 1. 成績CSVファイルを開く
df = pd.read_csv(file_path, low_memory=False)

# 競争種別コードの分類関数
def categorize_race_type(race_type):
    if race_type == 11:
        return 'サラブレッド系2歳'
    elif race_type == 12:
        return 'サラブレッド系3歳'
    elif race_type >= 13:
        return 'サラブレッド系3歳以上'
    else:
        return np.nan

# Target出力ファイルに競争種別とクラスコードの分類カラムを追加
df['Race_Type_Category'] = df['Race_Type'].apply(categorize_race_type)

# 必要なカラムの抽出
df = df[['Location','Turf_Dirt','Distance','Turf_Inside_Outside','33_Lap','Race_Type_Category']]

# 33_Lapの欠損値を除外
df = df.dropna(subset=['33_Lap'])

# グループ化
grouped = df.groupby(['Location', 'Turf_Dirt', 'Distance', 'Turf_Inside_Outside','Race_Type_Category'])

# 各グループの統計量を計算
grouped_stats = grouped['33_Lap'].agg(['mean', 'median', 'std']).reset_index()

# Zスコアに対応する値を計算
grouped_stats['33_Lap'] = round(grouped_stats['mean'],2)  # 平均値

# Zスコア +1, -1, +2, -2 に対応する値を計算
grouped_stats['33_Lap+1'] = round(grouped_stats['mean'] + grouped_stats['std'],2)
grouped_stats['33_Lap-1'] = round(grouped_stats['mean'] - grouped_stats['std'],2)
grouped_stats['33_Lap+2'] = round(grouped_stats['mean'] + 2 * grouped_stats['std'],2)
grouped_stats['33_Lap-2'] = round(grouped_stats['mean'] - 2 * grouped_stats['std'],2)

# 必要なカラムのみを選択
result_df = grouped_stats[['Location', 'Turf_Dirt', 'Distance','Turf_Inside_Outside','Race_Type_Category', '33_Lap-2','33_Lap-1','33_Lap', '33_Lap+1', '33_Lap+2', ]]

# データフレームをcsvファイルで保存
output_path = f'D:/Keiba/DataTables/30_IndexFiles/30_Standard33Laps_{datetime.now().strftime("%Y%m%d")}.csv'
result_df.to_csv(output_path, index=False, encoding='utf_8_sig')
print(f"結果を{output_path}に保存しました。")

結果をD:/Keiba/DataTables/30_IndexFiles/30_Standard33Laps_20241207.csvに保存しました。


# 直近追切偏差値ファイルを出力するコード

In [11]:
import pandas as pd
import numpy as np
from tkinter import Tk, filedialog
from IPython.display import display
from datetime import datetime

# ファイル選択ダイアログを表示するための設定
root = Tk()
root.withdraw()  # メインウィンドウを非表示にする

# Wコース追切CSVファイルのパスを選択
file_path1 = filedialog.askopenfilename(title="Wコース追切CSVファイルを選択してください", filetypes=[("CSV Files", "*.csv")])
if not file_path1:
    print("Wコース追切CSVファイルが選択されなかったため、処理を終了します。")
    exit()

# 1. Wコース追切CSVファイルを開く
w_traning_df = pd.read_csv(file_path1, low_memory=False, encoding='cp932')

# 必要なカラムの抽出
w_traning_df = w_traning_df[['場所', '7F', '6F', '5F', '4F', '3F', '2F', '1F']]

# カラムのリネームを行う
w_traning_df = w_traning_df.rename(columns={'場所': 'コース'})

# ラベル書き換え用のマッピング辞書
course_mapping1 = {
    '美浦': '美Ｗ',
    '栗東': 'ＣＷ'
}

# mapを用いてコース情報に書き換え
w_traning_df['コース'] = w_traning_df['コース'].map(course_mapping1)

# 坂路コース追切CSVファイルのパスを選択
file_path2 = filedialog.askopenfilename(title="坂路コース追切CSVファイルを選択してください", filetypes=[("CSV Files", "*.csv")])
if not file_path2:
    print("坂路コース追切CSVファイルが選択されなかったため、処理を終了します。")
    exit()

# 2. 坂路コース追切CSVファイルを開く
h_traning_df = pd.read_csv(file_path2, low_memory=False, encoding='cp932')

# 必要なカラムの抽出
h_traning_df = h_traning_df[['場所', 'Time1', 'Time2', 'Time3', 'Time4']]

# カラムのリネームを行う（Wコース形式に合わせる）
h_traning_df = h_traning_df.rename(columns={
    '場所': 'コース',
    'Time1': '4F',
    'Time2': '3F',
    'Time3': '2F',
    'Time4': '1F'
})

# Wコースにないカラムを補完
h_traning_df['7F'] = np.nan
h_traning_df['6F'] = np.nan
h_traning_df['5F'] = np.nan

# ラベル書き換え用のマッピング辞書
course_mapping2 = {
    '美浦': '美坂',
    '栗東': '栗坂'
}

# mapを用いてコース情報に書き換え
h_traning_df['コース'] = h_traning_df['コース'].map(course_mapping2)

# データフレームを縦に結合
combined_df = pd.concat([w_traning_df, h_traning_df], ignore_index=True)

# '坂' を含まないコースを抽出
round_course_mask = ~combined_df['コース'].str.contains('坂', na=False)

# 一時的に元の列を保存
temp_7F = combined_df.loc[round_course_mask, '7F']
temp_6F = combined_df.loc[round_course_mask, '6F']
temp_5F = combined_df.loc[round_course_mask, '5F']
temp_4F = combined_df.loc[round_course_mask, '4F']
temp_3F = combined_df.loc[round_course_mask, '3F']

# 周回コースに対して7F目～3F目を右に1列ずらす
combined_df.loc[round_course_mask, '6F'] = temp_7F
combined_df.loc[round_course_mask, '5F'] = temp_6F
combined_df.loc[round_course_mask, '4F'] = temp_5F
combined_df.loc[round_course_mask, '3F'] = temp_4F
combined_df.loc[round_course_mask, '2F'] = temp_3F

# 7F目は常に NaN に設定
combined_df.loc[round_course_mask, '7F'] = np.nan

# 7F目は常に NaN に設定
combined_df.loc[round_course_mask, '7F'] = np.nan

# タイム関連のカラム
time_columns = ['7F', '6F', '5F', '4F', '3F', '2F', '1F']

# コース別にタイム関連カラムの中央値を計算
course_median = combined_df.groupby(['コース'])[time_columns].median().reset_index()

# コース別にタイム関連カラムの標準偏差を計算
course_std = round(combined_df.groupby(['コース'])[time_columns].std().reset_index(), 2)

# 標準偏差のカラム名を変更
course_std.rename(columns={col: f'標準偏差_{col}' for col in time_columns}, inplace=True)

# 中央値データフレームに標準偏差を結合
course_median = pd.merge(course_median, course_std, on='コース', how='left')

# 上位20%のデータ平均を計算する関数
def calculate_top20times_mean(group):
    # 各列の分位点を計算
    top20_threshold = group[time_columns].quantile(0.2)

    # 上位20%のデータをフィルタリング
    top20_data = group[time_columns][group[time_columns] <= top20_threshold]

    # 平均値を計算
    return top20_data.mean()

# 上位10%のデータ平均を計算する関数
def calculate_top10times_mean(group):

    # 各列の分位点を計算
    top10_threshold = group[time_columns].quantile(0.1)

    # 上位10%のデータをフィルタリング
    top10_data = group[time_columns][group[time_columns] <= top10_threshold]

    # 平均値を計算
    return top10_data.mean()

# グループごとに計算
course_top20_mean = round(combined_df.groupby(['コース']).apply(calculate_top20times_mean), 1).reset_index()
course_top10_mean = round(combined_df.groupby(['コース']).apply(calculate_top10times_mean), 1).reset_index()

# データフレームをcsvファイルで保存
output_path1 = f'D:/Keiba/DataTables/30_IndexFiles/40_Recent_Standard_Traningdata_{datetime.now().strftime("%Y%m%d")}.csv'
course_median.to_csv(output_path1, index=False, encoding='utf_8_sig')
print(f"結果を{output_path1}に保存しました。")

output_path2 = f'D:/Keiba/DataTables/30_IndexFiles/40_Recent_Top20_Traningdata_{datetime.now().strftime("%Y%m%d")}.csv'
course_top20_mean.to_csv(output_path2, index=False, encoding='utf_8_sig')
print(f"結果を{output_path2}に保存しました。")

output_path3 = f'D:/Keiba/DataTables/30_IndexFiles/40_Recent_Top10_Traningdata_{datetime.now().strftime("%Y%m%d")}.csv'
course_top10_mean.to_csv(output_path3, index=False, encoding='utf_8_sig')
print(f"結果を{output_path3}に保存しました。")

結果をD:/Keiba/DataTables/30_IndexFiles/40_Recent_Standard_Traningdata_20241214.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/40_Recent_Top20_Traningdata_20241214.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/40_Recent_Top10_Traningdata_20241214.csvに保存しました。


# 過去追切偏差値ファイルを出力するコード

In [6]:
import pandas as pd
import numpy as np
from tkinter import Tk, filedialog
from IPython.display import display
from datetime import datetime

# ファイル選択ダイアログを表示するための設定
root = Tk()
root.withdraw()  # メインウィンドウを非表示にする

# 追切CSVファイルのパスを選択
file_path = filedialog.askopenfilename(title="追切CSVファイルを選択してください", filetypes=[("CSV Files", "*.csv")])
if not file_path:
    print("追切CSVファイルが選択されなかったため、処理を終了します。")
    exit()

# 追切CSVファイルを開く
traning_df = pd.read_csv(file_path, low_memory=False,encoding='cp932')

traning_df = traning_df.copy()

# 予想コメントのカラムを「|」で分割
split_columns = traning_df['予想コメント'].str.split('|', expand=True)
traning_df['日付'] = split_columns[0]
traning_df['コース'] = split_columns[1]
traning_df['馬場状態'] = split_columns[2]
traning_df['騎乗者'] = split_columns[3]
traning_df['タイム'] = split_columns[4]
traning_df['周回位置'] = split_columns[5]
traning_df['脚色'] = split_columns[6]

replace_map = {
    '南Ｗ': '美Ｗ',
    '南Ｄ': '美ダ',
    '南ダ': '美ダ',
    '南芝': '美芝'
}
traning_df['コース'] = traning_df['コース'].replace(replace_map)

# 7Fから1Fのカラムを np.nan で初期化
distance_columns = ['7F', '6F', '5F', '4F', '3F', '2F', '1F']
for col in distance_columns:
    traning_df[col] = np.nan  # 初期化

# 補正タイム用のカラムを追加
for col in distance_columns:
    traning_df[f'補正_{col}'] = np.nan  # 初期化

# 空文字列を NaN に変換
traning_df.replace("", pd.NA, inplace=True)

# 'コース', '馬場状態', '騎乗者' 列に欠損値がある行を削除（追切データが存在しないレコードを削除）
traning_df = traning_df.dropna(subset=['コース', '馬場状態', '騎乗者'])

# 各行のタイム情報を適切カラムに配置する処理
for index, row in traning_df.iterrows():
    times = row['タイム']
    if pd.isna(times):
        continue

    # タイムリストを取得
    time_list = times.split('-')
    time_length = len(time_list)

    # タイムを後ろから順に適切なカラムに配置
    for i in range(time_length):
        # 数値に変換できるかチェック
        try:
            traning_df.at[index, distance_columns[-(time_length - i)]] = float(time_list[i])
        except ValueError:
            # 数値に変換できない場合は NaN を代入
            traning_df.at[index, distance_columns[-(time_length - i)]] = np.nan

# 周り位置を数値型に変換
traning_df['周回位置'] = pd.to_numeric(traning_df['周回位置'], errors='coerce')

# タイム補正処理
for index, row in traning_df.iterrows():
    if not pd.isna(row['周回位置']):
        # 補正値を計算
        correction_value = (9 - row['周回位置']) * 0.1

        # 最も左に存在するタイムを探す
        valid_times = [col for col in distance_columns if not pd.isna(row[col])]
        if valid_times:
            leftmost_col = valid_times[0]
            leftmost_index = distance_columns.index(leftmost_col)

            # 全体時計に補正値を加算
            traning_df.at[index, f'補正_{leftmost_col}'] = round(row[leftmost_col] + correction_value, 1)

            # 残り区間タイムに均等割り振り
            remaining_cols = distance_columns[leftmost_index + 1:]
            if remaining_cols:
                equal_correction = correction_value / len(remaining_cols)
                for col in remaining_cols:
                    if not pd.isna(row[col]):
                        traning_df.at[index, f'補正_{col}'] = round(row[col] + equal_correction, 1)
        else:
            # 周回位置があるがタイムが存在しない場合（通常は発生しない想定）
            continue
    else:
        # 周り位置がない場合、元のタイムを補正タイムに転記
        for col in distance_columns:
            if not pd.isna(row[col]):
                traning_df.at[index, f'補正_{col}'] = row[col]

# 出力に必要なカラムを選択
format_df = traning_df[['レースID(新)', '場所', '芝・ダ', '距離', '芝(内・外)','競走種別', 'クラスコード', '入線順位',
                '調教師','調教師コード','コース', '馬場状態', '騎乗者','補正_7F', '補正_6F', '補正_5F', '補正_4F', '補正_3F', '補正_2F', '補正_1F' ,
                '周回位置', '脚色']].copy()

# '入線順位'が 0 の行を削除（競争中止した馬を除外）
format_df = format_df[format_df['入線順位'] != 0]

# 競争種別コードの分類関数
def categorize_race_type(race_type):
    if race_type == 11:
        return 'サラブレッド系2歳'
    elif race_type == 12:
        return 'サラブレッド系3歳'
    elif race_type >= 13:
        return 'サラブレッド系3歳以上'
    else:
        return np.nan

# クラスコードの分類関数
def categorize_class_code(class_code):
    if 7 <= class_code <= 15:
        return '新馬・未勝利'
    elif class_code == 23:
        return '1勝クラス'
    elif class_code == 43:
        return '2勝クラス'
    elif class_code == 67:
        return '3勝クラス'
    elif class_code >= 114:
        return 'OP・重賞'
    else:
        return np.nan

# 競争種別とクラスコードを書き換え
format_df['競走種別'] = format_df['競走種別'].apply(categorize_race_type)
format_df['クラスコード'] = format_df['クラスコード'].apply(categorize_class_code)

top3_data = format_df[format_df['入線順位'] <= 3].copy()

# タイム関連のカラム
time_columns = ['補正_7F', '補正_6F', '補正_5F', '補正_4F', '補正_3F', '補正_2F', '補正_1F']

# タイム列を数値型に変換（必要に応じてNaNを処理）
top3_data[time_columns] = top3_data[time_columns].apply(pd.to_numeric, errors='coerce')

# グループ化して各列の中央値を計算
course_median = round(top3_data.groupby(['コース','馬場状態'])[time_columns].median(),1).reset_index()
race_median = round(top3_data.groupby(['芝・ダ','距離','芝(内・外)','コース','馬場状態'])[time_columns].median(),1).reset_index()
class_median = round(top3_data.groupby(['競走種別','クラスコード','コース','馬場状態'])[time_columns].median(),1).reset_index()
trainer_median = round(top3_data.groupby(['調教師','調教師コード','コース','馬場状態'])[time_columns].median(),1).reset_index()

# タイム関連カラムの標準偏差を計算
course_std = round(top3_data.groupby(['コース','馬場状態'])[time_columns].std().reset_index(),2)
race_std = round(top3_data.groupby(['芝・ダ','距離','芝(内・外)','コース','馬場状態'])[time_columns].std().reset_index(),2)
class_std = round(top3_data.groupby(['競走種別','クラスコード','コース','馬場状態'])[time_columns].std().reset_index(),2)
trainer_std = round(top3_data.groupby(['調教師','調教師コード','コース','馬場状態'])[time_columns].std().reset_index(),2)

# 標準偏差のカラム名を変更
course_std.rename(columns={col: f'標準偏差_{col}' for col in time_columns}, inplace=True)
race_std.rename(columns={col: f'標準偏差_{col}' for col in time_columns}, inplace=True)
class_std.rename(columns={col: f'標準偏差_{col}' for col in time_columns}, inplace=True)
trainer_std.rename(columns={col: f'標準偏差_{col}' for col in time_columns}, inplace=True)

# 中央値データフレームに標準偏差を結合
course_median = pd.merge(course_median, course_std, on=['コース','馬場状態'], how='left')
race_median = pd.merge(race_median, race_std, on=['芝・ダ','距離','芝(内・外)','コース','馬場状態'], how='left')
class_median = pd.merge(class_median, class_std, on=['競走種別','クラスコード','コース','馬場状態'], how='left')
trainer_median = pd.merge(trainer_median, trainer_std, on=['調教師','調教師コード','コース','馬場状態'], how='left')

# 上位20%のデータ平均を計算する関数
def calculate_top20times_mean(group):
    # 各列の分位点を計算
    top20_threshold = group[time_columns].quantile(0.2)

    # 上位20%のデータをフィルタリング
    top20_data = group[time_columns][group[time_columns] <= top20_threshold]

    # 平均値を計算
    return top20_data.mean()

# 上位10%のデータ平均を計算する関数
def calculate_top10times_mean(group):

    # 各列の分位点を計算
    top10_threshold = group[time_columns].quantile(0.1)

    # 上位10%のデータをフィルタリング
    top10_data = group[time_columns][group[time_columns] <= top10_threshold]

    # 平均値を計算
    return top10_data.mean()

# グループごとに計算
course_top20_mean = round(top3_data.groupby(['コース', '馬場状態']).apply(calculate_top20times_mean),1).reset_index()
course_top10_mean = round(top3_data.groupby(['コース', '馬場状態']).apply(calculate_top10times_mean),1).reset_index()

race_top20_mean = round(top3_data.groupby(['芝・ダ', '距離','芝(内・外)', 'コース', '馬場状態']).apply(calculate_top20times_mean),1).reset_index()
race_top10_mean = round(top3_data.groupby(['芝・ダ', '距離','芝(内・外)', 'コース', '馬場状態']).apply(calculate_top10times_mean),1).reset_index()

class_top20_mean = round(top3_data.groupby(['競走種別', 'クラスコード', 'コース', '馬場状態']).apply(calculate_top20times_mean),1).reset_index()
class_top10_mean = round(top3_data.groupby(['競走種別', 'クラスコード', 'コース', '馬場状態']).apply(calculate_top10times_mean),1).reset_index()

trainer_top20_mean = round(top3_data.groupby(['調教師','調教師コード', 'コース', '馬場状態']).apply(calculate_top20times_mean),1).reset_index()
trainer_top10_mean = round(top3_data.groupby(['調教師','調教師コード', 'コース', '馬場状態']).apply(calculate_top10times_mean),1).reset_index()

# データフレームをcsvファイルで保存
output_path1 = f'D:/Keiba/DataTables/30_IndexFiles/41_course_median_Traningdata_{datetime.now().strftime("%Y%m%d")}.csv'
course_median.to_csv(output_path1, index=False, encoding='utf_8_sig')
print(f"結果を{output_path1}に保存しました。")

output_path2 = f'D:/Keiba/DataTables/30_IndexFiles/41_course_top20_Traningdata_{datetime.now().strftime("%Y%m%d")}.csv'
course_top20_mean.to_csv(output_path2, index=False, encoding='utf_8_sig')
print(f"結果を{output_path2}に保存しました。")

output_path3 = f'D:/Keiba/DataTables/30_IndexFiles/41_course_top10_Traningdata_{datetime.now().strftime("%Y%m%d")}.csv'
course_top10_mean.to_csv(output_path3, index=False, encoding='utf_8_sig')
print(f"結果を{output_path3}に保存しました。")

output_path4 = f'D:/Keiba/DataTables/30_IndexFiles/42_race_median_Traningdata_{datetime.now().strftime("%Y%m%d")}.csv'
race_median.to_csv(output_path4, index=False, encoding='utf_8_sig')
print(f"結果を{output_path4}に保存しました。")

output_path5 = f'D:/Keiba/DataTables/30_IndexFiles/42_race_top20_Traningdata_{datetime.now().strftime("%Y%m%d")}.csv'
race_top20_mean.to_csv(output_path5, index=False, encoding='utf_8_sig')
print(f"結果を{output_path5}に保存しました。")

output_path6 = f'D:/Keiba/DataTables/30_IndexFiles/42_race_top10_Traningdata_{datetime.now().strftime("%Y%m%d")}.csv'
race_top10_mean.to_csv(output_path6, index=False, encoding='utf_8_sig')
print(f"結果を{output_path6}に保存しました。")

output_path7 = f'D:/Keiba/DataTables/30_IndexFiles/43_class_median_Traningdata_{datetime.now().strftime("%Y%m%d")}.csv'
class_median.to_csv(output_path7, index=False, encoding='utf_8_sig')
print(f"結果を{output_path7}に保存しました。")

output_path8 = f'D:/Keiba/DataTables/30_IndexFiles/43_class_top20_Traningdata_{datetime.now().strftime("%Y%m%d")}.csv'
class_top20_mean.to_csv(output_path8, index=False, encoding='utf_8_sig')
print(f"結果を{output_path8}に保存しました。")

output_path9 = f'D:/Keiba/DataTables/30_IndexFiles/43_class_top10_Traningdata_{datetime.now().strftime("%Y%m%d")}.csv'
class_top10_mean.to_csv(output_path9, index=False, encoding='utf_8_sig')
print(f"結果を{output_path9}に保存しました。")

output_path10 = f'D:/Keiba/DataTables/30_IndexFiles/44_trainer_median_Traningdata_{datetime.now().strftime("%Y%m%d")}.csv'
trainer_median.to_csv(output_path10, index=False, encoding='utf_8_sig')
print(f"結果を{output_path10}に保存しました。")

output_path11 = f'D:/Keiba/DataTables/30_IndexFiles/44_trainer_top20_Traningdata_{datetime.now().strftime("%Y%m%d")}.csv'
trainer_top20_mean.to_csv(output_path11, index=False, encoding='utf_8_sig')
print(f"結果を{output_path11}に保存しました。")

output_path12 = f'D:/Keiba/DataTables/30_IndexFiles/44_trainer_top10_Traningdata_{datetime.now().strftime("%Y%m%d")}.csv'
trainer_top10_mean.to_csv(output_path12, index=False, encoding='utf_8_sig')
print(f"結果を{output_path12}に保存しました。")

結果をD:/Keiba/DataTables/30_IndexFiles/41_course_median_Traningdata_20241212.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/41_course_top20_Traningdata_20241212.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/41_course_top10_Traningdata_20241212.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/42_race_median_Traningdata_20241212.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/42_race_top20_Traningdata_20241212.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/42_race_top10_Traningdata_20241212.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/43_class_median_Traningdata_20241212.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/43_class_top20_Traningdata_20241212.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/43_class_top10_Traningdata_20241212.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/44_trainer_median_Traningdata_20241212.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/44_trainer_top20_Traningdata_20241212.csvに保存しました。
結果をD:/Keiba/DataTables/30_IndexFiles/44_trainer_top10_Tra

# スピード指数を出力するコード

## 概要
スピード指数を算出するためのコードを記録します。このスピード指数は競馬のレースごとに馬の成績を評価するための指標で、ペース補正、斤量補正、馬場指数、クラス補正など、複数の要素を組み合わせて算出します。

### 処理の概要

1. **CSVファイルの読み込み**
    - `pandas`を使って成績データのCSVファイルを読み込みます。このデータはレースごとの馬の成績や走破タイムなどが含まれています。

2. **データの前処理**

    a. **出走取り消し・競争除外の馬を除外**
    - 成績データから「外」「止」「消」の馬を除外します。

    b. **走破タイムの秒数への変換**
    - 走破タイム（`分.秒.10分の1秒`形式）を秒数（浮動小数点数）に変換する関数`convert_time_to_seconds`を使い、`走破タイム_秒`という新しいカラムに変換後の値を追加します。

    c. **コーナーロスの計算**
    - 各馬のコーナーでのロス距離を考慮した走破タイムの補正を行います。`calculate_corner_loss`関数を用いて、`コーナーロス`という新しいカラムにロス時間を計算して追加します。
    - 調整後の走破タイムは`調整走破タイム`というカラムに記録されます。

3. **クラスと競争種別の分類**
    - 各レースにおける競争種別コードとクラスコードをもとに分類し、新たに`Race_Type_Category`と`Class_Code_Category`カラムを追加します。

4. **基準タイムデータとのマージ**
    - コースごとの基準タイムデータ（StandardTimes2）およびクラス別基準タイムデータ（StandardTimes3）を読み込み、対象の成績データとマージします。
    - マージする際に、カラム名の整合性を取るためにリネーム処理を行います。

5. **スローorハイ関数の計算**
    - 新しいカラム`スローorハイ関数`を計算し、`Ave-3F`と`上り3F`の差を計算します。その結果を基に`スローorハイ関数範囲`を設定します。
    - `スローorハイ関数範囲`はペース補正ファイルと統一するために、あらかじめ定義した範囲で値を設定します。

6. **ペース補正ファイルとのマージ**
    - ペース補正ファイル（PacecorrectionTimes1）を読み込み、ペース補正係数を対象の成績データに結合します。

7. **各補正値の計算**

    a. **タイム補正値の計算**
    - `Top3_Finish_Time_standard`と`調整走破タイム`を基に、距離補正を考慮してタイム補正値を計算し、`タイム補正値`カラムに保存します。

    b. **平均3Fタイム補正値の計算**
    - `Top3_Average_3F`がNaNの場合、0に置き換えた上で、`Ave-3F`との差分を計算し、`Ave-3F補正値`として保存します。

    c. **斤量補正値の計算**
    - 斤量と基準斤量を用いて斤量補正を計算し、`斤量補正値`として保存します。

    d. **クラス補正値の計算**
    - `Top3_Finish_Time_standard`と`Top3_Finish_Time_class`の差を計算してクラス補正値を求めます。クラス補正値は範囲を-2から+2に収めるため、`np.clip`を使用して値を制限します。

    e. **ペース補正値の計算**
    - `スローorハイ関数`と`ペース補正係数`を乗算して、`ペース補正値`として保存します。`ペース補正係数`がNaNの場合は0に置き換えます。

8. **スピード指数の計算**
    - 各補正値を組み合わせて、最終的なスピード指数を算出します。
    - 計算式は以下の通りです：
    
      ```
      スピード指数 = タイム補正値 + Ave-3F補正値 + 馬場指数 + 斤量補正値 + クラス補正値 + ペース補正値 + 70
      ```
    - 計算後、`スピード指数`カラムに保存します。

9. **結果の保存**
    - 最終的に計算された`スピード指数`を含むデータを新たなCSVファイルとして保存します。
    - ファイル名には日付が付与され、指定のディレクトリに保存されます。

### 出力
- 出力ファイルには以下のカラムが含まれます。
  - レースID(新)
  - スピード指数
- 保存先：`D:/Keiba/Importdata/40_Speed_Score/SpeedScore_YYYYMMDD.csv`



In [24]:
import pandas as pd
import numpy as np
from tkinter import Tk, filedialog
from IPython.display import display
from datetime import datetime
import glob
import re

# ファイル選択ダイアログを表示するための設定
root = Tk()
root.withdraw()  # メインウィンドウを非表示にする

# スピード指数用成績CSVファイルのパスを選択して読み込み
file_path = filedialog.askopenfilename(title="スピード指数用成績CSVファイルを選択してください", filetypes=[("CSV Files", "*.csv")])
if not file_path:
    print("スピード指数用成績CSVファイルが選択されなかったため、処理を終了します。")
    exit()

df = pd.read_csv(file_path, low_memory=False, encoding='cp932')

# 出走取り消しや競争除外（外、止、消）になった馬を除外
df = df[~df['着順'].isin(['外', '止', '消'])]

# 走破タイムを時間（秒）に変換する関数
def convert_time_to_seconds(time_str):
    if pd.isna(time_str):
        return np.nan
    # ドットで分割する (例: "1.54.5" → ["1", "54", "5"])
    parts = time_str.split(".")
    if len(parts) == 3:
        try:
            minutes, seconds, tenths = map(int, parts)
            total_seconds = minutes * 60 + seconds + tenths * 0.1
            return total_seconds
        except ValueError:
            return np.nan
    else:
        return np.nan

# 走破タイムを秒に変換
df['走破タイム_秒'] = df['走破タイム'].apply(convert_time_to_seconds)

# コーナーロスを計算する関数
def calculate_corner_loss(row):
    corner_positions = [
        row.get('馬印4', 1) - 1,
        row.get('馬印5', 1) - 1,
        row.get('馬印6', 1) - 1,
        row.get('馬印7', 1) - 1
    ]
    total_distance_loss = sum(corner_positions) * 1.5  # 総距離ロス（m）
    finish_time_seconds = row['走破タイム_秒']
    distance_m = row['距離']

    if pd.isna(finish_time_seconds) or pd.isna(distance_m) or finish_time_seconds == 0:
        return np.nan

    avg_speed = distance_m / finish_time_seconds if finish_time_seconds > 0 else np.nan
    corner_loss = total_distance_loss / avg_speed if avg_speed > 0 else 0
    return round(corner_loss, 2)

# 馬印4～7を数値に変換し、不正な値を1に設定
for corner_col in ['馬印4', '馬印5', '馬印6', '馬印7']:
    df[corner_col] = pd.to_numeric(df[corner_col], errors='coerce').fillna(1).astype(int)

# コーナーロスを計算して新しいカラムに追加
df['コーナーロス'] = df.apply(calculate_corner_loss, axis=1)

# 調整走破タイムの計算
df['調整走破タイム'] = df['走破タイム_秒'] - df['コーナーロス']

# 馬場指数の計算
df['馬場指数'] = df['レース印１'] / 10

# 基準斤量の計算
# 斤量を数値に変換する関数
def extract_weight(weight_str):
    match = re.search(r'\d+', weight_str)
    return int(match.group()) if match else np.nan

# 斤量を数値に変換
df['斤量'] = df['斤量'].apply(extract_weight)

# 基準斤量の計算
df['基準斤量'] = df['斤量'] - df['馬齢斤量差']

# 競争種別コードの分類関数
def categorize_race_type(race_type):
    if race_type == 11:
        return 'サラブレッド系2歳'
    elif race_type == 12:
        return 'サラブレッド系3歳'
    elif race_type >= 13:
        return 'サラブレッド系3歳以上'
    else:
        return np.nan

# クラスコードの分類関数
def categorize_class_code(class_code):
    if 7 <= class_code <= 15:
        return '新馬・未勝利'
    elif class_code == 23:
        return '1勝クラス'
    elif class_code == 43:
        return '2勝クラス'
    elif class_code == 67:
        return '3勝クラス'
    elif class_code >= 114:
        return 'OP・重賞'
    else:
        return np.nan

# Target出力ファイルに競争種別とクラスコードの分類カラムを追加
df['Race_Type_Category'] = df['競走種別'].apply(categorize_race_type)
df['Class_Code_Category'] = df['クラスコード'].apply(categorize_class_code)

# スローorハイ関数の範囲を定義
ranges = [
    (-np.inf, -4.6),
    (-4.5, -3.6),
    (-3.5, -2.6),
    (-2.5, -1.6),
    (-1.5, -0.6),
    (-0.5, 0.5),
    (0.6, 1.5),
    (1.6, 2.5),
    (2.6, 3.5),
    (3.6, 4.5),
    (4.6, np.inf)
]

# スローorハイ関数のラベルを定義 (ペース補正ファイルと統一する)
labels = [
    '-inf～-4.6', '-4.5～-3.6', '-3.5～-2.6', '-2.5～-1.6', '-1.5～-0.6',
    '-0.5～0.5', '0.6～1.5', '1.6～2.5', '2.6～3.5', '3.6～4.5', '4.6～inf'
]

# Track_Condition の変換
df['Track_Condition_Category'] = df['馬場状態'].apply(lambda x: '良・稍' if x in ['良', '稍'] else '重・不' if x in ['重', '不'] else np.nan)

# スローorハイ関数を計算
df['スローorハイ関数'] = df['Ave-3F'] - df['上り3F']

# スローorハイ関数のラベルを設定
df['スローorハイ関数範囲'] = pd.cut(df['スローorハイ関数'], bins=[r[0] for r in ranges] + [ranges[-1][1]], labels=labels, right=True)

# ベースとなるデータフレーム
index_df = df[['レースID(新)',
            '場所',
            '芝・ダ',
            '距離',
            'Race_Type_Category',
            'Class_Code_Category',
            'Track_Condition_Category',
            '馬場指数',
            '基準斤量',
            '斤量',
            '走破タイム_秒',
            '調整走破タイム',
            'Ave-3F',
            '上り3F',
            'スローorハイ関数',
            'スローorハイ関数範囲'
            ]]

# コース基準タイムファイルのパスを取得（StandardTimes2_を含む最新ファイルを取得）
standard_time_file_path1 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/00_StandardTimes3_*.csv'), key=str)

# クラス別コース基準タイムファイルのパスを取得（StandardTimes3_を含む最新ファイルを取得）
standard_time_file_path2 = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/00_StandardTimes2_*.csv'), key=str)

# ペース補正ファイルのパスを取得（10_PacecorrectionTimes1_を含む最新ファイルを取得）
pace_setting_file_path = max(glob.glob(r'D:/Keiba/DataTables/30_IndexFiles/10_PacecorrectionTimes1_*.csv'), key=str)

# コース基準タイムファイルの読み込み
standard_times_df1 = pd.read_csv(standard_time_file_path1, low_memory=False)
standard_times_df2 = pd.read_csv(standard_time_file_path2, low_memory=False)

# ペース補正ファイルの読み込み
pace_setting_df = pd.read_csv(pace_setting_file_path, low_memory=False)

# マージのためのカラム名マッピング（基準タイムファイル/Target出力ファイル）
column_mappings = {
    'Location': '場所',
    'Turf_Dirt': '芝・ダ',
    'Distance': '距離',
    'Track_Condition_Category': '馬場状態',
    'Race_Type_Category':'競争種別',
    'Class_Code_Category' : 'クラスコード'
}


# 指数データフレームのカラムリネーム
index_df = index_df.rename(columns=column_mappings)

# 各基準タイムファイルとマージするためのカラムリネーム
standard_times_df1 = standard_times_df1.rename(columns=column_mappings)
standard_times_df2 = standard_times_df2.rename(columns=column_mappings)

# ペース補正データフレームのカラムリネーム
pace_setting_df = pace_setting_df.rename(columns=column_mappings)

# 基準タイムファイルをマージ
merged_df = pd.merge(
    index_df,
    standard_times_df1[['場所', '芝・ダ', '距離', '馬場状態','Top3_Average_3F', 'Top3_Finish_Time','Distance_Index']],
    on=['場所', '芝・ダ', '距離', '馬場状態'],
    how='left'
)

# クラス別基準タイムファイルをマージ
merged_df = pd.merge(
    merged_df,
    standard_times_df2[['場所', '芝・ダ', '距離', '馬場状態', '競争種別', 'クラスコード', 'Top3_Finish_Time']],
    on=['場所', '芝・ダ', '距離', '馬場状態', '競争種別', 'クラスコード'],
    how='left',
    suffixes=('_standard', '_class')
)

# ペース補正ファイルをマージ
merged_df = pd.merge(
    merged_df,
    pace_setting_df[['場所', '芝・ダ', '距離', 'スローorハイ関数範囲', 'ペース補正係数']],
    on=['場所', '芝・ダ', '距離', 'スローorハイ関数範囲'],
    how='left',
)

# タイム補正値の計算
merged_df['タイム補正値'] = (merged_df['Top3_Finish_Time_standard'] - merged_df['調整走破タイム']) * merged_df['Distance_Index']

# 平均3Fタイム補正値の計算
# 平均3FタイムがNaNの場合は0に置き換え
merged_df['Top3_Average_3F'] = merged_df['Top3_Average_3F'].fillna(0)

# 平均3Fタイム補正値の計算 (Top3_Average_3FがNaNの場合はAve-3F補正値を0にする)
merged_df['Ave-3F補正値'] = np.where(
    merged_df['Top3_Average_3F'] == 0,
    0,
    merged_df['Top3_Average_3F'] - merged_df['Ave-3F']
)

# 斤量補正値の計算
merged_df['斤量補正値'] = (merged_df['斤量'] - merged_df['基準斤量']) / merged_df['Distance_Index']

# クラス補正値の計算
merged_df['クラス補正値'] = merged_df['Top3_Finish_Time_standard'] - merged_df['Top3_Finish_Time_class']

# クラス補正値を -2 から +2 の範囲に収める
merged_df['クラス補正値'] = np.clip(merged_df['クラス補正値'], -2, 2)

# ペース補正値の計算
# ペース補正係数がNaNの場合は0に置き換え
merged_df['ペース補正係数'] = merged_df['ペース補正係数'].fillna(0)
merged_df['ペース補正値'] = merged_df['スローorハイ関数'] * merged_df['ペース補正係数']

# スピード指数の計算
merged_df['スピード指数'] = round(merged_df['タイム補正値'] + merged_df['Ave-3F補正値'] + merged_df['馬場指数'] + merged_df['斤量補正値'] + merged_df['クラス補正値'] + merged_df['ペース補正値'] + 70,1)

# スピード指数のデータフレーム
speedindex_df = merged_df[['レースID(新)','スピード指数']]

# データフレームをcsvファイルで保存
output_path = f'D:/Keiba/Importdata/40_Speed_Score/SpeedScore_{datetime.now().strftime("%Y%m%d")}.csv'
speedindex_df.to_csv(output_path, index=False, encoding='utf_8_sig')
print(f"結果を{output_path}に保存しました。")

結果をD:/Keiba/Importdata/40_Speed_Score/SpeedScore_20241201.csvに保存しました。
